# The ATM (AFL Tip-Master) v3.1

**Author:** Blake Noske

Sportbet and many other betting agencies offer several different markets to play on AFL matches. One such market is the Big Win Little Win. Outcomes in this market are as follow.
- Home team little win = Home team wins by a margin within the range of 1 to 39
- Home team big win = Home team wins by a margin greater than or equal to 40
- Draw = Margin is equal to 0
- Away team little win = Away team wins by a margin within the range of 1 to 39
- Away team big win = Awat team wins by a margin greater than or equal to 40

This market usually offers returns on over $2 except in cases where a defined outcome is almost certain. As a result of this, and algorithm only needs to have an accuracy above 0.5 to be profitable. This model aims to predict the results of this market form the perspective of the home team in each game (little win - LW, big win - BW, draw - D, little loss - LL, big loss - BL). To ensure that enough data has been gathered to enable accurate prediction this model **SHOULD NOT BE USED UNTIL ROUND 8**.

## Contents:
1. [Background](#intro)
2. [Data Clean and Prep](#clean) <br>
    2.1. [Elo Ladder](#elo)
3. [Model Training](#train)
4. [Inputs](#input)
5. [Outputs](#output)

## Background <a class="anchor" id="intro"></a>

The goal of this program is to make a profit on sport betting companies offering big win little win markets. A profit of 75% averaged over 4 rounds is to be considered ideal. This goal will allow for upset rounds to be gained back on during the course of a month.

There is limited data available publicly on AFL matches however the good people over at [AFLtables](https://afltables.com/afl/afl_index.html) produce a table of several measurement for a match showing each teams for and against stats for that round. The measurements gathered that are important to the prediction of results are:
- Kicks
- Marks
- Handballs
- Disposals
- Hit outs
- Tackles
- Rebound 50s
- Inside 50s
- Clearances
- Clangers
- Free kicks
- Contested possessions
- Uncontested possessions
- Contested marks
- Marks inside 50
- 1 percenters
- Bounces
- Goal assists

Each week this data is refreshed into an excel sheet where each game is given a unique identifier, the home and away teams are recorded, and the results from the perspective of each team are recorded (LW, BW, D, LL, BL)

Three further measurements are calculated in this program using this data:
- Inside 50 efficiency
- Rebound 50 efficiency
- Elos

The excel file is then downloaded into this program where preparation occurs to give each team their own unique table of stats for the year as well as a table of data where each game played has its own row from the perspective of the home team.

The data fed into the model is normally distributed with many variables having a mean of 0. Any model used must therefore allow for negative integers to be fed into it. Thus, the model selected for this version is a Gaussian Naive Bayes model. The benefits of this model are that it allows for each result class to be given a probability of occurrence. This is an important output when considering the odds offered for each result.

This model has been assessed for several weeks to determine its long term performance before it has been deemed ready to be published online. However I do not take any responsibility for its performance or money lost due to its usage. By using this model to inform your betting decisions you are taking full responsibility for your decisions and removing any liability associated with me.

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import markdown
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFECV
from sklearn.model_selection import StratifiedKFold

In [2]:
data = "2023 Data raw AFL v3.xlsx"

In [3]:
Ade1 = pd.read_excel(data,"Ade1")
Bri1 = pd.read_excel(data,"Bri1")
Car1 = pd.read_excel(data,"Car1")
Col1 = pd.read_excel(data,"Col1")
Ess1 = pd.read_excel(data,"Ess1")
Fre1 = pd.read_excel(data,"Fre1")
Gee1 = pd.read_excel(data,"Gee1")
Gol1 = pd.read_excel(data,"Gol1")
GWS1 = pd.read_excel(data,"GWS1")
Haw1 = pd.read_excel(data,"Haw1")
Mel1 = pd.read_excel(data,"Mel1")
Nor1 = pd.read_excel(data,"Nor1")
Por1 = pd.read_excel(data,"Por1")
Ric1 = pd.read_excel(data,"Ric1")
StK1 = pd.read_excel(data,"StK1")
Syd1 = pd.read_excel(data,"Syd1")
WCE1 = pd.read_excel(data,"WCE1")
Wes1 = pd.read_excel(data,"Wes1")
Ade2 = pd.read_excel(data,"Ade2")
Bri2 = pd.read_excel(data,"Bri2")
Car2 = pd.read_excel(data,"Car2")
Col2 = pd.read_excel(data,"Col2")
Ess2 = pd.read_excel(data,"Ess2")
Fre2 = pd.read_excel(data,"Fre2")
Gee2 = pd.read_excel(data,"Gee2")
Gol2 = pd.read_excel(data,"Gol2")
GWS2 = pd.read_excel(data,"GWS2")
Haw2 = pd.read_excel(data,"Haw2")
Mel2 = pd.read_excel(data,"Mel2")
Nor2 = pd.read_excel(data,"Nor2")
Por2 = pd.read_excel(data,"Por2")
Ric2 = pd.read_excel(data,"Ric2")
StK2 = pd.read_excel(data,"StK2")
Syd2 = pd.read_excel(data,"Syd2")
WCE2 = pd.read_excel(data,"WCE2")
Wes2 = pd.read_excel(data,"Wes2")

## Data prep and clean <a class="anchor" id="clean"></a>

The data must be cleaned from its raw state of two tables per team to a single table per team and a table of all the game stats.
Firstly, the two tables for each team will be merged.

In [4]:
Ade = pd.merge(Ade1, Ade2, left_on='#', right_on='#', how='left')
Bri = pd.merge(Bri1, Bri2, left_on='#', right_on='#', how='left')
Car = pd.merge(Car1, Car2, left_on='#', right_on='#', how='left')
Col = pd.merge(Col1, Col2, left_on='#', right_on='#', how='left')
Ess = pd.merge(Ess1, Ess2, left_on='#', right_on='#', how='left')
Fre = pd.merge(Fre1, Fre2, left_on='#', right_on='#', how='left')
Gee = pd.merge(Gee1, Gee2, left_on='#', right_on='#', how='left')
Gol = pd.merge(Gol1, Gol2, left_on='#', right_on='#', how='left')
GWS = pd.merge(GWS1, GWS2, left_on='#', right_on='#', how='left')
Haw = pd.merge(Haw1, Haw2, left_on='#', right_on='#', how='left')
Mel = pd.merge(Mel1, Mel2, left_on='#', right_on='#', how='left')
Nor = pd.merge(Nor1, Nor2, left_on='#', right_on='#', how='left')
Por = pd.merge(Por1, Por2, left_on='#', right_on='#', how='left')
Ric = pd.merge(Ric1, Ric2, left_on='#', right_on='#', how='left')
StK = pd.merge(StK1, StK2, left_on='#', right_on='#', how='left')
Syd = pd.merge(Syd1, Syd2, left_on='#', right_on='#', how='left')
WCE = pd.merge(WCE1, WCE2, left_on='#', right_on='#', how='left')
Wes = pd.merge(Wes1, Wes2, left_on='#', right_on='#', how='left')

Non-required columns are then removed and each teams table is split between game ID stats and performance stats.

In [5]:
Ade_class = pd.concat([Ade.iloc[:,0:2],Ade.iloc[:,28:30]],axis = 1)
Ade_stats = Ade.iloc[:,[3,4,5,6,9,10,11,12,13,14,17,20,21,22,23,24,25,26,27]]
Bri_class = pd.concat([Bri.iloc[:,0:2],Bri.iloc[:,28:30]],axis = 1)
Bri_stats = Bri.iloc[:,[3,4,5,6,9,10,11,12,13,14,17,20,21,22,23,24,25,26,27]]
Car_class = pd.concat([Car.iloc[:,0:2],Car.iloc[:,28:30]],axis = 1)
Car_stats = Car.iloc[:,[3,4,5,6,9,10,11,12,13,14,17,20,21,22,23,24,25,26,27]]
Col_class = pd.concat([Col.iloc[:,0:2],Col.iloc[:,28:30]],axis = 1)
Col_stats = Col.iloc[:,[3,4,5,6,9,10,11,12,13,14,17,20,21,22,23,24,25,26,27]]
Ess_class = pd.concat([Ess.iloc[:,0:2],Ess.iloc[:,28:30]],axis = 1)
Ess_stats = Ess.iloc[:,[3,4,5,6,9,10,11,12,13,14,17,20,21,22,23,24,25,26,27]]
Fre_class = pd.concat([Fre.iloc[:,0:2],Fre.iloc[:,28:30]],axis = 1)
Fre_stats = Fre.iloc[:,[3,4,5,6,9,10,11,12,13,14,17,20,21,22,23,24,25,26,27]]
Gee_class = pd.concat([Gee.iloc[:,0:2],Gee.iloc[:,28:30]],axis = 1)
Gee_stats = Gee.iloc[:,[3,4,5,6,9,10,11,12,13,14,17,20,21,22,23,24,25,26,27]]
Gol_class = pd.concat([Gol.iloc[:,0:2],Gol.iloc[:,28:30]],axis = 1)
Gol_stats = Gol.iloc[:,[3,4,5,6,9,10,11,12,13,14,17,20,21,22,23,24,25,26,27]]
GWS_class = pd.concat([GWS.iloc[:,0:2],GWS.iloc[:,28:30]],axis = 1)
GWS_stats = GWS.iloc[:,[3,4,5,6,9,10,11,12,13,14,17,20,21,22,23,24,25,26,27]]
Haw_class = pd.concat([Haw.iloc[:,0:2],Haw.iloc[:,28:30]],axis = 1)
Haw_stats = Haw.iloc[:,[3,4,5,6,9,10,11,12,13,14,17,20,21,22,23,24,25,26,27]]
Mel_class = pd.concat([Mel.iloc[:,0:2],Mel.iloc[:,28:30]],axis = 1)
Mel_stats = Mel.iloc[:,[3,4,5,6,9,10,11,12,13,14,17,20,21,22,23,24,25,26,27]]
Nor_class = pd.concat([Nor.iloc[:,0:2],Nor.iloc[:,28:30]],axis = 1)
Nor_stats = Nor.iloc[:,[3,4,5,6,9,10,11,12,13,14,17,20,21,22,23,24,25,26,27]]
Por_class = pd.concat([Por.iloc[:,0:2],Por.iloc[:,28:30]],axis = 1)
Por_stats = Por.iloc[:,[3,4,5,6,9,10,11,12,13,14,17,20,21,22,23,24,25,26,27]]
Ric_class = pd.concat([Ric.iloc[:,0:2],Ric.iloc[:,28:30]],axis = 1)
Ric_stats = Ric.iloc[:,[3,4,5,6,9,10,11,12,13,14,17,20,21,22,23,24,25,26,27]]
StK_class = pd.concat([StK.iloc[:,0:2],StK.iloc[:,28:30]],axis = 1)
StK_stats = StK.iloc[:,[3,4,5,6,9,10,11,12,13,14,17,20,21,22,23,24,25,26,27]]
Syd_class = pd.concat([Syd.iloc[:,0:2],Syd.iloc[:,28:30]],axis = 1)
Syd_stats = Syd.iloc[:,[3,4,5,6,9,10,11,12,13,14,17,20,21,22,23,24,25,26,27]]
WCE_class = pd.concat([WCE.iloc[:,0:2],WCE.iloc[:,28:30]],axis = 1)
WCE_stats = WCE.iloc[:,[3,4,5,6,9,10,11,12,13,14,17,20,21,22,23,24,25,26,27]]
Wes_class = pd.concat([Wes.iloc[:,0:2],Wes.iloc[:,28:30]],axis = 1)
Wes_stats = Wes.iloc[:,[3,4,5,6,9,10,11,12,13,14,17,20,21,22,23,24,25,26,27]]

A temporary table for each team is created to allow for the delimiter '-' to be used on each performance stat. This will allows for separate columns for for and against stats for each team.

In [6]:
Ade_stats_temp = pd.DataFrame(columns=['Kicks for','Kicks against','Marks for','Marks against','Handballs for',
                                      'Handballs against','Disposals for','Disposals against','Hitouts for',
                                       'Hitouts against','Tackles for','Tackles against','Rebound 50s for',
                                      'Rebound 50s against','Inside 50s for','Inside 50s against','Clearances for',
                                      'Clearances against','Clangers for','Clangers against','Frees for','Frees against',
                                      'Contested possessions for','Contested possessions against',
                                       'Uncontested possessions for','Uncontested possessions against',
                                       'Contested marks for','Contested marks against','Marks inside 50 for',
                                      'Marks inside 50 against','1%s for','1%s against','Bounces for','Bounces against',
                                      'Goal assists for','Goal assists against','Margin'])
Bri_stats_temp = pd.DataFrame(columns=['Kicks for','Kicks against','Marks for','Marks against','Handballs for',
                                      'Handballs against','Disposals for','Disposals against','Hitouts for',
                                       'Hitouts against','Tackles for','Tackles against','Rebound 50s for',
                                      'Rebound 50s against','Inside 50s for','Inside 50s against','Clearances for',
                                      'Clearances against','Clangers for','Clangers against','Frees for','Frees against',
                                      'Contested possessions for','Contested possessions against',
                                       'Uncontested possessions for','Uncontested possessions against',
                                       'Contested marks for','Contested marks against','Marks inside 50 for',
                                      'Marks inside 50 against','1%s for','1%s against','Bounces for','Bounces against',
                                      'Goal assists for','Goal assists against','Margin'])
Car_stats_temp = pd.DataFrame(columns=['Kicks for','Kicks against','Marks for','Marks against','Handballs for',
                                      'Handballs against','Disposals for','Disposals against','Hitouts for',
                                       'Hitouts against','Tackles for','Tackles against','Rebound 50s for',
                                      'Rebound 50s against','Inside 50s for','Inside 50s against','Clearances for',
                                      'Clearances against','Clangers for','Clangers against','Frees for','Frees against',
                                      'Contested possessions for','Contested possessions against',
                                       'Uncontested possessions for','Uncontested possessions against',
                                       'Contested marks for','Contested marks against','Marks inside 50 for',
                                      'Marks inside 50 against','1%s for','1%s against','Bounces for','Bounces against',
                                      'Goal assists for','Goal assists against','Margin'])
Col_stats_temp = pd.DataFrame(columns=['Kicks for','Kicks against','Marks for','Marks against','Handballs for',
                                      'Handballs against','Disposals for','Disposals against','Hitouts for',
                                       'Hitouts against','Tackles for','Tackles against','Rebound 50s for',
                                      'Rebound 50s against','Inside 50s for','Inside 50s against','Clearances for',
                                      'Clearances against','Clangers for','Clangers against','Frees for','Frees against',
                                      'Contested possessions for','Contested possessions against',
                                       'Uncontested possessions for','Uncontested possessions against',
                                       'Contested marks for','Contested marks against','Marks inside 50 for',
                                      'Marks inside 50 against','1%s for','1%s against','Bounces for','Bounces against',
                                      'Goal assists for','Goal assists against','Margin'])
Ess_stats_temp = pd.DataFrame(columns=['Kicks for','Kicks against','Marks for','Marks against','Handballs for',
                                      'Handballs against','Disposals for','Disposals against','Hitouts for',
                                       'Hitouts against','Tackles for','Tackles against','Rebound 50s for',
                                      'Rebound 50s against','Inside 50s for','Inside 50s against','Clearances for',
                                      'Clearances against','Clangers for','Clangers against','Frees for','Frees against',
                                      'Contested possessions for','Contested possessions against',
                                       'Uncontested possessions for','Uncontested possessions against',
                                       'Contested marks for','Contested marks against','Marks inside 50 for',
                                      'Marks inside 50 against','1%s for','1%s against','Bounces for','Bounces against',
                                      'Goal assists for','Goal assists against','Margin'])
Fre_stats_temp = pd.DataFrame(columns=['Kicks for','Kicks against','Marks for','Marks against','Handballs for',
                                      'Handballs against','Disposals for','Disposals against','Hitouts for',
                                       'Hitouts against','Tackles for','Tackles against','Rebound 50s for',
                                      'Rebound 50s against','Inside 50s for','Inside 50s against','Clearances for',
                                      'Clearances against','Clangers for','Clangers against','Frees for','Frees against',
                                      'Contested possessions for','Contested possessions against',
                                       'Uncontested possessions for','Uncontested possessions against',
                                       'Contested marks for','Contested marks against','Marks inside 50 for',
                                      'Marks inside 50 against','1%s for','1%s against','Bounces for','Bounces against',
                                      'Goal assists for','Goal assists against','Margin'])
Gee_stats_temp = pd.DataFrame(columns=['Kicks for','Kicks against','Marks for','Marks against','Handballs for',
                                      'Handballs against','Disposals for','Disposals against','Hitouts for',
                                       'Hitouts against','Tackles for','Tackles against','Rebound 50s for',
                                      'Rebound 50s against','Inside 50s for','Inside 50s against','Clearances for',
                                      'Clearances against','Clangers for','Clangers against','Frees for','Frees against',
                                      'Contested possessions for','Contested possessions against',
                                       'Uncontested possessions for','Uncontested possessions against',
                                       'Contested marks for','Contested marks against','Marks inside 50 for',
                                      'Marks inside 50 against','1%s for','1%s against','Bounces for','Bounces against',
                                      'Goal assists for','Goal assists against','Margin'])
Gol_stats_temp = pd.DataFrame(columns=['Kicks for','Kicks against','Marks for','Marks against','Handballs for',
                                      'Handballs against','Disposals for','Disposals against','Hitouts for',
                                       'Hitouts against','Tackles for','Tackles against','Rebound 50s for',
                                      'Rebound 50s against','Inside 50s for','Inside 50s against','Clearances for',
                                      'Clearances against','Clangers for','Clangers against','Frees for','Frees against',
                                      'Contested possessions for','Contested possessions against',
                                       'Uncontested possessions for','Uncontested possessions against',
                                       'Contested marks for','Contested marks against','Marks inside 50 for',
                                      'Marks inside 50 against','1%s for','1%s against','Bounces for','Bounces against',
                                      'Goal assists for','Goal assists against','Margin'])
GWS_stats_temp = pd.DataFrame(columns=['Kicks for','Kicks against','Marks for','Marks against','Handballs for',
                                      'Handballs against','Disposals for','Disposals against','Hitouts for',
                                       'Hitouts against','Tackles for','Tackles against','Rebound 50s for',
                                      'Rebound 50s against','Inside 50s for','Inside 50s against','Clearances for',
                                      'Clearances against','Clangers for','Clangers against','Frees for','Frees against',
                                      'Contested possessions for','Contested possessions against',
                                       'Uncontested possessions for','Uncontested possessions against',
                                       'Contested marks for','Contested marks against','Marks inside 50 for',
                                      'Marks inside 50 against','1%s for','1%s against','Bounces for','Bounces against',
                                      'Goal assists for','Goal assists against','Margin'])
Haw_stats_temp = pd.DataFrame(columns=['Kicks for','Kicks against','Marks for','Marks against','Handballs for',
                                      'Handballs against','Disposals for','Disposals against','Hitouts for',
                                       'Hitouts against','Tackles for','Tackles against','Rebound 50s for',
                                      'Rebound 50s against','Inside 50s for','Inside 50s against','Clearances for',
                                      'Clearances against','Clangers for','Clangers against','Frees for','Frees against',
                                      'Contested possessions for','Contested possessions against',
                                       'Uncontested possessions for','Uncontested possessions against',
                                       'Contested marks for','Contested marks against','Marks inside 50 for',
                                      'Marks inside 50 against','1%s for','1%s against','Bounces for','Bounces against',
                                      'Goal assists for','Goal assists against','Margin'])
Mel_stats_temp = pd.DataFrame(columns=['Kicks for','Kicks against','Marks for','Marks against','Handballs for',
                                      'Handballs against','Disposals for','Disposals against','Hitouts for',
                                       'Hitouts against','Tackles for','Tackles against','Rebound 50s for',
                                      'Rebound 50s against','Inside 50s for','Inside 50s against','Clearances for',
                                      'Clearances against','Clangers for','Clangers against','Frees for','Frees against',
                                      'Contested possessions for','Contested possessions against',
                                       'Uncontested possessions for','Uncontested possessions against',
                                       'Contested marks for','Contested marks against','Marks inside 50 for',
                                      'Marks inside 50 against','1%s for','1%s against','Bounces for','Bounces against',
                                      'Goal assists for','Goal assists against','Margin'])
Nor_stats_temp = pd.DataFrame(columns=['Kicks for','Kicks against','Marks for','Marks against','Handballs for',
                                      'Handballs against','Disposals for','Disposals against','Hitouts for',
                                       'Hitouts against','Tackles for','Tackles against','Rebound 50s for',
                                      'Rebound 50s against','Inside 50s for','Inside 50s against','Clearances for',
                                      'Clearances against','Clangers for','Clangers against','Frees for','Frees against',
                                      'Contested possessions for','Contested possessions against',
                                       'Uncontested possessions for','Uncontested possessions against',
                                       'Contested marks for','Contested marks against','Marks inside 50 for',
                                      'Marks inside 50 against','1%s for','1%s against','Bounces for','Bounces against',
                                      'Goal assists for','Goal assists against','Margin'])
Por_stats_temp = pd.DataFrame(columns=['Kicks for','Kicks against','Marks for','Marks against','Handballs for',
                                      'Handballs against','Disposals for','Disposals against','Hitouts for',
                                       'Hitouts against','Tackles for','Tackles against','Rebound 50s for',
                                      'Rebound 50s against','Inside 50s for','Inside 50s against','Clearances for',
                                      'Clearances against','Clangers for','Clangers against','Frees for','Frees against',
                                      'Contested possessions for','Contested possessions against',
                                       'Uncontested possessions for','Uncontested possessions against',
                                       'Contested marks for','Contested marks against','Marks inside 50 for',
                                      'Marks inside 50 against','1%s for','1%s against','Bounces for','Bounces against',
                                      'Goal assists for','Goal assists against','Margin'])
Ric_stats_temp = pd.DataFrame(columns=['Kicks for','Kicks against','Marks for','Marks against','Handballs for',
                                      'Handballs against','Disposals for','Disposals against','Hitouts for',
                                       'Hitouts against','Tackles for','Tackles against','Rebound 50s for',
                                      'Rebound 50s against','Inside 50s for','Inside 50s against','Clearances for',
                                      'Clearances against','Clangers for','Clangers against','Frees for','Frees against',
                                      'Contested possessions for','Contested possessions against',
                                       'Uncontested possessions for','Uncontested possessions against',
                                       'Contested marks for','Contested marks against','Marks inside 50 for',
                                      'Marks inside 50 against','1%s for','1%s against','Bounces for','Bounces against',
                                      'Goal assists for','Goal assists against','Margin'])
StK_stats_temp = pd.DataFrame(columns=['Kicks for','Kicks against','Marks for','Marks against','Handballs for',
                                      'Handballs against','Disposals for','Disposals against','Hitouts for',
                                       'Hitouts against','Tackles for','Tackles against','Rebound 50s for',
                                      'Rebound 50s against','Inside 50s for','Inside 50s against','Clearances for',
                                      'Clearances against','Clangers for','Clangers against','Frees for','Frees against',
                                      'Contested possessions for','Contested possessions against',
                                       'Uncontested possessions for','Uncontested possessions against',
                                       'Contested marks for','Contested marks against','Marks inside 50 for',
                                      'Marks inside 50 against','1%s for','1%s against','Bounces for','Bounces against',
                                      'Goal assists for','Goal assists against','Margin'])
Syd_stats_temp = pd.DataFrame(columns=['Kicks for','Kicks against','Marks for','Marks against','Handballs for',
                                      'Handballs against','Disposals for','Disposals against','Hitouts for',
                                       'Hitouts against','Tackles for','Tackles against','Rebound 50s for',
                                      'Rebound 50s against','Inside 50s for','Inside 50s against','Clearances for',
                                      'Clearances against','Clangers for','Clangers against','Frees for','Frees against',
                                      'Contested possessions for','Contested possessions against',
                                       'Uncontested possessions for','Uncontested possessions against',
                                       'Contested marks for','Contested marks against','Marks inside 50 for',
                                      'Marks inside 50 against','1%s for','1%s against','Bounces for','Bounces against',
                                      'Goal assists for','Goal assists against','Margin'])
WCE_stats_temp = pd.DataFrame(columns=['Kicks for','Kicks against','Marks for','Marks against','Handballs for',
                                      'Handballs against','Disposals for','Disposals against','Hitouts for',
                                       'Hitouts against','Tackles for','Tackles against','Rebound 50s for',
                                      'Rebound 50s against','Inside 50s for','Inside 50s against','Clearances for',
                                      'Clearances against','Clangers for','Clangers against','Frees for','Frees against',
                                      'Contested possessions for','Contested possessions against',
                                       'Uncontested possessions for','Uncontested possessions against',
                                       'Contested marks for','Contested marks against','Marks inside 50 for',
                                      'Marks inside 50 against','1%s for','1%s against','Bounces for','Bounces against',
                                      'Goal assists for','Goal assists against','Margin'])
Wes_stats_temp = pd.DataFrame(columns=['Kicks for','Kicks against','Marks for','Marks against','Handballs for',
                                      'Handballs against','Disposals for','Disposals against','Hitouts for',
                                       'Hitouts against','Tackles for','Tackles against','Rebound 50s for',
                                      'Rebound 50s against','Inside 50s for','Inside 50s against','Clearances for',
                                      'Clearances against','Clangers for','Clangers against','Frees for','Frees against',
                                      'Contested possessions for','Contested possessions against',
                                       'Uncontested possessions for','Uncontested possessions against',
                                       'Contested marks for','Contested marks against','Marks inside 50 for',
                                      'Marks inside 50 against','1%s for','1%s against','Bounces for','Bounces against',
                                      'Goal assists for','Goal assists against','Margin'])

In [7]:
Ade_stats_temp[['Kicks for','Kicks against']] = Ade_stats['KI'].str.split('-',expand=True)
Ade_stats_temp[['Marks for','Marks against']] = Ade_stats['MK'].str.split('-',expand=True)
Ade_stats_temp[['Handballs for','Handballs against']] = Ade_stats['HB'].str.split('-',expand=True)
Ade_stats_temp[['Disposals for','Disposals against']] = Ade_stats['DI'].str.split('-',expand=True)
Ade_stats_temp[['Hitouts for','Hitouts against']] = Ade_stats['HO'].str.split('-',expand=True)
Ade_stats_temp[['Tackles for','Tackles against']] = Ade_stats['TK'].str.split('-',expand=True)
Ade_stats_temp[['Rebound 50s for','Rebound 50s against']] = Ade_stats['RB'].str.split('-',expand=True)
Ade_stats_temp[['Inside 50s for','Inside 50s against']] = Ade_stats['IF'].str.split('-',expand=True)
Ade_stats_temp[['Clearances for','Clearances against']] = Ade_stats['CL'].str.split('-',expand=True)
Ade_stats_temp[['Clangers for','Clangers against']] = Ade_stats['CG'].str.split('-',expand=True)
Ade_stats_temp[['Frees for','Frees against']] = Ade_stats['FF'].str.split('-',expand=True)
Ade_stats_temp[['Contested possessions for','Contested possessions against']] = Ade_stats['CP'].str.split('-',expand=True)
Ade_stats_temp[['Uncontested possessions for','Uncontested possessions against']] = Ade_stats['UP'].str.split('-',expand=True)
Ade_stats_temp[['Contested marks for','Contested marks against']] = Ade_stats['CM'].str.split('-',expand=True)
Ade_stats_temp[['Marks inside 50 for','Marks inside 50 against']] = Ade_stats['MI'].str.split('-',expand=True)
Ade_stats_temp[['1%s for','1%s against']] = Ade_stats['1%'].str.split('-',expand=True)
Ade_stats_temp[['Bounces for','Bounces against']] = Ade_stats['BO'].str.split('-',expand=True)
Ade_stats_temp[['Goal assists for','Goal assists against']] = Ade_stats['GA'].str.split('-',expand=True)
Ade_stats_temp['Margin'] = Ade_stats['Margin']

Ade_stats_temp['Kicks for'] = pd.to_numeric(Ade_stats_temp['Kicks for'])
Ade_stats_temp['Kicks against'] = pd.to_numeric(Ade_stats_temp['Kicks against'])
Ade_stats_temp['Marks for'] = pd.to_numeric(Ade_stats_temp['Marks for'])
Ade_stats_temp['Marks against'] = pd.to_numeric(Ade_stats_temp['Marks against'])
Ade_stats_temp['Handballs for'] = pd.to_numeric(Ade_stats_temp['Handballs for'])
Ade_stats_temp['Handballs against'] = pd.to_numeric(Ade_stats_temp['Handballs against'])
Ade_stats_temp['Disposals for'] = pd.to_numeric(Ade_stats_temp['Disposals for'])
Ade_stats_temp['Disposals against'] = pd.to_numeric(Ade_stats_temp['Disposals against'])
Ade_stats_temp['Hitouts for'] = pd.to_numeric(Ade_stats_temp['Hitouts for'])
Ade_stats_temp['Hitouts against'] = pd.to_numeric(Ade_stats_temp['Hitouts against'])
Ade_stats_temp['Tackles for'] = pd.to_numeric(Ade_stats_temp['Tackles for'])
Ade_stats_temp['Tackles against'] = pd.to_numeric(Ade_stats_temp['Tackles against'])
Ade_stats_temp['Rebound 50s for'] = pd.to_numeric(Ade_stats_temp['Rebound 50s for'])
Ade_stats_temp['Rebound 50s against'] = pd.to_numeric(Ade_stats_temp['Rebound 50s against'])
Ade_stats_temp['Inside 50s for'] = pd.to_numeric(Ade_stats_temp['Inside 50s for'])
Ade_stats_temp['Inside 50s against'] = pd.to_numeric(Ade_stats_temp['Inside 50s against'])
Ade_stats_temp['Clearances for'] = pd.to_numeric(Ade_stats_temp['Clearances for'])
Ade_stats_temp['Clearances against'] = pd.to_numeric(Ade_stats_temp['Clearances against'])
Ade_stats_temp['Clangers for'] = pd.to_numeric(Ade_stats_temp['Clangers for'])
Ade_stats_temp['Clangers against'] = pd.to_numeric(Ade_stats_temp['Clearances against'])
Ade_stats_temp['Frees for'] = pd.to_numeric(Ade_stats_temp['Frees for'])
Ade_stats_temp['Frees against'] = pd.to_numeric(Ade_stats_temp['Frees against'])
Ade_stats_temp['Contested possessions for'] = pd.to_numeric(Ade_stats_temp['Contested possessions for'])
Ade_stats_temp['Contested possessions against'] = pd.to_numeric(Ade_stats_temp['Contested possessions against'])
Ade_stats_temp['Uncontested possessions for'] = pd.to_numeric(Ade_stats_temp['Uncontested possessions for'])
Ade_stats_temp['Uncontested possessions against'] = pd.to_numeric(Ade_stats_temp['Uncontested possessions against'])
Ade_stats_temp['Contested marks for'] = pd.to_numeric(Ade_stats_temp['Contested marks for'])
Ade_stats_temp['Contested marks against'] = pd.to_numeric(Ade_stats_temp['Contested marks against'])
Ade_stats_temp['Marks inside 50 for'] = pd.to_numeric(Ade_stats_temp['Marks inside 50 for'])
Ade_stats_temp['Marks inside 50 against'] = pd.to_numeric(Ade_stats_temp['Marks inside 50 against'])
Ade_stats_temp['1%s for'] = pd.to_numeric(Ade_stats_temp['1%s for'])
Ade_stats_temp['1%s against'] = pd.to_numeric(Ade_stats_temp['1%s against'])
Ade_stats_temp['Bounces for'] = pd.to_numeric(Ade_stats_temp['Bounces for'])
Ade_stats_temp['Bounces against'] = pd.to_numeric(Ade_stats_temp['Bounces against'])
Ade_stats_temp['Goal assists for'] = pd.to_numeric(Ade_stats_temp['Goal assists for'])
Ade_stats_temp['Goal assists against'] = pd.to_numeric(Ade_stats_temp['Goal assists against'])

Ade = pd.concat([Ade_class, Ade_stats_temp], axis=1)
Ade.columns = ['Header', '#','Game ID','Home', 'Kicks for', 'Kicks against',
       'Marks for', 'Marks against', 'Handballs for', 'Handballs against',
       'Disposals for', 'Disposals against', 'Hitouts for', 'Hitouts against',
       'Tackles for', 'Tackles against', 'Rebound 50s for',
       'Rebound 50s against', 'Inside 50s for', 'Inside 50s against',
       'Clearances for', 'Clearances against', 'Clangers for',
       'Clangers against', 'Frees for', 'Frees against',
       'Contested possessions for', 'Contested possessions against',
       'Uncontested possessions for', 'Uncontested possessions against',
       'Contested marks for', 'Contested marks against', 'Marks inside 50 for',
       'Marks inside 50 against', '1%s for', '1%s against', 'Bounces for',
       'Bounces against', 'Goal assists for', 'Goal assists against',
       'Margin']

In [8]:
Bri_stats_temp[['Kicks for','Kicks against']] = Bri_stats['KI'].str.split('-',expand=True)
Bri_stats_temp[['Marks for','Marks against']] = Bri_stats['MK'].str.split('-',expand=True)
Bri_stats_temp[['Handballs for','Handballs against']] = Bri_stats['HB'].str.split('-',expand=True)
Bri_stats_temp[['Disposals for','Disposals against']] = Bri_stats['DI'].str.split('-',expand=True)
Bri_stats_temp[['Hitouts for','Hitouts against']] = Bri_stats['HO'].str.split('-',expand=True)
Bri_stats_temp[['Tackles for','Tackles against']] = Bri_stats['TK'].str.split('-',expand=True)
Bri_stats_temp[['Rebound 50s for','Rebound 50s against']] = Bri_stats['RB'].str.split('-',expand=True)
Bri_stats_temp[['Inside 50s for','Inside 50s against']] = Bri_stats['IF'].str.split('-',expand=True)
Bri_stats_temp[['Clearances for','Clearances against']] = Bri_stats['CL'].str.split('-',expand=True)
Bri_stats_temp[['Clangers for','Clangers against']] = Bri_stats['CG'].str.split('-',expand=True)
Bri_stats_temp[['Frees for','Frees against']] = Bri_stats['FF'].str.split('-',expand=True)
Bri_stats_temp[['Contested possessions for','Contested possessions against']] = Bri_stats['CP'].str.split('-',expand=True)
Bri_stats_temp[['Uncontested possessions for','Uncontested possessions against']] = Bri_stats['UP'].str.split('-',expand=True)
Bri_stats_temp[['Contested marks for','Contested marks against']] = Bri_stats['CM'].str.split('-',expand=True)
Bri_stats_temp[['Marks inside 50 for','Marks inside 50 against']] = Bri_stats['MI'].str.split('-',expand=True)
Bri_stats_temp[['1%s for','1%s against']] = Bri_stats['1%'].str.split('-',expand=True)
Bri_stats_temp[['Bounces for','Bounces against']] = Bri_stats['BO'].str.split('-',expand=True)
Bri_stats_temp[['Goal assists for','Goal assists against']] = Bri_stats['GA'].str.split('-',expand=True)
Bri_stats_temp['Margin'] = Bri_stats['Margin']

Bri_stats_temp['Kicks for'] = pd.to_numeric(Bri_stats_temp['Kicks for'])
Bri_stats_temp['Kicks against'] = pd.to_numeric(Bri_stats_temp['Kicks against'])
Bri_stats_temp['Marks for'] = pd.to_numeric(Bri_stats_temp['Marks for'])
Bri_stats_temp['Marks against'] = pd.to_numeric(Bri_stats_temp['Marks against'])
Bri_stats_temp['Handballs for'] = pd.to_numeric(Bri_stats_temp['Handballs for'])
Bri_stats_temp['Handballs against'] = pd.to_numeric(Bri_stats_temp['Handballs against'])
Bri_stats_temp['Disposals for'] = pd.to_numeric(Bri_stats_temp['Disposals for'])
Bri_stats_temp['Disposals against'] = pd.to_numeric(Bri_stats_temp['Disposals against'])
Bri_stats_temp['Hitouts for'] = pd.to_numeric(Bri_stats_temp['Hitouts for'])
Bri_stats_temp['Hitouts against'] = pd.to_numeric(Bri_stats_temp['Hitouts against'])
Bri_stats_temp['Tackles for'] = pd.to_numeric(Bri_stats_temp['Tackles for'])
Bri_stats_temp['Tackles against'] = pd.to_numeric(Bri_stats_temp['Tackles against'])
Bri_stats_temp['Rebound 50s for'] = pd.to_numeric(Bri_stats_temp['Rebound 50s for'])
Bri_stats_temp['Rebound 50s against'] = pd.to_numeric(Bri_stats_temp['Rebound 50s against'])
Bri_stats_temp['Inside 50s for'] = pd.to_numeric(Bri_stats_temp['Inside 50s for'])
Bri_stats_temp['Inside 50s against'] = pd.to_numeric(Bri_stats_temp['Inside 50s against'])
Bri_stats_temp['Clearances for'] = pd.to_numeric(Bri_stats_temp['Clearances for'])
Bri_stats_temp['Clearances against'] = pd.to_numeric(Bri_stats_temp['Clearances against'])
Bri_stats_temp['Clangers for'] = pd.to_numeric(Bri_stats_temp['Clangers for'])
Bri_stats_temp['Clangers against'] = pd.to_numeric(Bri_stats_temp['Clearances against'])
Bri_stats_temp['Frees for'] = pd.to_numeric(Bri_stats_temp['Frees for'])
Bri_stats_temp['Frees against'] = pd.to_numeric(Bri_stats_temp['Frees against'])
Bri_stats_temp['Contested possessions for'] = pd.to_numeric(Bri_stats_temp['Contested possessions for'])
Bri_stats_temp['Contested possessions against'] = pd.to_numeric(Bri_stats_temp['Contested possessions against'])
Bri_stats_temp['Uncontested possessions for'] = pd.to_numeric(Bri_stats_temp['Uncontested possessions for'])
Bri_stats_temp['Uncontested possessions against'] = pd.to_numeric(Bri_stats_temp['Uncontested possessions against'])
Bri_stats_temp['Contested marks for'] = pd.to_numeric(Bri_stats_temp['Contested marks for'])
Bri_stats_temp['Contested marks against'] = pd.to_numeric(Bri_stats_temp['Contested marks against'])
Bri_stats_temp['Marks inside 50 for'] = pd.to_numeric(Bri_stats_temp['Marks inside 50 for'])
Bri_stats_temp['Marks inside 50 against'] = pd.to_numeric(Bri_stats_temp['Marks inside 50 against'])
Bri_stats_temp['1%s for'] = pd.to_numeric(Bri_stats_temp['1%s for'])
Bri_stats_temp['1%s against'] = pd.to_numeric(Bri_stats_temp['1%s against'])
Bri_stats_temp['Bounces for'] = pd.to_numeric(Bri_stats_temp['Bounces for'])
Bri_stats_temp['Bounces against'] = pd.to_numeric(Bri_stats_temp['Bounces against'])
Bri_stats_temp['Goal assists for'] = pd.to_numeric(Bri_stats_temp['Goal assists for'])
Bri_stats_temp['Goal assists against'] = pd.to_numeric(Bri_stats_temp['Goal assists against'])

Bri = pd.concat([Bri_class, Bri_stats_temp], axis=1)
Bri.columns = ['Header', '#','Game ID','Home', 'Kicks for', 'Kicks against',
       'Marks for', 'Marks against', 'Handballs for', 'Handballs against',
       'Disposals for', 'Disposals against', 'Hitouts for', 'Hitouts against',
       'Tackles for', 'Tackles against', 'Rebound 50s for',
       'Rebound 50s against', 'Inside 50s for', 'Inside 50s against',
       'Clearances for', 'Clearances against', 'Clangers for',
       'Clangers against', 'Frees for', 'Frees against',
       'Contested possessions for', 'Contested possessions against',
       'Uncontested possessions for', 'Uncontested possessions against',
       'Contested marks for', 'Contested marks against', 'Marks inside 50 for',
       'Marks inside 50 against', '1%s for', '1%s against', 'Bounces for',
       'Bounces against', 'Goal assists for', 'Goal assists against',
       'Margin']

In [9]:
Car_stats_temp[['Kicks for','Kicks against']] = Car_stats['KI'].str.split('-',expand=True)
Car_stats_temp[['Marks for','Marks against']] = Car_stats['MK'].str.split('-',expand=True)
Car_stats_temp[['Handballs for','Handballs against']] = Car_stats['HB'].str.split('-',expand=True)
Car_stats_temp[['Disposals for','Disposals against']] = Car_stats['DI'].str.split('-',expand=True)
Car_stats_temp[['Hitouts for','Hitouts against']] = Car_stats['HO'].str.split('-',expand=True)
Car_stats_temp[['Tackles for','Tackles against']] = Car_stats['TK'].str.split('-',expand=True)
Car_stats_temp[['Rebound 50s for','Rebound 50s against']] = Car_stats['RB'].str.split('-',expand=True)
Car_stats_temp[['Inside 50s for','Inside 50s against']] = Car_stats['IF'].str.split('-',expand=True)
Car_stats_temp[['Clearances for','Clearances against']] = Car_stats['CL'].str.split('-',expand=True)
Car_stats_temp[['Clangers for','Clangers against']] = Car_stats['CG'].str.split('-',expand=True)
Car_stats_temp[['Frees for','Frees against']] = Car_stats['FF'].str.split('-',expand=True)
Car_stats_temp[['Contested possessions for','Contested possessions against']] = Car_stats['CP'].str.split('-',expand=True)
Car_stats_temp[['Uncontested possessions for','Uncontested possessions against']] = Car_stats['UP'].str.split('-',expand=True)
Car_stats_temp[['Contested marks for','Contested marks against']] = Car_stats['CM'].str.split('-',expand=True)
Car_stats_temp[['Marks inside 50 for','Marks inside 50 against']] = Car_stats['MI'].str.split('-',expand=True)
Car_stats_temp[['1%s for','1%s against']] = Car_stats['1%'].str.split('-',expand=True)
Car_stats_temp[['Bounces for','Bounces against']] = Car_stats['BO'].str.split('-',expand=True)
Car_stats_temp[['Goal assists for','Goal assists against']] = Car_stats['GA'].str.split('-',expand=True)
Car_stats_temp['Margin'] = Car_stats['Margin']

Car_stats_temp['Kicks for'] = pd.to_numeric(Car_stats_temp['Kicks for'])
Car_stats_temp['Kicks against'] = pd.to_numeric(Car_stats_temp['Kicks against'])
Car_stats_temp['Marks for'] = pd.to_numeric(Car_stats_temp['Marks for'])
Car_stats_temp['Marks against'] = pd.to_numeric(Car_stats_temp['Marks against'])
Car_stats_temp['Handballs for'] = pd.to_numeric(Car_stats_temp['Handballs for'])
Car_stats_temp['Handballs against'] = pd.to_numeric(Car_stats_temp['Handballs against'])
Car_stats_temp['Disposals for'] = pd.to_numeric(Car_stats_temp['Disposals for'])
Car_stats_temp['Disposals against'] = pd.to_numeric(Car_stats_temp['Disposals against'])
Car_stats_temp['Hitouts for'] = pd.to_numeric(Car_stats_temp['Hitouts for'])
Car_stats_temp['Hitouts against'] = pd.to_numeric(Car_stats_temp['Hitouts against'])
Car_stats_temp['Tackles for'] = pd.to_numeric(Car_stats_temp['Tackles for'])
Car_stats_temp['Tackles against'] = pd.to_numeric(Car_stats_temp['Tackles against'])
Car_stats_temp['Rebound 50s for'] = pd.to_numeric(Car_stats_temp['Rebound 50s for'])
Car_stats_temp['Rebound 50s against'] = pd.to_numeric(Car_stats_temp['Rebound 50s against'])
Car_stats_temp['Inside 50s for'] = pd.to_numeric(Car_stats_temp['Inside 50s for'])
Car_stats_temp['Inside 50s against'] = pd.to_numeric(Car_stats_temp['Inside 50s against'])
Car_stats_temp['Clearances for'] = pd.to_numeric(Car_stats_temp['Clearances for'])
Car_stats_temp['Clearances against'] = pd.to_numeric(Car_stats_temp['Clearances against'])
Car_stats_temp['Clangers for'] = pd.to_numeric(Car_stats_temp['Clangers for'])
Car_stats_temp['Clangers against'] = pd.to_numeric(Car_stats_temp['Clearances against'])
Car_stats_temp['Frees for'] = pd.to_numeric(Car_stats_temp['Frees for'])
Car_stats_temp['Frees against'] = pd.to_numeric(Car_stats_temp['Frees against'])
Car_stats_temp['Contested possessions for'] = pd.to_numeric(Car_stats_temp['Contested possessions for'])
Car_stats_temp['Contested possessions against'] = pd.to_numeric(Car_stats_temp['Contested possessions against'])
Car_stats_temp['Uncontested possessions for'] = pd.to_numeric(Car_stats_temp['Uncontested possessions for'])
Car_stats_temp['Uncontested possessions against'] = pd.to_numeric(Car_stats_temp['Uncontested possessions against'])
Car_stats_temp['Contested marks for'] = pd.to_numeric(Car_stats_temp['Contested marks for'])
Car_stats_temp['Contested marks against'] = pd.to_numeric(Car_stats_temp['Contested marks against'])
Car_stats_temp['Marks inside 50 for'] = pd.to_numeric(Car_stats_temp['Marks inside 50 for'])
Car_stats_temp['Marks inside 50 against'] = pd.to_numeric(Car_stats_temp['Marks inside 50 against'])
Car_stats_temp['1%s for'] = pd.to_numeric(Car_stats_temp['1%s for'])
Car_stats_temp['1%s against'] = pd.to_numeric(Car_stats_temp['1%s against'])
Car_stats_temp['Bounces for'] = pd.to_numeric(Car_stats_temp['Bounces for'])
Car_stats_temp['Bounces against'] = pd.to_numeric(Car_stats_temp['Bounces against'])
Car_stats_temp['Goal assists for'] = pd.to_numeric(Car_stats_temp['Goal assists for'])
Car_stats_temp['Goal assists against'] = pd.to_numeric(Car_stats_temp['Goal assists against'])

Car = pd.concat([Car_class, Car_stats_temp], axis=1)
Car.columns = ['Header', '#','Game ID','Home', 'Kicks for', 'Kicks against',
       'Marks for', 'Marks against', 'Handballs for', 'Handballs against',
       'Disposals for', 'Disposals against', 'Hitouts for', 'Hitouts against',
       'Tackles for', 'Tackles against', 'Rebound 50s for',
       'Rebound 50s against', 'Inside 50s for', 'Inside 50s against',
       'Clearances for', 'Clearances against', 'Clangers for',
       'Clangers against', 'Frees for', 'Frees against',
       'Contested possessions for', 'Contested possessions against',
       'Uncontested possessions for', 'Uncontested possessions against',
       'Contested marks for', 'Contested marks against', 'Marks inside 50 for',
       'Marks inside 50 against', '1%s for', '1%s against', 'Bounces for',
       'Bounces against', 'Goal assists for', 'Goal assists against',
       'Margin']

In [10]:
Col_stats_temp[['Kicks for','Kicks against']] = Col_stats['KI'].str.split('-',expand=True)
Col_stats_temp[['Marks for','Marks against']] = Col_stats['MK'].str.split('-',expand=True)
Col_stats_temp[['Handballs for','Handballs against']] = Col_stats['HB'].str.split('-',expand=True)
Col_stats_temp[['Disposals for','Disposals against']] = Col_stats['DI'].str.split('-',expand=True)
Col_stats_temp[['Hitouts for','Hitouts against']] = Col_stats['HO'].str.split('-',expand=True)
Col_stats_temp[['Tackles for','Tackles against']] = Col_stats['TK'].str.split('-',expand=True)
Col_stats_temp[['Rebound 50s for','Rebound 50s against']] = Col_stats['RB'].str.split('-',expand=True)
Col_stats_temp[['Inside 50s for','Inside 50s against']] = Col_stats['IF'].str.split('-',expand=True)
Col_stats_temp[['Clearances for','Clearances against']] = Col_stats['CL'].str.split('-',expand=True)
Col_stats_temp[['Clangers for','Clangers against']] = Col_stats['CG'].str.split('-',expand=True)
Col_stats_temp[['Frees for','Frees against']] = Col_stats['FF'].str.split('-',expand=True)
Col_stats_temp[['Contested possessions for','Contested possessions against']] = Col_stats['CP'].str.split('-',expand=True)
Col_stats_temp[['Uncontested possessions for','Uncontested possessions against']] = Col_stats['UP'].str.split('-',expand=True)
Col_stats_temp[['Contested marks for','Contested marks against']] = Col_stats['CM'].str.split('-',expand=True)
Col_stats_temp[['Marks inside 50 for','Marks inside 50 against']] = Col_stats['MI'].str.split('-',expand=True)
Col_stats_temp[['1%s for','1%s against']] = Col_stats['1%'].str.split('-',expand=True)
Col_stats_temp[['Bounces for','Bounces against']] = Col_stats['BO'].str.split('-',expand=True)
Col_stats_temp[['Goal assists for','Goal assists against']] = Col_stats['GA'].str.split('-',expand=True)
Col_stats_temp['Margin'] = Col_stats['Margin']

Col_stats_temp['Kicks for'] = pd.to_numeric(Col_stats_temp['Kicks for'])
Col_stats_temp['Kicks against'] = pd.to_numeric(Col_stats_temp['Kicks against'])
Col_stats_temp['Marks for'] = pd.to_numeric(Col_stats_temp['Marks for'])
Col_stats_temp['Marks against'] = pd.to_numeric(Col_stats_temp['Marks against'])
Col_stats_temp['Handballs for'] = pd.to_numeric(Col_stats_temp['Handballs for'])
Col_stats_temp['Handballs against'] = pd.to_numeric(Col_stats_temp['Handballs against'])
Col_stats_temp['Disposals for'] = pd.to_numeric(Col_stats_temp['Disposals for'])
Col_stats_temp['Disposals against'] = pd.to_numeric(Col_stats_temp['Disposals against'])
Col_stats_temp['Hitouts for'] = pd.to_numeric(Col_stats_temp['Hitouts for'])
Col_stats_temp['Hitouts against'] = pd.to_numeric(Col_stats_temp['Hitouts against'])
Col_stats_temp['Tackles for'] = pd.to_numeric(Col_stats_temp['Tackles for'])
Col_stats_temp['Tackles against'] = pd.to_numeric(Col_stats_temp['Tackles against'])
Col_stats_temp['Rebound 50s for'] = pd.to_numeric(Col_stats_temp['Rebound 50s for'])
Col_stats_temp['Rebound 50s against'] = pd.to_numeric(Col_stats_temp['Rebound 50s against'])
Col_stats_temp['Inside 50s for'] = pd.to_numeric(Col_stats_temp['Inside 50s for'])
Col_stats_temp['Inside 50s against'] = pd.to_numeric(Col_stats_temp['Inside 50s against'])
Col_stats_temp['Clearances for'] = pd.to_numeric(Col_stats_temp['Clearances for'])
Col_stats_temp['Clearances against'] = pd.to_numeric(Col_stats_temp['Clearances against'])
Col_stats_temp['Clangers for'] = pd.to_numeric(Col_stats_temp['Clangers for'])
Col_stats_temp['Clangers against'] = pd.to_numeric(Col_stats_temp['Clearances against'])
Col_stats_temp['Frees for'] = pd.to_numeric(Col_stats_temp['Frees for'])
Col_stats_temp['Frees against'] = pd.to_numeric(Col_stats_temp['Frees against'])
Col_stats_temp['Contested possessions for'] = pd.to_numeric(Col_stats_temp['Contested possessions for'])
Col_stats_temp['Contested possessions against'] = pd.to_numeric(Col_stats_temp['Contested possessions against'])
Col_stats_temp['Uncontested possessions for'] = pd.to_numeric(Col_stats_temp['Uncontested possessions for'])
Col_stats_temp['Uncontested possessions against'] = pd.to_numeric(Col_stats_temp['Uncontested possessions against'])
Col_stats_temp['Contested marks for'] = pd.to_numeric(Col_stats_temp['Contested marks for'])
Col_stats_temp['Contested marks against'] = pd.to_numeric(Col_stats_temp['Contested marks against'])
Col_stats_temp['Marks inside 50 for'] = pd.to_numeric(Col_stats_temp['Marks inside 50 for'])
Col_stats_temp['Marks inside 50 against'] = pd.to_numeric(Col_stats_temp['Marks inside 50 against'])
Col_stats_temp['1%s for'] = pd.to_numeric(Col_stats_temp['1%s for'])
Col_stats_temp['1%s against'] = pd.to_numeric(Col_stats_temp['1%s against'])
Col_stats_temp['Bounces for'] = pd.to_numeric(Col_stats_temp['Bounces for'])
Col_stats_temp['Bounces against'] = pd.to_numeric(Col_stats_temp['Bounces against'])
Col_stats_temp['Goal assists for'] = pd.to_numeric(Col_stats_temp['Goal assists for'])
Col_stats_temp['Goal assists against'] = pd.to_numeric(Col_stats_temp['Goal assists against'])

Col = pd.concat([Col_class, Col_stats_temp], axis=1)
Col.columns = ['Header', '#','Game ID','Home', 'Kicks for', 'Kicks against',
       'Marks for', 'Marks against', 'Handballs for', 'Handballs against',
       'Disposals for', 'Disposals against', 'Hitouts for', 'Hitouts against',
       'Tackles for', 'Tackles against', 'Rebound 50s for',
       'Rebound 50s against', 'Inside 50s for', 'Inside 50s against',
       'Clearances for', 'Clearances against', 'Clangers for',
       'Clangers against', 'Frees for', 'Frees against',
       'Contested possessions for', 'Contested possessions against',
       'Uncontested possessions for', 'Uncontested possessions against',
       'Contested marks for', 'Contested marks against', 'Marks inside 50 for',
       'Marks inside 50 against', '1%s for', '1%s against', 'Bounces for',
       'Bounces against', 'Goal assists for', 'Goal assists against',
       'Margin']

In [11]:
Ess_stats_temp[['Kicks for','Kicks against']] = Ess_stats['KI'].str.split('-',expand=True)
Ess_stats_temp[['Marks for','Marks against']] = Ess_stats['MK'].str.split('-',expand=True)
Ess_stats_temp[['Handballs for','Handballs against']] = Ess_stats['HB'].str.split('-',expand=True)
Ess_stats_temp[['Disposals for','Disposals against']] = Ess_stats['DI'].str.split('-',expand=True)
Ess_stats_temp[['Hitouts for','Hitouts against']] = Ess_stats['HO'].str.split('-',expand=True)
Ess_stats_temp[['Tackles for','Tackles against']] = Ess_stats['TK'].str.split('-',expand=True)
Ess_stats_temp[['Rebound 50s for','Rebound 50s against']] = Ess_stats['RB'].str.split('-',expand=True)
Ess_stats_temp[['Inside 50s for','Inside 50s against']] = Ess_stats['IF'].str.split('-',expand=True)
Ess_stats_temp[['Clearances for','Clearances against']] = Ess_stats['CL'].str.split('-',expand=True)
Ess_stats_temp[['Clangers for','Clangers against']] = Ess_stats['CG'].str.split('-',expand=True)
Ess_stats_temp[['Frees for','Frees against']] = Ess_stats['FF'].str.split('-',expand=True)
Ess_stats_temp[['Contested possessions for','Contested possessions against']] = Ess_stats['CP'].str.split('-',expand=True)
Ess_stats_temp[['Uncontested possessions for','Uncontested possessions against']] = Ess_stats['UP'].str.split('-',expand=True)
Ess_stats_temp[['Contested marks for','Contested marks against']] = Ess_stats['CM'].str.split('-',expand=True)
Ess_stats_temp[['Marks inside 50 for','Marks inside 50 against']] = Ess_stats['MI'].str.split('-',expand=True)
Ess_stats_temp[['1%s for','1%s against']] = Ess_stats['1%'].str.split('-',expand=True)
Ess_stats_temp[['Bounces for','Bounces against']] = Ess_stats['BO'].str.split('-',expand=True)
Ess_stats_temp[['Goal assists for','Goal assists against']] = Ess_stats['GA'].str.split('-',expand=True)
Ess_stats_temp['Margin'] = Ess_stats['Margin']

Ess_stats_temp['Kicks for'] = pd.to_numeric(Ess_stats_temp['Kicks for'])
Ess_stats_temp['Kicks against'] = pd.to_numeric(Ess_stats_temp['Kicks against'])
Ess_stats_temp['Marks for'] = pd.to_numeric(Ess_stats_temp['Marks for'])
Ess_stats_temp['Marks against'] = pd.to_numeric(Ess_stats_temp['Marks against'])
Ess_stats_temp['Handballs for'] = pd.to_numeric(Ess_stats_temp['Handballs for'])
Ess_stats_temp['Handballs against'] = pd.to_numeric(Ess_stats_temp['Handballs against'])
Ess_stats_temp['Disposals for'] = pd.to_numeric(Ess_stats_temp['Disposals for'])
Ess_stats_temp['Disposals against'] = pd.to_numeric(Ess_stats_temp['Disposals against'])
Ess_stats_temp['Hitouts for'] = pd.to_numeric(Ess_stats_temp['Hitouts for'])
Ess_stats_temp['Hitouts against'] = pd.to_numeric(Ess_stats_temp['Hitouts against'])
Ess_stats_temp['Tackles for'] = pd.to_numeric(Ess_stats_temp['Tackles for'])
Ess_stats_temp['Tackles against'] = pd.to_numeric(Ess_stats_temp['Tackles against'])
Ess_stats_temp['Rebound 50s for'] = pd.to_numeric(Ess_stats_temp['Rebound 50s for'])
Ess_stats_temp['Rebound 50s against'] = pd.to_numeric(Ess_stats_temp['Rebound 50s against'])
Ess_stats_temp['Inside 50s for'] = pd.to_numeric(Ess_stats_temp['Inside 50s for'])
Ess_stats_temp['Inside 50s against'] = pd.to_numeric(Ess_stats_temp['Inside 50s against'])
Ess_stats_temp['Clearances for'] = pd.to_numeric(Ess_stats_temp['Clearances for'])
Ess_stats_temp['Clearances against'] = pd.to_numeric(Ess_stats_temp['Clearances against'])
Ess_stats_temp['Clangers for'] = pd.to_numeric(Ess_stats_temp['Clangers for'])
Ess_stats_temp['Clangers against'] = pd.to_numeric(Ess_stats_temp['Clearances against'])
Ess_stats_temp['Frees for'] = pd.to_numeric(Ess_stats_temp['Frees for'])
Ess_stats_temp['Frees against'] = pd.to_numeric(Ess_stats_temp['Frees against'])
Ess_stats_temp['Contested possessions for'] = pd.to_numeric(Ess_stats_temp['Contested possessions for'])
Ess_stats_temp['Contested possessions against'] = pd.to_numeric(Ess_stats_temp['Contested possessions against'])
Ess_stats_temp['Uncontested possessions for'] = pd.to_numeric(Ess_stats_temp['Uncontested possessions for'])
Ess_stats_temp['Uncontested possessions against'] = pd.to_numeric(Ess_stats_temp['Uncontested possessions against'])
Ess_stats_temp['Contested marks for'] = pd.to_numeric(Ess_stats_temp['Contested marks for'])
Ess_stats_temp['Contested marks against'] = pd.to_numeric(Ess_stats_temp['Contested marks against'])
Ess_stats_temp['Marks inside 50 for'] = pd.to_numeric(Ess_stats_temp['Marks inside 50 for'])
Ess_stats_temp['Marks inside 50 against'] = pd.to_numeric(Ess_stats_temp['Marks inside 50 against'])
Ess_stats_temp['1%s for'] = pd.to_numeric(Ess_stats_temp['1%s for'])
Ess_stats_temp['1%s against'] = pd.to_numeric(Ess_stats_temp['1%s against'])
Ess_stats_temp['Bounces for'] = pd.to_numeric(Ess_stats_temp['Bounces for'])
Ess_stats_temp['Bounces against'] = pd.to_numeric(Ess_stats_temp['Bounces against'])
Ess_stats_temp['Goal assists for'] = pd.to_numeric(Ess_stats_temp['Goal assists for'])
Ess_stats_temp['Goal assists against'] = pd.to_numeric(Ess_stats_temp['Goal assists against'])

Ess = pd.concat([Ess_class, Ess_stats_temp], axis=1)
Ess.columns = ['Header', '#','Game ID','Home', 'Kicks for', 'Kicks against',
       'Marks for', 'Marks against', 'Handballs for', 'Handballs against',
       'Disposals for', 'Disposals against', 'Hitouts for', 'Hitouts against',
       'Tackles for', 'Tackles against', 'Rebound 50s for',
       'Rebound 50s against', 'Inside 50s for', 'Inside 50s against',
       'Clearances for', 'Clearances against', 'Clangers for',
       'Clangers against', 'Frees for', 'Frees against',
       'Contested possessions for', 'Contested possessions against',
       'Uncontested possessions for', 'Uncontested possessions against',
       'Contested marks for', 'Contested marks against', 'Marks inside 50 for',
       'Marks inside 50 against', '1%s for', '1%s against', 'Bounces for',
       'Bounces against', 'Goal assists for', 'Goal assists against',
       'Margin']

In [12]:
Fre_stats_temp[['Kicks for','Kicks against']] = Fre_stats['KI'].str.split('-',expand=True)
Fre_stats_temp[['Marks for','Marks against']] = Fre_stats['MK'].str.split('-',expand=True)
Fre_stats_temp[['Handballs for','Handballs against']] = Fre_stats['HB'].str.split('-',expand=True)
Fre_stats_temp[['Disposals for','Disposals against']] = Fre_stats['DI'].str.split('-',expand=True)
Fre_stats_temp[['Hitouts for','Hitouts against']] = Fre_stats['HO'].str.split('-',expand=True)
Fre_stats_temp[['Tackles for','Tackles against']] = Fre_stats['TK'].str.split('-',expand=True)
Fre_stats_temp[['Rebound 50s for','Rebound 50s against']] = Fre_stats['RB'].str.split('-',expand=True)
Fre_stats_temp[['Inside 50s for','Inside 50s against']] = Fre_stats['IF'].str.split('-',expand=True)
Fre_stats_temp[['Clearances for','Clearances against']] = Fre_stats['CL'].str.split('-',expand=True)
Fre_stats_temp[['Clangers for','Clangers against']] = Fre_stats['CG'].str.split('-',expand=True)
Fre_stats_temp[['Frees for','Frees against']] = Fre_stats['FF'].str.split('-',expand=True)
Fre_stats_temp[['Contested possessions for','Contested possessions against']] = Fre_stats['CP'].str.split('-',expand=True)
Fre_stats_temp[['Uncontested possessions for','Uncontested possessions against']] = Fre_stats['UP'].str.split('-',expand=True)
Fre_stats_temp[['Contested marks for','Contested marks against']] = Fre_stats['CM'].str.split('-',expand=True)
Fre_stats_temp[['Marks inside 50 for','Marks inside 50 against']] = Fre_stats['MI'].str.split('-',expand=True)
Fre_stats_temp[['1%s for','1%s against']] = Fre_stats['1%'].str.split('-',expand=True)
Fre_stats_temp[['Bounces for','Bounces against']] = Fre_stats['BO'].str.split('-',expand=True)
Fre_stats_temp[['Goal assists for','Goal assists against']] = Fre_stats['GA'].str.split('-',expand=True)
Fre_stats_temp['Margin'] = Fre_stats['Margin']

Fre_stats_temp['Kicks for'] = pd.to_numeric(Fre_stats_temp['Kicks for'])
Fre_stats_temp['Kicks against'] = pd.to_numeric(Fre_stats_temp['Kicks against'])
Fre_stats_temp['Marks for'] = pd.to_numeric(Fre_stats_temp['Marks for'])
Fre_stats_temp['Marks against'] = pd.to_numeric(Fre_stats_temp['Marks against'])
Fre_stats_temp['Handballs for'] = pd.to_numeric(Fre_stats_temp['Handballs for'])
Fre_stats_temp['Handballs against'] = pd.to_numeric(Fre_stats_temp['Handballs against'])
Fre_stats_temp['Disposals for'] = pd.to_numeric(Fre_stats_temp['Disposals for'])
Fre_stats_temp['Disposals against'] = pd.to_numeric(Fre_stats_temp['Disposals against'])
Fre_stats_temp['Hitouts for'] = pd.to_numeric(Fre_stats_temp['Hitouts for'])
Fre_stats_temp['Hitouts against'] = pd.to_numeric(Fre_stats_temp['Hitouts against'])
Fre_stats_temp['Tackles for'] = pd.to_numeric(Fre_stats_temp['Tackles for'])
Fre_stats_temp['Tackles against'] = pd.to_numeric(Fre_stats_temp['Tackles against'])
Fre_stats_temp['Rebound 50s for'] = pd.to_numeric(Fre_stats_temp['Rebound 50s for'])
Fre_stats_temp['Rebound 50s against'] = pd.to_numeric(Fre_stats_temp['Rebound 50s against'])
Fre_stats_temp['Inside 50s for'] = pd.to_numeric(Fre_stats_temp['Inside 50s for'])
Fre_stats_temp['Inside 50s against'] = pd.to_numeric(Fre_stats_temp['Inside 50s against'])
Fre_stats_temp['Clearances for'] = pd.to_numeric(Fre_stats_temp['Clearances for'])
Fre_stats_temp['Clearances against'] = pd.to_numeric(Fre_stats_temp['Clearances against'])
Fre_stats_temp['Clangers for'] = pd.to_numeric(Fre_stats_temp['Clangers for'])
Fre_stats_temp['Clangers against'] = pd.to_numeric(Fre_stats_temp['Clearances against'])
Fre_stats_temp['Frees for'] = pd.to_numeric(Fre_stats_temp['Frees for'])
Fre_stats_temp['Frees against'] = pd.to_numeric(Fre_stats_temp['Frees against'])
Fre_stats_temp['Contested possessions for'] = pd.to_numeric(Fre_stats_temp['Contested possessions for'])
Fre_stats_temp['Contested possessions against'] = pd.to_numeric(Fre_stats_temp['Contested possessions against'])
Fre_stats_temp['Uncontested possessions for'] = pd.to_numeric(Fre_stats_temp['Uncontested possessions for'])
Fre_stats_temp['Uncontested possessions against'] = pd.to_numeric(Fre_stats_temp['Uncontested possessions against'])
Fre_stats_temp['Contested marks for'] = pd.to_numeric(Fre_stats_temp['Contested marks for'])
Fre_stats_temp['Contested marks against'] = pd.to_numeric(Fre_stats_temp['Contested marks against'])
Fre_stats_temp['Marks inside 50 for'] = pd.to_numeric(Fre_stats_temp['Marks inside 50 for'])
Fre_stats_temp['Marks inside 50 against'] = pd.to_numeric(Fre_stats_temp['Marks inside 50 against'])
Fre_stats_temp['1%s for'] = pd.to_numeric(Fre_stats_temp['1%s for'])
Fre_stats_temp['1%s against'] = pd.to_numeric(Fre_stats_temp['1%s against'])
Fre_stats_temp['Bounces for'] = pd.to_numeric(Fre_stats_temp['Bounces for'])
Fre_stats_temp['Bounces against'] = pd.to_numeric(Fre_stats_temp['Bounces against'])
Fre_stats_temp['Goal assists for'] = pd.to_numeric(Fre_stats_temp['Goal assists for'])
Fre_stats_temp['Goal assists against'] = pd.to_numeric(Fre_stats_temp['Goal assists against'])

Fre = pd.concat([Fre_class, Fre_stats_temp], axis=1)
Fre.columns = ['Header', '#','Game ID','Home', 'Kicks for', 'Kicks against',
       'Marks for', 'Marks against', 'Handballs for', 'Handballs against',
       'Disposals for', 'Disposals against', 'Hitouts for', 'Hitouts against',
       'Tackles for', 'Tackles against', 'Rebound 50s for',
       'Rebound 50s against', 'Inside 50s for', 'Inside 50s against',
       'Clearances for', 'Clearances against', 'Clangers for',
       'Clangers against', 'Frees for', 'Frees against',
       'Contested possessions for', 'Contested possessions against',
       'Uncontested possessions for', 'Uncontested possessions against',
       'Contested marks for', 'Contested marks against', 'Marks inside 50 for',
       'Marks inside 50 against', '1%s for', '1%s against', 'Bounces for',
       'Bounces against', 'Goal assists for', 'Goal assists against',
       'Margin']

In [13]:
Gee_stats_temp[['Kicks for','Kicks against']] = Gee_stats['KI'].str.split('-',expand=True)
Gee_stats_temp[['Marks for','Marks against']] = Gee_stats['MK'].str.split('-',expand=True)
Gee_stats_temp[['Handballs for','Handballs against']] = Gee_stats['HB'].str.split('-',expand=True)
Gee_stats_temp[['Disposals for','Disposals against']] = Gee_stats['DI'].str.split('-',expand=True)
Gee_stats_temp[['Hitouts for','Hitouts against']] = Gee_stats['HO'].str.split('-',expand=True)
Gee_stats_temp[['Tackles for','Tackles against']] = Gee_stats['TK'].str.split('-',expand=True)
Gee_stats_temp[['Rebound 50s for','Rebound 50s against']] = Gee_stats['RB'].str.split('-',expand=True)
Gee_stats_temp[['Inside 50s for','Inside 50s against']] = Gee_stats['IF'].str.split('-',expand=True)
Gee_stats_temp[['Clearances for','Clearances against']] = Gee_stats['CL'].str.split('-',expand=True)
Gee_stats_temp[['Clangers for','Clangers against']] = Gee_stats['CG'].str.split('-',expand=True)
Gee_stats_temp[['Frees for','Frees against']] = Gee_stats['FF'].str.split('-',expand=True)
Gee_stats_temp[['Contested possessions for','Contested possessions against']] = Gee_stats['CP'].str.split('-',expand=True)
Gee_stats_temp[['Uncontested possessions for','Uncontested possessions against']] = Gee_stats['UP'].str.split('-',expand=True)
Gee_stats_temp[['Contested marks for','Contested marks against']] = Gee_stats['CM'].str.split('-',expand=True)
Gee_stats_temp[['Marks inside 50 for','Marks inside 50 against']] = Gee_stats['MI'].str.split('-',expand=True)
Gee_stats_temp[['1%s for','1%s against']] = Gee_stats['1%'].str.split('-',expand=True)
Gee_stats_temp[['Bounces for','Bounces against']] = Gee_stats['BO'].str.split('-',expand=True)
Gee_stats_temp[['Goal assists for','Goal assists against']] = Gee_stats['GA'].str.split('-',expand=True)
Gee_stats_temp['Margin'] = Gee_stats['Margin']

Gee_stats_temp['Kicks for'] = pd.to_numeric(Gee_stats_temp['Kicks for'])
Gee_stats_temp['Kicks against'] = pd.to_numeric(Gee_stats_temp['Kicks against'])
Gee_stats_temp['Marks for'] = pd.to_numeric(Gee_stats_temp['Marks for'])
Gee_stats_temp['Marks against'] = pd.to_numeric(Gee_stats_temp['Marks against'])
Gee_stats_temp['Handballs for'] = pd.to_numeric(Gee_stats_temp['Handballs for'])
Gee_stats_temp['Handballs against'] = pd.to_numeric(Gee_stats_temp['Handballs against'])
Gee_stats_temp['Disposals for'] = pd.to_numeric(Gee_stats_temp['Disposals for'])
Gee_stats_temp['Disposals against'] = pd.to_numeric(Gee_stats_temp['Disposals against'])
Gee_stats_temp['Hitouts for'] = pd.to_numeric(Gee_stats_temp['Hitouts for'])
Gee_stats_temp['Hitouts against'] = pd.to_numeric(Gee_stats_temp['Hitouts against'])
Gee_stats_temp['Tackles for'] = pd.to_numeric(Gee_stats_temp['Tackles for'])
Gee_stats_temp['Tackles against'] = pd.to_numeric(Gee_stats_temp['Tackles against'])
Gee_stats_temp['Rebound 50s for'] = pd.to_numeric(Gee_stats_temp['Rebound 50s for'])
Gee_stats_temp['Rebound 50s against'] = pd.to_numeric(Gee_stats_temp['Rebound 50s against'])
Gee_stats_temp['Inside 50s for'] = pd.to_numeric(Gee_stats_temp['Inside 50s for'])
Gee_stats_temp['Inside 50s against'] = pd.to_numeric(Gee_stats_temp['Inside 50s against'])
Gee_stats_temp['Clearances for'] = pd.to_numeric(Gee_stats_temp['Clearances for'])
Gee_stats_temp['Clearances against'] = pd.to_numeric(Gee_stats_temp['Clearances against'])
Gee_stats_temp['Clangers for'] = pd.to_numeric(Gee_stats_temp['Clangers for'])
Gee_stats_temp['Clangers against'] = pd.to_numeric(Gee_stats_temp['Clearances against'])
Gee_stats_temp['Frees for'] = pd.to_numeric(Gee_stats_temp['Frees for'])
Gee_stats_temp['Frees against'] = pd.to_numeric(Gee_stats_temp['Frees against'])
Gee_stats_temp['Contested possessions for'] = pd.to_numeric(Gee_stats_temp['Contested possessions for'])
Gee_stats_temp['Contested possessions against'] = pd.to_numeric(Gee_stats_temp['Contested possessions against'])
Gee_stats_temp['Uncontested possessions for'] = pd.to_numeric(Gee_stats_temp['Uncontested possessions for'])
Gee_stats_temp['Uncontested possessions against'] = pd.to_numeric(Gee_stats_temp['Uncontested possessions against'])
Gee_stats_temp['Contested marks for'] = pd.to_numeric(Gee_stats_temp['Contested marks for'])
Gee_stats_temp['Contested marks against'] = pd.to_numeric(Gee_stats_temp['Contested marks against'])
Gee_stats_temp['Marks inside 50 for'] = pd.to_numeric(Gee_stats_temp['Marks inside 50 for'])
Gee_stats_temp['Marks inside 50 against'] = pd.to_numeric(Gee_stats_temp['Marks inside 50 against'])
Gee_stats_temp['1%s for'] = pd.to_numeric(Gee_stats_temp['1%s for'])
Gee_stats_temp['1%s against'] = pd.to_numeric(Gee_stats_temp['1%s against'])
Gee_stats_temp['Bounces for'] = pd.to_numeric(Gee_stats_temp['Bounces for'])
Gee_stats_temp['Bounces against'] = pd.to_numeric(Gee_stats_temp['Bounces against'])
Gee_stats_temp['Goal assists for'] = pd.to_numeric(Gee_stats_temp['Goal assists for'])
Gee_stats_temp['Goal assists against'] = pd.to_numeric(Gee_stats_temp['Goal assists against'])

Gee = pd.concat([Gee_class, Gee_stats_temp], axis=1)
Gee.columns = ['Header', '#','Game ID','Home', 'Kicks for', 'Kicks against',
       'Marks for', 'Marks against', 'Handballs for', 'Handballs against',
       'Disposals for', 'Disposals against', 'Hitouts for', 'Hitouts against',
       'Tackles for', 'Tackles against', 'Rebound 50s for',
       'Rebound 50s against', 'Inside 50s for', 'Inside 50s against',
       'Clearances for', 'Clearances against', 'Clangers for',
       'Clangers against', 'Frees for', 'Frees against',
       'Contested possessions for', 'Contested possessions against',
       'Uncontested possessions for', 'Uncontested possessions against',
       'Contested marks for', 'Contested marks against', 'Marks inside 50 for',
       'Marks inside 50 against', '1%s for', '1%s against', 'Bounces for',
       'Bounces against', 'Goal assists for', 'Goal assists against',
       'Margin']

In [14]:
Gol_stats_temp[['Kicks for','Kicks against']] = Gol_stats['KI'].str.split('-',expand=True)
Gol_stats_temp[['Marks for','Marks against']] = Gol_stats['MK'].str.split('-',expand=True)
Gol_stats_temp[['Handballs for','Handballs against']] = Gol_stats['HB'].str.split('-',expand=True)
Gol_stats_temp[['Disposals for','Disposals against']] = Gol_stats['DI'].str.split('-',expand=True)
Gol_stats_temp[['Hitouts for','Hitouts against']] = Gol_stats['HO'].str.split('-',expand=True)
Gol_stats_temp[['Tackles for','Tackles against']] = Gol_stats['TK'].str.split('-',expand=True)
Gol_stats_temp[['Rebound 50s for','Rebound 50s against']] = Gol_stats['RB'].str.split('-',expand=True)
Gol_stats_temp[['Inside 50s for','Inside 50s against']] = Gol_stats['IF'].str.split('-',expand=True)
Gol_stats_temp[['Clearances for','Clearances against']] = Gol_stats['CL'].str.split('-',expand=True)
Gol_stats_temp[['Clangers for','Clangers against']] = Gol_stats['CG'].str.split('-',expand=True)
Gol_stats_temp[['Frees for','Frees against']] = Gol_stats['FF'].str.split('-',expand=True)
Gol_stats_temp[['Contested possessions for','Contested possessions against']] = Gol_stats['CP'].str.split('-',expand=True)
Gol_stats_temp[['Uncontested possessions for','Uncontested possessions against']] = Gol_stats['UP'].str.split('-',expand=True)
Gol_stats_temp[['Contested marks for','Contested marks against']] = Gol_stats['CM'].str.split('-',expand=True)
Gol_stats_temp[['Marks inside 50 for','Marks inside 50 against']] = Gol_stats['MI'].str.split('-',expand=True)
Gol_stats_temp[['1%s for','1%s against']] = Gol_stats['1%'].str.split('-',expand=True)
Gol_stats_temp[['Bounces for','Bounces against']] = Gol_stats['BO'].str.split('-',expand=True)
Gol_stats_temp[['Goal assists for','Goal assists against']] = Gol_stats['GA'].str.split('-',expand=True)
Gol_stats_temp['Margin'] = Gol_stats['Margin']

Gol_stats_temp['Kicks for'] = pd.to_numeric(Gol_stats_temp['Kicks for'])
Gol_stats_temp['Kicks against'] = pd.to_numeric(Gol_stats_temp['Kicks against'])
Gol_stats_temp['Marks for'] = pd.to_numeric(Gol_stats_temp['Marks for'])
Gol_stats_temp['Marks against'] = pd.to_numeric(Gol_stats_temp['Marks against'])
Gol_stats_temp['Handballs for'] = pd.to_numeric(Gol_stats_temp['Handballs for'])
Gol_stats_temp['Handballs against'] = pd.to_numeric(Gol_stats_temp['Handballs against'])
Gol_stats_temp['Disposals for'] = pd.to_numeric(Gol_stats_temp['Disposals for'])
Gol_stats_temp['Disposals against'] = pd.to_numeric(Gol_stats_temp['Disposals against'])
Gol_stats_temp['Hitouts for'] = pd.to_numeric(Gol_stats_temp['Hitouts for'])
Gol_stats_temp['Hitouts against'] = pd.to_numeric(Gol_stats_temp['Hitouts against'])
Gol_stats_temp['Tackles for'] = pd.to_numeric(Gol_stats_temp['Tackles for'])
Gol_stats_temp['Tackles against'] = pd.to_numeric(Gol_stats_temp['Tackles against'])
Gol_stats_temp['Rebound 50s for'] = pd.to_numeric(Gol_stats_temp['Rebound 50s for'])
Gol_stats_temp['Rebound 50s against'] = pd.to_numeric(Gol_stats_temp['Rebound 50s against'])
Gol_stats_temp['Inside 50s for'] = pd.to_numeric(Gol_stats_temp['Inside 50s for'])
Gol_stats_temp['Inside 50s against'] = pd.to_numeric(Gol_stats_temp['Inside 50s against'])
Gol_stats_temp['Clearances for'] = pd.to_numeric(Gol_stats_temp['Clearances for'])
Gol_stats_temp['Clearances against'] = pd.to_numeric(Gol_stats_temp['Clearances against'])
Gol_stats_temp['Clangers for'] = pd.to_numeric(Gol_stats_temp['Clangers for'])
Gol_stats_temp['Clangers against'] = pd.to_numeric(Gol_stats_temp['Clearances against'])
Gol_stats_temp['Frees for'] = pd.to_numeric(Gol_stats_temp['Frees for'])
Gol_stats_temp['Frees against'] = pd.to_numeric(Gol_stats_temp['Frees against'])
Gol_stats_temp['Contested possessions for'] = pd.to_numeric(Gol_stats_temp['Contested possessions for'])
Gol_stats_temp['Contested possessions against'] = pd.to_numeric(Gol_stats_temp['Contested possessions against'])
Gol_stats_temp['Uncontested possessions for'] = pd.to_numeric(Gol_stats_temp['Uncontested possessions for'])
Gol_stats_temp['Uncontested possessions against'] = pd.to_numeric(Gol_stats_temp['Uncontested possessions against'])
Gol_stats_temp['Contested marks for'] = pd.to_numeric(Gol_stats_temp['Contested marks for'])
Gol_stats_temp['Contested marks against'] = pd.to_numeric(Gol_stats_temp['Contested marks against'])
Gol_stats_temp['Marks inside 50 for'] = pd.to_numeric(Gol_stats_temp['Marks inside 50 for'])
Gol_stats_temp['Marks inside 50 against'] = pd.to_numeric(Gol_stats_temp['Marks inside 50 against'])
Gol_stats_temp['1%s for'] = pd.to_numeric(Gol_stats_temp['1%s for'])
Gol_stats_temp['1%s against'] = pd.to_numeric(Gol_stats_temp['1%s against'])
Gol_stats_temp['Bounces for'] = pd.to_numeric(Gol_stats_temp['Bounces for'])
Gol_stats_temp['Bounces against'] = pd.to_numeric(Gol_stats_temp['Bounces against'])
Gol_stats_temp['Goal assists for'] = pd.to_numeric(Gol_stats_temp['Goal assists for'])
Gol_stats_temp['Goal assists against'] = pd.to_numeric(Gol_stats_temp['Goal assists against'])

Gol = pd.concat([Gol_class, Gol_stats_temp], axis=1)
Gol.columns = ['Header', '#','Game ID','Home', 'Kicks for', 'Kicks against',
       'Marks for', 'Marks against', 'Handballs for', 'Handballs against',
       'Disposals for', 'Disposals against', 'Hitouts for', 'Hitouts against',
       'Tackles for', 'Tackles against', 'Rebound 50s for',
       'Rebound 50s against', 'Inside 50s for', 'Inside 50s against',
       'Clearances for', 'Clearances against', 'Clangers for',
       'Clangers against', 'Frees for', 'Frees against',
       'Contested possessions for', 'Contested possessions against',
       'Uncontested possessions for', 'Uncontested possessions against',
       'Contested marks for', 'Contested marks against', 'Marks inside 50 for',
       'Marks inside 50 against', '1%s for', '1%s against', 'Bounces for',
       'Bounces against', 'Goal assists for', 'Goal assists against',
       'Margin']

In [15]:
GWS_stats_temp[['Kicks for','Kicks against']] = GWS_stats['KI'].str.split('-',expand=True)
GWS_stats_temp[['Marks for','Marks against']] = GWS_stats['MK'].str.split('-',expand=True)
GWS_stats_temp[['Handballs for','Handballs against']] = GWS_stats['HB'].str.split('-',expand=True)
GWS_stats_temp[['Disposals for','Disposals against']] = GWS_stats['DI'].str.split('-',expand=True)
GWS_stats_temp[['Hitouts for','Hitouts against']] = GWS_stats['HO'].str.split('-',expand=True)
GWS_stats_temp[['Tackles for','Tackles against']] = GWS_stats['TK'].str.split('-',expand=True)
GWS_stats_temp[['Rebound 50s for','Rebound 50s against']] = GWS_stats['RB'].str.split('-',expand=True)
GWS_stats_temp[['Inside 50s for','Inside 50s against']] = GWS_stats['IF'].str.split('-',expand=True)
GWS_stats_temp[['Clearances for','Clearances against']] = GWS_stats['CL'].str.split('-',expand=True)
GWS_stats_temp[['Clangers for','Clangers against']] = GWS_stats['CG'].str.split('-',expand=True)
GWS_stats_temp[['Frees for','Frees against']] = GWS_stats['FF'].str.split('-',expand=True)
GWS_stats_temp[['Contested possessions for','Contested possessions against']] = GWS_stats['CP'].str.split('-',expand=True)
GWS_stats_temp[['Uncontested possessions for','Uncontested possessions against']] = GWS_stats['UP'].str.split('-',expand=True)
GWS_stats_temp[['Contested marks for','Contested marks against']] = GWS_stats['CM'].str.split('-',expand=True)
GWS_stats_temp[['Marks inside 50 for','Marks inside 50 against']] = GWS_stats['MI'].str.split('-',expand=True)
GWS_stats_temp[['1%s for','1%s against']] = GWS_stats['1%'].str.split('-',expand=True)
GWS_stats_temp[['Bounces for','Bounces against']] = GWS_stats['BO'].str.split('-',expand=True)
GWS_stats_temp[['Goal assists for','Goal assists against']] = GWS_stats['GA'].str.split('-',expand=True)
GWS_stats_temp['Margin'] = GWS_stats['Margin']

GWS_stats_temp['Kicks for'] = pd.to_numeric(GWS_stats_temp['Kicks for'])
GWS_stats_temp['Kicks against'] = pd.to_numeric(GWS_stats_temp['Kicks against'])
GWS_stats_temp['Marks for'] = pd.to_numeric(GWS_stats_temp['Marks for'])
GWS_stats_temp['Marks against'] = pd.to_numeric(GWS_stats_temp['Marks against'])
GWS_stats_temp['Handballs for'] = pd.to_numeric(GWS_stats_temp['Handballs for'])
GWS_stats_temp['Handballs against'] = pd.to_numeric(GWS_stats_temp['Handballs against'])
GWS_stats_temp['Disposals for'] = pd.to_numeric(GWS_stats_temp['Disposals for'])
GWS_stats_temp['Disposals against'] = pd.to_numeric(GWS_stats_temp['Disposals against'])
GWS_stats_temp['Hitouts for'] = pd.to_numeric(GWS_stats_temp['Hitouts for'])
GWS_stats_temp['Hitouts against'] = pd.to_numeric(GWS_stats_temp['Hitouts against'])
GWS_stats_temp['Tackles for'] = pd.to_numeric(GWS_stats_temp['Tackles for'])
GWS_stats_temp['Tackles against'] = pd.to_numeric(GWS_stats_temp['Tackles against'])
GWS_stats_temp['Rebound 50s for'] = pd.to_numeric(GWS_stats_temp['Rebound 50s for'])
GWS_stats_temp['Rebound 50s against'] = pd.to_numeric(GWS_stats_temp['Rebound 50s against'])
GWS_stats_temp['Inside 50s for'] = pd.to_numeric(GWS_stats_temp['Inside 50s for'])
GWS_stats_temp['Inside 50s against'] = pd.to_numeric(GWS_stats_temp['Inside 50s against'])
GWS_stats_temp['Clearances for'] = pd.to_numeric(GWS_stats_temp['Clearances for'])
GWS_stats_temp['Clearances against'] = pd.to_numeric(GWS_stats_temp['Clearances against'])
GWS_stats_temp['Clangers for'] = pd.to_numeric(GWS_stats_temp['Clangers for'])
GWS_stats_temp['Clangers against'] = pd.to_numeric(GWS_stats_temp['Clearances against'])
GWS_stats_temp['Frees for'] = pd.to_numeric(GWS_stats_temp['Frees for'])
GWS_stats_temp['Frees against'] = pd.to_numeric(GWS_stats_temp['Frees against'])
GWS_stats_temp['Contested possessions for'] = pd.to_numeric(GWS_stats_temp['Contested possessions for'])
GWS_stats_temp['Contested possessions against'] = pd.to_numeric(GWS_stats_temp['Contested possessions against'])
GWS_stats_temp['Uncontested possessions for'] = pd.to_numeric(GWS_stats_temp['Uncontested possessions for'])
GWS_stats_temp['Uncontested possessions against'] = pd.to_numeric(GWS_stats_temp['Uncontested possessions against'])
GWS_stats_temp['Contested marks for'] = pd.to_numeric(GWS_stats_temp['Contested marks for'])
GWS_stats_temp['Contested marks against'] = pd.to_numeric(GWS_stats_temp['Contested marks against'])
GWS_stats_temp['Marks inside 50 for'] = pd.to_numeric(GWS_stats_temp['Marks inside 50 for'])
GWS_stats_temp['Marks inside 50 against'] = pd.to_numeric(GWS_stats_temp['Marks inside 50 against'])
GWS_stats_temp['1%s for'] = pd.to_numeric(GWS_stats_temp['1%s for'])
GWS_stats_temp['1%s against'] = pd.to_numeric(GWS_stats_temp['1%s against'])
GWS_stats_temp['Bounces for'] = pd.to_numeric(GWS_stats_temp['Bounces for'])
GWS_stats_temp['Bounces against'] = pd.to_numeric(GWS_stats_temp['Bounces against'])
GWS_stats_temp['Goal assists for'] = pd.to_numeric(GWS_stats_temp['Goal assists for'])
GWS_stats_temp['Goal assists against'] = pd.to_numeric(GWS_stats_temp['Goal assists against'])

GWS = pd.concat([GWS_class, GWS_stats_temp], axis=1)
GWS.columns = ['Header', '#','Game ID','Home', 'Kicks for', 'Kicks against',
       'Marks for', 'Marks against', 'Handballs for', 'Handballs against',
       'Disposals for', 'Disposals against', 'Hitouts for', 'Hitouts against',
       'Tackles for', 'Tackles against', 'Rebound 50s for',
       'Rebound 50s against', 'Inside 50s for', 'Inside 50s against',
       'Clearances for', 'Clearances against', 'Clangers for',
       'Clangers against', 'Frees for', 'Frees against',
       'Contested possessions for', 'Contested possessions against',
       'Uncontested possessions for', 'Uncontested possessions against',
       'Contested marks for', 'Contested marks against', 'Marks inside 50 for',
       'Marks inside 50 against', '1%s for', '1%s against', 'Bounces for',
       'Bounces against', 'Goal assists for', 'Goal assists against',
       'Margin']

In [16]:
Haw_stats_temp[['Kicks for','Kicks against']] = Haw_stats['KI'].str.split('-',expand=True)
Haw_stats_temp[['Marks for','Marks against']] = Haw_stats['MK'].str.split('-',expand=True)
Haw_stats_temp[['Handballs for','Handballs against']] = Haw_stats['HB'].str.split('-',expand=True)
Haw_stats_temp[['Disposals for','Disposals against']] = Haw_stats['DI'].str.split('-',expand=True)
Haw_stats_temp[['Hitouts for','Hitouts against']] = Haw_stats['HO'].str.split('-',expand=True)
Haw_stats_temp[['Tackles for','Tackles against']] = Haw_stats['TK'].str.split('-',expand=True)
Haw_stats_temp[['Rebound 50s for','Rebound 50s against']] = Haw_stats['RB'].str.split('-',expand=True)
Haw_stats_temp[['Inside 50s for','Inside 50s against']] = Haw_stats['IF'].str.split('-',expand=True)
Haw_stats_temp[['Clearances for','Clearances against']] = Haw_stats['CL'].str.split('-',expand=True)
Haw_stats_temp[['Clangers for','Clangers against']] = Haw_stats['CG'].str.split('-',expand=True)
Haw_stats_temp[['Frees for','Frees against']] = Haw_stats['FF'].str.split('-',expand=True)
Haw_stats_temp[['Contested possessions for','Contested possessions against']] = Haw_stats['CP'].str.split('-',expand=True)
Haw_stats_temp[['Uncontested possessions for','Uncontested possessions against']] = Haw_stats['UP'].str.split('-',expand=True)
Haw_stats_temp[['Contested marks for','Contested marks against']] = Haw_stats['CM'].str.split('-',expand=True)
Haw_stats_temp[['Marks inside 50 for','Marks inside 50 against']] = Haw_stats['MI'].str.split('-',expand=True)
Haw_stats_temp[['1%s for','1%s against']] = Haw_stats['1%'].str.split('-',expand=True)
Haw_stats_temp[['Bounces for','Bounces against']] = Haw_stats['BO'].str.split('-',expand=True)
Haw_stats_temp[['Goal assists for','Goal assists against']] = Haw_stats['GA'].str.split('-',expand=True)
Haw_stats_temp['Margin'] = Haw_stats['Margin']

Haw_stats_temp['Kicks for'] = pd.to_numeric(Haw_stats_temp['Kicks for'])
Haw_stats_temp['Kicks against'] = pd.to_numeric(Haw_stats_temp['Kicks against'])
Haw_stats_temp['Marks for'] = pd.to_numeric(Haw_stats_temp['Marks for'])
Haw_stats_temp['Marks against'] = pd.to_numeric(Haw_stats_temp['Marks against'])
Haw_stats_temp['Handballs for'] = pd.to_numeric(Haw_stats_temp['Handballs for'])
Haw_stats_temp['Handballs against'] = pd.to_numeric(Haw_stats_temp['Handballs against'])
Haw_stats_temp['Disposals for'] = pd.to_numeric(Haw_stats_temp['Disposals for'])
Haw_stats_temp['Disposals against'] = pd.to_numeric(Haw_stats_temp['Disposals against'])
Haw_stats_temp['Hitouts for'] = pd.to_numeric(Haw_stats_temp['Hitouts for'])
Haw_stats_temp['Hitouts against'] = pd.to_numeric(Haw_stats_temp['Hitouts against'])
Haw_stats_temp['Tackles for'] = pd.to_numeric(Haw_stats_temp['Tackles for'])
Haw_stats_temp['Tackles against'] = pd.to_numeric(Haw_stats_temp['Tackles against'])
Haw_stats_temp['Rebound 50s for'] = pd.to_numeric(Haw_stats_temp['Rebound 50s for'])
Haw_stats_temp['Rebound 50s against'] = pd.to_numeric(Haw_stats_temp['Rebound 50s against'])
Haw_stats_temp['Inside 50s for'] = pd.to_numeric(Haw_stats_temp['Inside 50s for'])
Haw_stats_temp['Inside 50s against'] = pd.to_numeric(Haw_stats_temp['Inside 50s against'])
Haw_stats_temp['Clearances for'] = pd.to_numeric(Haw_stats_temp['Clearances for'])
Haw_stats_temp['Clearances against'] = pd.to_numeric(Haw_stats_temp['Clearances against'])
Haw_stats_temp['Clangers for'] = pd.to_numeric(Haw_stats_temp['Clangers for'])
Haw_stats_temp['Clangers against'] = pd.to_numeric(Haw_stats_temp['Clearances against'])
Haw_stats_temp['Frees for'] = pd.to_numeric(Haw_stats_temp['Frees for'])
Haw_stats_temp['Frees against'] = pd.to_numeric(Haw_stats_temp['Frees against'])
Haw_stats_temp['Contested possessions for'] = pd.to_numeric(Haw_stats_temp['Contested possessions for'])
Haw_stats_temp['Contested possessions against'] = pd.to_numeric(Haw_stats_temp['Contested possessions against'])
Haw_stats_temp['Uncontested possessions for'] = pd.to_numeric(Haw_stats_temp['Uncontested possessions for'])
Haw_stats_temp['Uncontested possessions against'] = pd.to_numeric(Haw_stats_temp['Uncontested possessions against'])
Haw_stats_temp['Contested marks for'] = pd.to_numeric(Haw_stats_temp['Contested marks for'])
Haw_stats_temp['Contested marks against'] = pd.to_numeric(Haw_stats_temp['Contested marks against'])
Haw_stats_temp['Marks inside 50 for'] = pd.to_numeric(Haw_stats_temp['Marks inside 50 for'])
Haw_stats_temp['Marks inside 50 against'] = pd.to_numeric(Haw_stats_temp['Marks inside 50 against'])
Haw_stats_temp['1%s for'] = pd.to_numeric(Haw_stats_temp['1%s for'])
Haw_stats_temp['1%s against'] = pd.to_numeric(Haw_stats_temp['1%s against'])
Haw_stats_temp['Bounces for'] = pd.to_numeric(Haw_stats_temp['Bounces for'])
Haw_stats_temp['Bounces against'] = pd.to_numeric(Haw_stats_temp['Bounces against'])
Haw_stats_temp['Goal assists for'] = pd.to_numeric(Haw_stats_temp['Goal assists for'])
Haw_stats_temp['Goal assists against'] = pd.to_numeric(Haw_stats_temp['Goal assists against'])

Haw = pd.concat([Haw_class, Haw_stats_temp], axis=1)
Haw.columns = ['Header', '#','Game ID','Home', 'Kicks for', 'Kicks against',
       'Marks for', 'Marks against', 'Handballs for', 'Handballs against',
       'Disposals for', 'Disposals against', 'Hitouts for', 'Hitouts against',
       'Tackles for', 'Tackles against', 'Rebound 50s for',
       'Rebound 50s against', 'Inside 50s for', 'Inside 50s against',
       'Clearances for', 'Clearances against', 'Clangers for',
       'Clangers against', 'Frees for', 'Frees against',
       'Contested possessions for', 'Contested possessions against',
       'Uncontested possessions for', 'Uncontested possessions against',
       'Contested marks for', 'Contested marks against', 'Marks inside 50 for',
       'Marks inside 50 against', '1%s for', '1%s against', 'Bounces for',
       'Bounces against', 'Goal assists for', 'Goal assists against',
       'Margin']

In [17]:
Mel_stats_temp[['Kicks for','Kicks against']] = Mel_stats['KI'].str.split('-',expand=True)
Mel_stats_temp[['Marks for','Marks against']] = Mel_stats['MK'].str.split('-',expand=True)
Mel_stats_temp[['Handballs for','Handballs against']] = Mel_stats['HB'].str.split('-',expand=True)
Mel_stats_temp[['Disposals for','Disposals against']] = Mel_stats['DI'].str.split('-',expand=True)
Mel_stats_temp[['Hitouts for','Hitouts against']] = Mel_stats['HO'].str.split('-',expand=True)
Mel_stats_temp[['Tackles for','Tackles against']] = Mel_stats['TK'].str.split('-',expand=True)
Mel_stats_temp[['Rebound 50s for','Rebound 50s against']] = Mel_stats['RB'].str.split('-',expand=True)
Mel_stats_temp[['Inside 50s for','Inside 50s against']] = Mel_stats['IF'].str.split('-',expand=True)
Mel_stats_temp[['Clearances for','Clearances against']] = Mel_stats['CL'].str.split('-',expand=True)
Mel_stats_temp[['Clangers for','Clangers against']] = Mel_stats['CG'].str.split('-',expand=True)
Mel_stats_temp[['Frees for','Frees against']] = Mel_stats['FF'].str.split('-',expand=True)
Mel_stats_temp[['Contested possessions for','Contested possessions against']] = Mel_stats['CP'].str.split('-',expand=True)
Mel_stats_temp[['Uncontested possessions for','Uncontested possessions against']] = Mel_stats['UP'].str.split('-',expand=True)
Mel_stats_temp[['Contested marks for','Contested marks against']] = Mel_stats['CM'].str.split('-',expand=True)
Mel_stats_temp[['Marks inside 50 for','Marks inside 50 against']] = Mel_stats['MI'].str.split('-',expand=True)
Mel_stats_temp[['1%s for','1%s against']] = Mel_stats['1%'].str.split('-',expand=True)
Mel_stats_temp[['Bounces for','Bounces against']] = Mel_stats['BO'].str.split('-',expand=True)
Mel_stats_temp[['Goal assists for','Goal assists against']] = Mel_stats['GA'].str.split('-',expand=True)
Mel_stats_temp['Margin'] = Mel_stats['Margin']

Mel_stats_temp['Kicks for'] = pd.to_numeric(Mel_stats_temp['Kicks for'])
Mel_stats_temp['Kicks against'] = pd.to_numeric(Mel_stats_temp['Kicks against'])
Mel_stats_temp['Marks for'] = pd.to_numeric(Mel_stats_temp['Marks for'])
Mel_stats_temp['Marks against'] = pd.to_numeric(Mel_stats_temp['Marks against'])
Mel_stats_temp['Handballs for'] = pd.to_numeric(Mel_stats_temp['Handballs for'])
Mel_stats_temp['Handballs against'] = pd.to_numeric(Mel_stats_temp['Handballs against'])
Mel_stats_temp['Disposals for'] = pd.to_numeric(Mel_stats_temp['Disposals for'])
Mel_stats_temp['Disposals against'] = pd.to_numeric(Mel_stats_temp['Disposals against'])
Mel_stats_temp['Hitouts for'] = pd.to_numeric(Mel_stats_temp['Hitouts for'])
Mel_stats_temp['Hitouts against'] = pd.to_numeric(Mel_stats_temp['Hitouts against'])
Mel_stats_temp['Tackles for'] = pd.to_numeric(Mel_stats_temp['Tackles for'])
Mel_stats_temp['Tackles against'] = pd.to_numeric(Mel_stats_temp['Tackles against'])
Mel_stats_temp['Rebound 50s for'] = pd.to_numeric(Mel_stats_temp['Rebound 50s for'])
Mel_stats_temp['Rebound 50s against'] = pd.to_numeric(Mel_stats_temp['Rebound 50s against'])
Mel_stats_temp['Inside 50s for'] = pd.to_numeric(Mel_stats_temp['Inside 50s for'])
Mel_stats_temp['Inside 50s against'] = pd.to_numeric(Mel_stats_temp['Inside 50s against'])
Mel_stats_temp['Clearances for'] = pd.to_numeric(Mel_stats_temp['Clearances for'])
Mel_stats_temp['Clearances against'] = pd.to_numeric(Mel_stats_temp['Clearances against'])
Mel_stats_temp['Clangers for'] = pd.to_numeric(Mel_stats_temp['Clangers for'])
Mel_stats_temp['Clangers against'] = pd.to_numeric(Mel_stats_temp['Clearances against'])
Mel_stats_temp['Frees for'] = pd.to_numeric(Mel_stats_temp['Frees for'])
Mel_stats_temp['Frees against'] = pd.to_numeric(Mel_stats_temp['Frees against'])
Mel_stats_temp['Contested possessions for'] = pd.to_numeric(Mel_stats_temp['Contested possessions for'])
Mel_stats_temp['Contested possessions against'] = pd.to_numeric(Mel_stats_temp['Contested possessions against'])
Mel_stats_temp['Uncontested possessions for'] = pd.to_numeric(Mel_stats_temp['Uncontested possessions for'])
Mel_stats_temp['Uncontested possessions against'] = pd.to_numeric(Mel_stats_temp['Uncontested possessions against'])
Mel_stats_temp['Contested marks for'] = pd.to_numeric(Mel_stats_temp['Contested marks for'])
Mel_stats_temp['Contested marks against'] = pd.to_numeric(Mel_stats_temp['Contested marks against'])
Mel_stats_temp['Marks inside 50 for'] = pd.to_numeric(Mel_stats_temp['Marks inside 50 for'])
Mel_stats_temp['Marks inside 50 against'] = pd.to_numeric(Mel_stats_temp['Marks inside 50 against'])
Mel_stats_temp['1%s for'] = pd.to_numeric(Mel_stats_temp['1%s for'])
Mel_stats_temp['1%s against'] = pd.to_numeric(Mel_stats_temp['1%s against'])
Mel_stats_temp['Bounces for'] = pd.to_numeric(Mel_stats_temp['Bounces for'])
Mel_stats_temp['Bounces against'] = pd.to_numeric(Mel_stats_temp['Bounces against'])
Mel_stats_temp['Goal assists for'] = pd.to_numeric(Mel_stats_temp['Goal assists for'])
Mel_stats_temp['Goal assists against'] = pd.to_numeric(Mel_stats_temp['Goal assists against'])

Mel = pd.concat([Mel_class, Mel_stats_temp], axis=1)
Mel.columns = ['Header', '#','Game ID','Home', 'Kicks for', 'Kicks against',
       'Marks for', 'Marks against', 'Handballs for', 'Handballs against',
       'Disposals for', 'Disposals against', 'Hitouts for', 'Hitouts against',
       'Tackles for', 'Tackles against', 'Rebound 50s for',
       'Rebound 50s against', 'Inside 50s for', 'Inside 50s against',
       'Clearances for', 'Clearances against', 'Clangers for',
       'Clangers against', 'Frees for', 'Frees against',
       'Contested possessions for', 'Contested possessions against',
       'Uncontested possessions for', 'Uncontested possessions against',
       'Contested marks for', 'Contested marks against', 'Marks inside 50 for',
       'Marks inside 50 against', '1%s for', '1%s against', 'Bounces for',
       'Bounces against', 'Goal assists for', 'Goal assists against',
       'Margin']

In [18]:
Nor_stats_temp[['Kicks for','Kicks against']] = Nor_stats['KI'].str.split('-',expand=True)
Nor_stats_temp[['Marks for','Marks against']] = Nor_stats['MK'].str.split('-',expand=True)
Nor_stats_temp[['Handballs for','Handballs against']] = Nor_stats['HB'].str.split('-',expand=True)
Nor_stats_temp[['Disposals for','Disposals against']] = Nor_stats['DI'].str.split('-',expand=True)
Nor_stats_temp[['Hitouts for','Hitouts against']] = Nor_stats['HO'].str.split('-',expand=True)
Nor_stats_temp[['Tackles for','Tackles against']] = Nor_stats['TK'].str.split('-',expand=True)
Nor_stats_temp[['Rebound 50s for','Rebound 50s against']] = Nor_stats['RB'].str.split('-',expand=True)
Nor_stats_temp[['Inside 50s for','Inside 50s against']] = Nor_stats['IF'].str.split('-',expand=True)
Nor_stats_temp[['Clearances for','Clearances against']] = Nor_stats['CL'].str.split('-',expand=True)
Nor_stats_temp[['Clangers for','Clangers against']] = Nor_stats['CG'].str.split('-',expand=True)
Nor_stats_temp[['Frees for','Frees against']] = Nor_stats['FF'].str.split('-',expand=True)
Nor_stats_temp[['Contested possessions for','Contested possessions against']] = Nor_stats['CP'].str.split('-',expand=True)
Nor_stats_temp[['Uncontested possessions for','Uncontested possessions against']] = Nor_stats['UP'].str.split('-',expand=True)
Nor_stats_temp[['Contested marks for','Contested marks against']] = Nor_stats['CM'].str.split('-',expand=True)
Nor_stats_temp[['Marks inside 50 for','Marks inside 50 against']] = Nor_stats['MI'].str.split('-',expand=True)
Nor_stats_temp[['1%s for','1%s against']] = Nor_stats['1%'].str.split('-',expand=True)
Nor_stats_temp[['Bounces for','Bounces against']] = Nor_stats['BO'].str.split('-',expand=True)
Nor_stats_temp[['Goal assists for','Goal assists against']] = Nor_stats['GA'].str.split('-',expand=True)
Nor_stats_temp['Margin'] = Nor_stats['Margin']

Nor_stats_temp['Kicks for'] = pd.to_numeric(Nor_stats_temp['Kicks for'])
Nor_stats_temp['Kicks against'] = pd.to_numeric(Nor_stats_temp['Kicks against'])
Nor_stats_temp['Marks for'] = pd.to_numeric(Nor_stats_temp['Marks for'])
Nor_stats_temp['Marks against'] = pd.to_numeric(Nor_stats_temp['Marks against'])
Nor_stats_temp['Handballs for'] = pd.to_numeric(Nor_stats_temp['Handballs for'])
Nor_stats_temp['Handballs against'] = pd.to_numeric(Nor_stats_temp['Handballs against'])
Nor_stats_temp['Disposals for'] = pd.to_numeric(Nor_stats_temp['Disposals for'])
Nor_stats_temp['Disposals against'] = pd.to_numeric(Nor_stats_temp['Disposals against'])
Nor_stats_temp['Hitouts for'] = pd.to_numeric(Nor_stats_temp['Hitouts for'])
Nor_stats_temp['Hitouts against'] = pd.to_numeric(Nor_stats_temp['Hitouts against'])
Nor_stats_temp['Tackles for'] = pd.to_numeric(Nor_stats_temp['Tackles for'])
Nor_stats_temp['Tackles against'] = pd.to_numeric(Nor_stats_temp['Tackles against'])
Nor_stats_temp['Rebound 50s for'] = pd.to_numeric(Nor_stats_temp['Rebound 50s for'])
Nor_stats_temp['Rebound 50s against'] = pd.to_numeric(Nor_stats_temp['Rebound 50s against'])
Nor_stats_temp['Inside 50s for'] = pd.to_numeric(Nor_stats_temp['Inside 50s for'])
Nor_stats_temp['Inside 50s against'] = pd.to_numeric(Nor_stats_temp['Inside 50s against'])
Nor_stats_temp['Clearances for'] = pd.to_numeric(Nor_stats_temp['Clearances for'])
Nor_stats_temp['Clearances against'] = pd.to_numeric(Nor_stats_temp['Clearances against'])
Nor_stats_temp['Clangers for'] = pd.to_numeric(Nor_stats_temp['Clangers for'])
Nor_stats_temp['Clangers against'] = pd.to_numeric(Nor_stats_temp['Clearances against'])
Nor_stats_temp['Frees for'] = pd.to_numeric(Nor_stats_temp['Frees for'])
Nor_stats_temp['Frees against'] = pd.to_numeric(Nor_stats_temp['Frees against'])
Nor_stats_temp['Contested possessions for'] = pd.to_numeric(Nor_stats_temp['Contested possessions for'])
Nor_stats_temp['Contested possessions against'] = pd.to_numeric(Nor_stats_temp['Contested possessions against'])
Nor_stats_temp['Uncontested possessions for'] = pd.to_numeric(Nor_stats_temp['Uncontested possessions for'])
Nor_stats_temp['Uncontested possessions against'] = pd.to_numeric(Nor_stats_temp['Uncontested possessions against'])
Nor_stats_temp['Contested marks for'] = pd.to_numeric(Nor_stats_temp['Contested marks for'])
Nor_stats_temp['Contested marks against'] = pd.to_numeric(Nor_stats_temp['Contested marks against'])
Nor_stats_temp['Marks inside 50 for'] = pd.to_numeric(Nor_stats_temp['Marks inside 50 for'])
Nor_stats_temp['Marks inside 50 against'] = pd.to_numeric(Nor_stats_temp['Marks inside 50 against'])
Nor_stats_temp['1%s for'] = pd.to_numeric(Nor_stats_temp['1%s for'])
Nor_stats_temp['1%s against'] = pd.to_numeric(Nor_stats_temp['1%s against'])
Nor_stats_temp['Bounces for'] = pd.to_numeric(Nor_stats_temp['Bounces for'])
Nor_stats_temp['Bounces against'] = pd.to_numeric(Nor_stats_temp['Bounces against'])
Nor_stats_temp['Goal assists for'] = pd.to_numeric(Nor_stats_temp['Goal assists for'])
Nor_stats_temp['Goal assists against'] = pd.to_numeric(Nor_stats_temp['Goal assists against'])

Nor = pd.concat([Nor_class, Nor_stats_temp], axis=1)
Nor.columns = ['Header', '#','Game ID','Home', 'Kicks for', 'Kicks against',
       'Marks for', 'Marks against', 'Handballs for', 'Handballs against',
       'Disposals for', 'Disposals against', 'Hitouts for', 'Hitouts against',
       'Tackles for', 'Tackles against', 'Rebound 50s for',
       'Rebound 50s against', 'Inside 50s for', 'Inside 50s against',
       'Clearances for', 'Clearances against', 'Clangers for',
       'Clangers against', 'Frees for', 'Frees against',
       'Contested possessions for', 'Contested possessions against',
       'Uncontested possessions for', 'Uncontested possessions against',
       'Contested marks for', 'Contested marks against', 'Marks inside 50 for',
       'Marks inside 50 against', '1%s for', '1%s against', 'Bounces for',
       'Bounces against', 'Goal assists for', 'Goal assists against',
       'Margin']

In [19]:
Por_stats_temp[['Kicks for','Kicks against']] = Por_stats['KI'].str.split('-',expand=True)
Por_stats_temp[['Marks for','Marks against']] = Por_stats['MK'].str.split('-',expand=True)
Por_stats_temp[['Handballs for','Handballs against']] = Por_stats['HB'].str.split('-',expand=True)
Por_stats_temp[['Disposals for','Disposals against']] = Por_stats['DI'].str.split('-',expand=True)
Por_stats_temp[['Hitouts for','Hitouts against']] = Por_stats['HO'].str.split('-',expand=True)
Por_stats_temp[['Tackles for','Tackles against']] = Por_stats['TK'].str.split('-',expand=True)
Por_stats_temp[['Rebound 50s for','Rebound 50s against']] = Por_stats['RB'].str.split('-',expand=True)
Por_stats_temp[['Inside 50s for','Inside 50s against']] = Por_stats['IF'].str.split('-',expand=True)
Por_stats_temp[['Clearances for','Clearances against']] = Por_stats['CL'].str.split('-',expand=True)
Por_stats_temp[['Clangers for','Clangers against']] = Por_stats['CG'].str.split('-',expand=True)
Por_stats_temp[['Frees for','Frees against']] = Por_stats['FF'].str.split('-',expand=True)
Por_stats_temp[['Contested possessions for','Contested possessions against']] = Por_stats['CP'].str.split('-',expand=True)
Por_stats_temp[['Uncontested possessions for','Uncontested possessions against']] = Por_stats['UP'].str.split('-',expand=True)
Por_stats_temp[['Contested marks for','Contested marks against']] = Por_stats['CM'].str.split('-',expand=True)
Por_stats_temp[['Marks inside 50 for','Marks inside 50 against']] = Por_stats['MI'].str.split('-',expand=True)
Por_stats_temp[['1%s for','1%s against']] = Por_stats['1%'].str.split('-',expand=True)
Por_stats_temp[['Bounces for','Bounces against']] = Por_stats['BO'].str.split('-',expand=True)
Por_stats_temp[['Goal assists for','Goal assists against']] = Por_stats['GA'].str.split('-',expand=True)
Por_stats_temp['Margin'] = Por_stats['Margin']

Por_stats_temp['Kicks for'] = pd.to_numeric(Por_stats_temp['Kicks for'])
Por_stats_temp['Kicks against'] = pd.to_numeric(Por_stats_temp['Kicks against'])
Por_stats_temp['Marks for'] = pd.to_numeric(Por_stats_temp['Marks for'])
Por_stats_temp['Marks against'] = pd.to_numeric(Por_stats_temp['Marks against'])
Por_stats_temp['Handballs for'] = pd.to_numeric(Por_stats_temp['Handballs for'])
Por_stats_temp['Handballs against'] = pd.to_numeric(Por_stats_temp['Handballs against'])
Por_stats_temp['Disposals for'] = pd.to_numeric(Por_stats_temp['Disposals for'])
Por_stats_temp['Disposals against'] = pd.to_numeric(Por_stats_temp['Disposals against'])
Por_stats_temp['Hitouts for'] = pd.to_numeric(Por_stats_temp['Hitouts for'])
Por_stats_temp['Hitouts against'] = pd.to_numeric(Por_stats_temp['Hitouts against'])
Por_stats_temp['Tackles for'] = pd.to_numeric(Por_stats_temp['Tackles for'])
Por_stats_temp['Tackles against'] = pd.to_numeric(Por_stats_temp['Tackles against'])
Por_stats_temp['Rebound 50s for'] = pd.to_numeric(Por_stats_temp['Rebound 50s for'])
Por_stats_temp['Rebound 50s against'] = pd.to_numeric(Por_stats_temp['Rebound 50s against'])
Por_stats_temp['Inside 50s for'] = pd.to_numeric(Por_stats_temp['Inside 50s for'])
Por_stats_temp['Inside 50s against'] = pd.to_numeric(Por_stats_temp['Inside 50s against'])
Por_stats_temp['Clearances for'] = pd.to_numeric(Por_stats_temp['Clearances for'])
Por_stats_temp['Clearances against'] = pd.to_numeric(Por_stats_temp['Clearances against'])
Por_stats_temp['Clangers for'] = pd.to_numeric(Por_stats_temp['Clangers for'])
Por_stats_temp['Clangers against'] = pd.to_numeric(Por_stats_temp['Clearances against'])
Por_stats_temp['Frees for'] = pd.to_numeric(Por_stats_temp['Frees for'])
Por_stats_temp['Frees against'] = pd.to_numeric(Por_stats_temp['Frees against'])
Por_stats_temp['Contested possessions for'] = pd.to_numeric(Por_stats_temp['Contested possessions for'])
Por_stats_temp['Contested possessions against'] = pd.to_numeric(Por_stats_temp['Contested possessions against'])
Por_stats_temp['Uncontested possessions for'] = pd.to_numeric(Por_stats_temp['Uncontested possessions for'])
Por_stats_temp['Uncontested possessions against'] = pd.to_numeric(Por_stats_temp['Uncontested possessions against'])
Por_stats_temp['Contested marks for'] = pd.to_numeric(Por_stats_temp['Contested marks for'])
Por_stats_temp['Contested marks against'] = pd.to_numeric(Por_stats_temp['Contested marks against'])
Por_stats_temp['Marks inside 50 for'] = pd.to_numeric(Por_stats_temp['Marks inside 50 for'])
Por_stats_temp['Marks inside 50 against'] = pd.to_numeric(Por_stats_temp['Marks inside 50 against'])
Por_stats_temp['1%s for'] = pd.to_numeric(Por_stats_temp['1%s for'])
Por_stats_temp['1%s against'] = pd.to_numeric(Por_stats_temp['1%s against'])
Por_stats_temp['Bounces for'] = pd.to_numeric(Por_stats_temp['Bounces for'])
Por_stats_temp['Bounces against'] = pd.to_numeric(Por_stats_temp['Bounces against'])
Por_stats_temp['Goal assists for'] = pd.to_numeric(Por_stats_temp['Goal assists for'])
Por_stats_temp['Goal assists against'] = pd.to_numeric(Por_stats_temp['Goal assists against'])

Por = pd.concat([Por_class, Por_stats_temp], axis=1)
Por.columns = ['Header', '#','Game ID','Home', 'Kicks for', 'Kicks against',
       'Marks for', 'Marks against', 'Handballs for', 'Handballs against',
       'Disposals for', 'Disposals against', 'Hitouts for', 'Hitouts against',
       'Tackles for', 'Tackles against', 'Rebound 50s for',
       'Rebound 50s against', 'Inside 50s for', 'Inside 50s against',
       'Clearances for', 'Clearances against', 'Clangers for',
       'Clangers against', 'Frees for', 'Frees against',
       'Contested possessions for', 'Contested possessions against',
       'Uncontested possessions for', 'Uncontested possessions against',
       'Contested marks for', 'Contested marks against', 'Marks inside 50 for',
       'Marks inside 50 against', '1%s for', '1%s against', 'Bounces for',
       'Bounces against', 'Goal assists for', 'Goal assists against',
       'Margin']

In [20]:
Ric_stats_temp[['Kicks for','Kicks against']] = Ric_stats['KI'].str.split('-',expand=True)
Ric_stats_temp[['Marks for','Marks against']] = Ric_stats['MK'].str.split('-',expand=True)
Ric_stats_temp[['Handballs for','Handballs against']] = Ric_stats['HB'].str.split('-',expand=True)
Ric_stats_temp[['Disposals for','Disposals against']] = Ric_stats['DI'].str.split('-',expand=True)
Ric_stats_temp[['Hitouts for','Hitouts against']] = Ric_stats['HO'].str.split('-',expand=True)
Ric_stats_temp[['Tackles for','Tackles against']] = Ric_stats['TK'].str.split('-',expand=True)
Ric_stats_temp[['Rebound 50s for','Rebound 50s against']] = Ric_stats['RB'].str.split('-',expand=True)
Ric_stats_temp[['Inside 50s for','Inside 50s against']] = Ric_stats['IF'].str.split('-',expand=True)
Ric_stats_temp[['Clearances for','Clearances against']] = Ric_stats['CL'].str.split('-',expand=True)
Ric_stats_temp[['Clangers for','Clangers against']] = Ric_stats['CG'].str.split('-',expand=True)
Ric_stats_temp[['Frees for','Frees against']] = Ric_stats['FF'].str.split('-',expand=True)
Ric_stats_temp[['Contested possessions for','Contested possessions against']] = Ric_stats['CP'].str.split('-',expand=True)
Ric_stats_temp[['Uncontested possessions for','Uncontested possessions against']] = Ric_stats['UP'].str.split('-',expand=True)
Ric_stats_temp[['Contested marks for','Contested marks against']] = Ric_stats['CM'].str.split('-',expand=True)
Ric_stats_temp[['Marks inside 50 for','Marks inside 50 against']] = Ric_stats['MI'].str.split('-',expand=True)
Ric_stats_temp[['1%s for','1%s against']] = Ric_stats['1%'].str.split('-',expand=True)
Ric_stats_temp[['Bounces for','Bounces against']] = Ric_stats['BO'].str.split('-',expand=True)
Ric_stats_temp[['Goal assists for','Goal assists against']] = Ric_stats['GA'].str.split('-',expand=True)
Ric_stats_temp['Margin'] = Ric_stats['Margin']

Ric_stats_temp['Kicks for'] = pd.to_numeric(Ric_stats_temp['Kicks for'])
Ric_stats_temp['Kicks against'] = pd.to_numeric(Ric_stats_temp['Kicks against'])
Ric_stats_temp['Marks for'] = pd.to_numeric(Ric_stats_temp['Marks for'])
Ric_stats_temp['Marks against'] = pd.to_numeric(Ric_stats_temp['Marks against'])
Ric_stats_temp['Handballs for'] = pd.to_numeric(Ric_stats_temp['Handballs for'])
Ric_stats_temp['Handballs against'] = pd.to_numeric(Ric_stats_temp['Handballs against'])
Ric_stats_temp['Disposals for'] = pd.to_numeric(Ric_stats_temp['Disposals for'])
Ric_stats_temp['Disposals against'] = pd.to_numeric(Ric_stats_temp['Disposals against'])
Ric_stats_temp['Hitouts for'] = pd.to_numeric(Ric_stats_temp['Hitouts for'])
Ric_stats_temp['Hitouts against'] = pd.to_numeric(Ric_stats_temp['Hitouts against'])
Ric_stats_temp['Tackles for'] = pd.to_numeric(Ric_stats_temp['Tackles for'])
Ric_stats_temp['Tackles against'] = pd.to_numeric(Ric_stats_temp['Tackles against'])
Ric_stats_temp['Rebound 50s for'] = pd.to_numeric(Ric_stats_temp['Rebound 50s for'])
Ric_stats_temp['Rebound 50s against'] = pd.to_numeric(Ric_stats_temp['Rebound 50s against'])
Ric_stats_temp['Inside 50s for'] = pd.to_numeric(Ric_stats_temp['Inside 50s for'])
Ric_stats_temp['Inside 50s against'] = pd.to_numeric(Ric_stats_temp['Inside 50s against'])
Ric_stats_temp['Clearances for'] = pd.to_numeric(Ric_stats_temp['Clearances for'])
Ric_stats_temp['Clearances against'] = pd.to_numeric(Ric_stats_temp['Clearances against'])
Ric_stats_temp['Clangers for'] = pd.to_numeric(Ric_stats_temp['Clangers for'])
Ric_stats_temp['Clangers against'] = pd.to_numeric(Ric_stats_temp['Clearances against'])
Ric_stats_temp['Frees for'] = pd.to_numeric(Ric_stats_temp['Frees for'])
Ric_stats_temp['Frees against'] = pd.to_numeric(Ric_stats_temp['Frees against'])
Ric_stats_temp['Contested possessions for'] = pd.to_numeric(Ric_stats_temp['Contested possessions for'])
Ric_stats_temp['Contested possessions against'] = pd.to_numeric(Ric_stats_temp['Contested possessions against'])
Ric_stats_temp['Uncontested possessions for'] = pd.to_numeric(Ric_stats_temp['Uncontested possessions for'])
Ric_stats_temp['Uncontested possessions against'] = pd.to_numeric(Ric_stats_temp['Uncontested possessions against'])
Ric_stats_temp['Contested marks for'] = pd.to_numeric(Ric_stats_temp['Contested marks for'])
Ric_stats_temp['Contested marks against'] = pd.to_numeric(Ric_stats_temp['Contested marks against'])
Ric_stats_temp['Marks inside 50 for'] = pd.to_numeric(Ric_stats_temp['Marks inside 50 for'])
Ric_stats_temp['Marks inside 50 against'] = pd.to_numeric(Ric_stats_temp['Marks inside 50 against'])
Ric_stats_temp['1%s for'] = pd.to_numeric(Ric_stats_temp['1%s for'])
Ric_stats_temp['1%s against'] = pd.to_numeric(Ric_stats_temp['1%s against'])
Ric_stats_temp['Bounces for'] = pd.to_numeric(Ric_stats_temp['Bounces for'])
Ric_stats_temp['Bounces against'] = pd.to_numeric(Ric_stats_temp['Bounces against'])
Ric_stats_temp['Goal assists for'] = pd.to_numeric(Ric_stats_temp['Goal assists for'])
Ric_stats_temp['Goal assists against'] = pd.to_numeric(Ric_stats_temp['Goal assists against'])

Ric = pd.concat([Ric_class, Ric_stats_temp], axis=1)
Ric.columns = ['Header', '#','Game ID','Home', 'Kicks for', 'Kicks against',
       'Marks for', 'Marks against', 'Handballs for', 'Handballs against',
       'Disposals for', 'Disposals against', 'Hitouts for', 'Hitouts against',
       'Tackles for', 'Tackles against', 'Rebound 50s for',
       'Rebound 50s against', 'Inside 50s for', 'Inside 50s against',
       'Clearances for', 'Clearances against', 'Clangers for',
       'Clangers against', 'Frees for', 'Frees against',
       'Contested possessions for', 'Contested possessions against',
       'Uncontested possessions for', 'Uncontested possessions against',
       'Contested marks for', 'Contested marks against', 'Marks inside 50 for',
       'Marks inside 50 against', '1%s for', '1%s against', 'Bounces for',
       'Bounces against', 'Goal assists for', 'Goal assists against',
       'Margin']

In [21]:
StK_stats_temp[['Kicks for','Kicks against']] = StK_stats['KI'].str.split('-',expand=True)
StK_stats_temp[['Marks for','Marks against']] = StK_stats['MK'].str.split('-',expand=True)
StK_stats_temp[['Handballs for','Handballs against']] = StK_stats['HB'].str.split('-',expand=True)
StK_stats_temp[['Disposals for','Disposals against']] = StK_stats['DI'].str.split('-',expand=True)
StK_stats_temp[['Hitouts for','Hitouts against']] = StK_stats['HO'].str.split('-',expand=True)
StK_stats_temp[['Tackles for','Tackles against']] = StK_stats['TK'].str.split('-',expand=True)
StK_stats_temp[['Rebound 50s for','Rebound 50s against']] = StK_stats['RB'].str.split('-',expand=True)
StK_stats_temp[['Inside 50s for','Inside 50s against']] = StK_stats['IF'].str.split('-',expand=True)
StK_stats_temp[['Clearances for','Clearances against']] = StK_stats['CL'].str.split('-',expand=True)
StK_stats_temp[['Clangers for','Clangers against']] = StK_stats['CG'].str.split('-',expand=True)
StK_stats_temp[['Frees for','Frees against']] = StK_stats['FF'].str.split('-',expand=True)
StK_stats_temp[['Contested possessions for','Contested possessions against']] = StK_stats['CP'].str.split('-',expand=True)
StK_stats_temp[['Uncontested possessions for','Uncontested possessions against']] = StK_stats['UP'].str.split('-',expand=True)
StK_stats_temp[['Contested marks for','Contested marks against']] = StK_stats['CM'].str.split('-',expand=True)
StK_stats_temp[['Marks inside 50 for','Marks inside 50 against']] = StK_stats['MI'].str.split('-',expand=True)
StK_stats_temp[['1%s for','1%s against']] = StK_stats['1%'].str.split('-',expand=True)
StK_stats_temp[['Bounces for','Bounces against']] = StK_stats['BO'].str.split('-',expand=True)
StK_stats_temp[['Goal assists for','Goal assists against']] = StK_stats['GA'].str.split('-',expand=True)
StK_stats_temp['Margin'] = StK_stats['Margin']

StK_stats_temp['Kicks for'] = pd.to_numeric(StK_stats_temp['Kicks for'])
StK_stats_temp['Kicks against'] = pd.to_numeric(StK_stats_temp['Kicks against'])
StK_stats_temp['Marks for'] = pd.to_numeric(StK_stats_temp['Marks for'])
StK_stats_temp['Marks against'] = pd.to_numeric(StK_stats_temp['Marks against'])
StK_stats_temp['Handballs for'] = pd.to_numeric(StK_stats_temp['Handballs for'])
StK_stats_temp['Handballs against'] = pd.to_numeric(StK_stats_temp['Handballs against'])
StK_stats_temp['Disposals for'] = pd.to_numeric(StK_stats_temp['Disposals for'])
StK_stats_temp['Disposals against'] = pd.to_numeric(StK_stats_temp['Disposals against'])
StK_stats_temp['Hitouts for'] = pd.to_numeric(StK_stats_temp['Hitouts for'])
StK_stats_temp['Hitouts against'] = pd.to_numeric(StK_stats_temp['Hitouts against'])
StK_stats_temp['Tackles for'] = pd.to_numeric(StK_stats_temp['Tackles for'])
StK_stats_temp['Tackles against'] = pd.to_numeric(StK_stats_temp['Tackles against'])
StK_stats_temp['Rebound 50s for'] = pd.to_numeric(StK_stats_temp['Rebound 50s for'])
StK_stats_temp['Rebound 50s against'] = pd.to_numeric(StK_stats_temp['Rebound 50s against'])
StK_stats_temp['Inside 50s for'] = pd.to_numeric(StK_stats_temp['Inside 50s for'])
StK_stats_temp['Inside 50s against'] = pd.to_numeric(StK_stats_temp['Inside 50s against'])
StK_stats_temp['Clearances for'] = pd.to_numeric(StK_stats_temp['Clearances for'])
StK_stats_temp['Clearances against'] = pd.to_numeric(StK_stats_temp['Clearances against'])
StK_stats_temp['Clangers for'] = pd.to_numeric(StK_stats_temp['Clangers for'])
StK_stats_temp['Clangers against'] = pd.to_numeric(StK_stats_temp['Clearances against'])
StK_stats_temp['Frees for'] = pd.to_numeric(StK_stats_temp['Frees for'])
StK_stats_temp['Frees against'] = pd.to_numeric(StK_stats_temp['Frees against'])
StK_stats_temp['Contested possessions for'] = pd.to_numeric(StK_stats_temp['Contested possessions for'])
StK_stats_temp['Contested possessions against'] = pd.to_numeric(StK_stats_temp['Contested possessions against'])
StK_stats_temp['Uncontested possessions for'] = pd.to_numeric(StK_stats_temp['Uncontested possessions for'])
StK_stats_temp['Uncontested possessions against'] = pd.to_numeric(StK_stats_temp['Uncontested possessions against'])
StK_stats_temp['Contested marks for'] = pd.to_numeric(StK_stats_temp['Contested marks for'])
StK_stats_temp['Contested marks against'] = pd.to_numeric(StK_stats_temp['Contested marks against'])
StK_stats_temp['Marks inside 50 for'] = pd.to_numeric(StK_stats_temp['Marks inside 50 for'])
StK_stats_temp['Marks inside 50 against'] = pd.to_numeric(StK_stats_temp['Marks inside 50 against'])
StK_stats_temp['1%s for'] = pd.to_numeric(StK_stats_temp['1%s for'])
StK_stats_temp['1%s against'] = pd.to_numeric(StK_stats_temp['1%s against'])
StK_stats_temp['Bounces for'] = pd.to_numeric(StK_stats_temp['Bounces for'])
StK_stats_temp['Bounces against'] = pd.to_numeric(StK_stats_temp['Bounces against'])
StK_stats_temp['Goal assists for'] = pd.to_numeric(StK_stats_temp['Goal assists for'])
StK_stats_temp['Goal assists against'] = pd.to_numeric(StK_stats_temp['Goal assists against'])

StK = pd.concat([StK_class, StK_stats_temp], axis=1)
StK.columns = ['Header', '#','Game ID','Home', 'Kicks for', 'Kicks against',
       'Marks for', 'Marks against', 'Handballs for', 'Handballs against',
       'Disposals for', 'Disposals against', 'Hitouts for', 'Hitouts against',
       'Tackles for', 'Tackles against', 'Rebound 50s for',
       'Rebound 50s against', 'Inside 50s for', 'Inside 50s against',
       'Clearances for', 'Clearances against', 'Clangers for',
       'Clangers against', 'Frees for', 'Frees against',
       'Contested possessions for', 'Contested possessions against',
       'Uncontested possessions for', 'Uncontested possessions against',
       'Contested marks for', 'Contested marks against', 'Marks inside 50 for',
       'Marks inside 50 against', '1%s for', '1%s against', 'Bounces for',
       'Bounces against', 'Goal assists for', 'Goal assists against',
       'Margin']

In [22]:
Syd_stats_temp[['Kicks for','Kicks against']] = Syd_stats['KI'].str.split('-',expand=True)
Syd_stats_temp[['Marks for','Marks against']] = Syd_stats['MK'].str.split('-',expand=True)
Syd_stats_temp[['Handballs for','Handballs against']] = Syd_stats['HB'].str.split('-',expand=True)
Syd_stats_temp[['Disposals for','Disposals against']] = Syd_stats['DI'].str.split('-',expand=True)
Syd_stats_temp[['Hitouts for','Hitouts against']] = Syd_stats['HO'].str.split('-',expand=True)
Syd_stats_temp[['Tackles for','Tackles against']] = Syd_stats['TK'].str.split('-',expand=True)
Syd_stats_temp[['Rebound 50s for','Rebound 50s against']] = Syd_stats['RB'].str.split('-',expand=True)
Syd_stats_temp[['Inside 50s for','Inside 50s against']] = Syd_stats['IF'].str.split('-',expand=True)
Syd_stats_temp[['Clearances for','Clearances against']] = Syd_stats['CL'].str.split('-',expand=True)
Syd_stats_temp[['Clangers for','Clangers against']] = Syd_stats['CG'].str.split('-',expand=True)
Syd_stats_temp[['Frees for','Frees against']] = Syd_stats['FF'].str.split('-',expand=True)
Syd_stats_temp[['Contested possessions for','Contested possessions against']] = Syd_stats['CP'].str.split('-',expand=True)
Syd_stats_temp[['Uncontested possessions for','Uncontested possessions against']] = Syd_stats['UP'].str.split('-',expand=True)
Syd_stats_temp[['Contested marks for','Contested marks against']] = Syd_stats['CM'].str.split('-',expand=True)
Syd_stats_temp[['Marks inside 50 for','Marks inside 50 against']] = Syd_stats['MI'].str.split('-',expand=True)
Syd_stats_temp[['1%s for','1%s against']] = Syd_stats['1%'].str.split('-',expand=True)
Syd_stats_temp[['Bounces for','Bounces against']] = Syd_stats['BO'].str.split('-',expand=True)
Syd_stats_temp[['Goal assists for','Goal assists against']] = Syd_stats['GA'].str.split('-',expand=True)
Syd_stats_temp['Margin'] = Syd_stats['Margin']

Syd_stats_temp['Kicks for'] = pd.to_numeric(Syd_stats_temp['Kicks for'])
Syd_stats_temp['Kicks against'] = pd.to_numeric(Syd_stats_temp['Kicks against'])
Syd_stats_temp['Marks for'] = pd.to_numeric(Syd_stats_temp['Marks for'])
Syd_stats_temp['Marks against'] = pd.to_numeric(Syd_stats_temp['Marks against'])
Syd_stats_temp['Handballs for'] = pd.to_numeric(Syd_stats_temp['Handballs for'])
Syd_stats_temp['Handballs against'] = pd.to_numeric(Syd_stats_temp['Handballs against'])
Syd_stats_temp['Disposals for'] = pd.to_numeric(Syd_stats_temp['Disposals for'])
Syd_stats_temp['Disposals against'] = pd.to_numeric(Syd_stats_temp['Disposals against'])
Syd_stats_temp['Hitouts for'] = pd.to_numeric(Syd_stats_temp['Hitouts for'])
Syd_stats_temp['Hitouts against'] = pd.to_numeric(Syd_stats_temp['Hitouts against'])
Syd_stats_temp['Tackles for'] = pd.to_numeric(Syd_stats_temp['Tackles for'])
Syd_stats_temp['Tackles against'] = pd.to_numeric(Syd_stats_temp['Tackles against'])
Syd_stats_temp['Rebound 50s for'] = pd.to_numeric(Syd_stats_temp['Rebound 50s for'])
Syd_stats_temp['Rebound 50s against'] = pd.to_numeric(Syd_stats_temp['Rebound 50s against'])
Syd_stats_temp['Inside 50s for'] = pd.to_numeric(Syd_stats_temp['Inside 50s for'])
Syd_stats_temp['Inside 50s against'] = pd.to_numeric(Syd_stats_temp['Inside 50s against'])
Syd_stats_temp['Clearances for'] = pd.to_numeric(Syd_stats_temp['Clearances for'])
Syd_stats_temp['Clearances against'] = pd.to_numeric(Syd_stats_temp['Clearances against'])
Syd_stats_temp['Clangers for'] = pd.to_numeric(Syd_stats_temp['Clangers for'])
Syd_stats_temp['Clangers against'] = pd.to_numeric(Syd_stats_temp['Clearances against'])
Syd_stats_temp['Frees for'] = pd.to_numeric(Syd_stats_temp['Frees for'])
Syd_stats_temp['Frees against'] = pd.to_numeric(Syd_stats_temp['Frees against'])
Syd_stats_temp['Contested possessions for'] = pd.to_numeric(Syd_stats_temp['Contested possessions for'])
Syd_stats_temp['Contested possessions against'] = pd.to_numeric(Syd_stats_temp['Contested possessions against'])
Syd_stats_temp['Uncontested possessions for'] = pd.to_numeric(Syd_stats_temp['Uncontested possessions for'])
Syd_stats_temp['Uncontested possessions against'] = pd.to_numeric(Syd_stats_temp['Uncontested possessions against'])
Syd_stats_temp['Contested marks for'] = pd.to_numeric(Syd_stats_temp['Contested marks for'])
Syd_stats_temp['Contested marks against'] = pd.to_numeric(Syd_stats_temp['Contested marks against'])
Syd_stats_temp['Marks inside 50 for'] = pd.to_numeric(Syd_stats_temp['Marks inside 50 for'])
Syd_stats_temp['Marks inside 50 against'] = pd.to_numeric(Syd_stats_temp['Marks inside 50 against'])
Syd_stats_temp['1%s for'] = pd.to_numeric(Syd_stats_temp['1%s for'])
Syd_stats_temp['1%s against'] = pd.to_numeric(Syd_stats_temp['1%s against'])
Syd_stats_temp['Bounces for'] = pd.to_numeric(Syd_stats_temp['Bounces for'])
Syd_stats_temp['Bounces against'] = pd.to_numeric(Syd_stats_temp['Bounces against'])
Syd_stats_temp['Goal assists for'] = pd.to_numeric(Syd_stats_temp['Goal assists for'])
Syd_stats_temp['Goal assists against'] = pd.to_numeric(Syd_stats_temp['Goal assists against'])

Syd = pd.concat([Syd_class, Syd_stats_temp], axis=1)
Syd.columns = ['Header', '#','Game ID','Home', 'Kicks for', 'Kicks against',
       'Marks for', 'Marks against', 'Handballs for', 'Handballs against',
       'Disposals for', 'Disposals against', 'Hitouts for', 'Hitouts against',
       'Tackles for', 'Tackles against', 'Rebound 50s for',
       'Rebound 50s against', 'Inside 50s for', 'Inside 50s against',
       'Clearances for', 'Clearances against', 'Clangers for',
       'Clangers against', 'Frees for', 'Frees against',
       'Contested possessions for', 'Contested possessions against',
       'Uncontested possessions for', 'Uncontested possessions against',
       'Contested marks for', 'Contested marks against', 'Marks inside 50 for',
       'Marks inside 50 against', '1%s for', '1%s against', 'Bounces for',
       'Bounces against', 'Goal assists for', 'Goal assists against',
       'Margin']

In [23]:
WCE_stats_temp[['Kicks for','Kicks against']] = WCE_stats['KI'].str.split('-',expand=True)
WCE_stats_temp[['Marks for','Marks against']] = WCE_stats['MK'].str.split('-',expand=True)
WCE_stats_temp[['Handballs for','Handballs against']] = WCE_stats['HB'].str.split('-',expand=True)
WCE_stats_temp[['Disposals for','Disposals against']] = WCE_stats['DI'].str.split('-',expand=True)
WCE_stats_temp[['Hitouts for','Hitouts against']] = WCE_stats['HO'].str.split('-',expand=True)
WCE_stats_temp[['Tackles for','Tackles against']] = WCE_stats['TK'].str.split('-',expand=True)
WCE_stats_temp[['Rebound 50s for','Rebound 50s against']] = WCE_stats['RB'].str.split('-',expand=True)
WCE_stats_temp[['Inside 50s for','Inside 50s against']] = WCE_stats['IF'].str.split('-',expand=True)
WCE_stats_temp[['Clearances for','Clearances against']] = WCE_stats['CL'].str.split('-',expand=True)
WCE_stats_temp[['Clangers for','Clangers against']] = WCE_stats['CG'].str.split('-',expand=True)
WCE_stats_temp[['Frees for','Frees against']] = WCE_stats['FF'].str.split('-',expand=True)
WCE_stats_temp[['Contested possessions for','Contested possessions against']] = WCE_stats['CP'].str.split('-',expand=True)
WCE_stats_temp[['Uncontested possessions for','Uncontested possessions against']] = WCE_stats['UP'].str.split('-',expand=True)
WCE_stats_temp[['Contested marks for','Contested marks against']] = WCE_stats['CM'].str.split('-',expand=True)
WCE_stats_temp[['Marks inside 50 for','Marks inside 50 against']] = WCE_stats['MI'].str.split('-',expand=True)
WCE_stats_temp[['1%s for','1%s against']] = WCE_stats['1%'].str.split('-',expand=True)
WCE_stats_temp[['Bounces for','Bounces against']] = WCE_stats['BO'].str.split('-',expand=True)
WCE_stats_temp[['Goal assists for','Goal assists against']] = WCE_stats['GA'].str.split('-',expand=True)
WCE_stats_temp['Margin'] = WCE_stats['Margin']

WCE_stats_temp['Kicks for'] = pd.to_numeric(WCE_stats_temp['Kicks for'])
WCE_stats_temp['Kicks against'] = pd.to_numeric(WCE_stats_temp['Kicks against'])
WCE_stats_temp['Marks for'] = pd.to_numeric(WCE_stats_temp['Marks for'])
WCE_stats_temp['Marks against'] = pd.to_numeric(WCE_stats_temp['Marks against'])
WCE_stats_temp['Handballs for'] = pd.to_numeric(WCE_stats_temp['Handballs for'])
WCE_stats_temp['Handballs against'] = pd.to_numeric(WCE_stats_temp['Handballs against'])
WCE_stats_temp['Disposals for'] = pd.to_numeric(WCE_stats_temp['Disposals for'])
WCE_stats_temp['Disposals against'] = pd.to_numeric(WCE_stats_temp['Disposals against'])
WCE_stats_temp['Hitouts for'] = pd.to_numeric(WCE_stats_temp['Hitouts for'])
WCE_stats_temp['Hitouts against'] = pd.to_numeric(WCE_stats_temp['Hitouts against'])
WCE_stats_temp['Tackles for'] = pd.to_numeric(WCE_stats_temp['Tackles for'])
WCE_stats_temp['Tackles against'] = pd.to_numeric(WCE_stats_temp['Tackles against'])
WCE_stats_temp['Rebound 50s for'] = pd.to_numeric(WCE_stats_temp['Rebound 50s for'])
WCE_stats_temp['Rebound 50s against'] = pd.to_numeric(WCE_stats_temp['Rebound 50s against'])
WCE_stats_temp['Inside 50s for'] = pd.to_numeric(WCE_stats_temp['Inside 50s for'])
WCE_stats_temp['Inside 50s against'] = pd.to_numeric(WCE_stats_temp['Inside 50s against'])
WCE_stats_temp['Clearances for'] = pd.to_numeric(WCE_stats_temp['Clearances for'])
WCE_stats_temp['Clearances against'] = pd.to_numeric(WCE_stats_temp['Clearances against'])
WCE_stats_temp['Clangers for'] = pd.to_numeric(WCE_stats_temp['Clangers for'])
WCE_stats_temp['Clangers against'] = pd.to_numeric(WCE_stats_temp['Clearances against'])
WCE_stats_temp['Frees for'] = pd.to_numeric(WCE_stats_temp['Frees for'])
WCE_stats_temp['Frees against'] = pd.to_numeric(WCE_stats_temp['Frees against'])
WCE_stats_temp['Contested possessions for'] = pd.to_numeric(WCE_stats_temp['Contested possessions for'])
WCE_stats_temp['Contested possessions against'] = pd.to_numeric(WCE_stats_temp['Contested possessions against'])
WCE_stats_temp['Uncontested possessions for'] = pd.to_numeric(WCE_stats_temp['Uncontested possessions for'])
WCE_stats_temp['Uncontested possessions against'] = pd.to_numeric(WCE_stats_temp['Uncontested possessions against'])
WCE_stats_temp['Contested marks for'] = pd.to_numeric(WCE_stats_temp['Contested marks for'])
WCE_stats_temp['Contested marks against'] = pd.to_numeric(WCE_stats_temp['Contested marks against'])
WCE_stats_temp['Marks inside 50 for'] = pd.to_numeric(WCE_stats_temp['Marks inside 50 for'])
WCE_stats_temp['Marks inside 50 against'] = pd.to_numeric(WCE_stats_temp['Marks inside 50 against'])
WCE_stats_temp['1%s for'] = pd.to_numeric(WCE_stats_temp['1%s for'])
WCE_stats_temp['1%s against'] = pd.to_numeric(WCE_stats_temp['1%s against'])
WCE_stats_temp['Bounces for'] = pd.to_numeric(WCE_stats_temp['Bounces for'])
WCE_stats_temp['Bounces against'] = pd.to_numeric(WCE_stats_temp['Bounces against'])
WCE_stats_temp['Goal assists for'] = pd.to_numeric(WCE_stats_temp['Goal assists for'])
WCE_stats_temp['Goal assists against'] = pd.to_numeric(WCE_stats_temp['Goal assists against'])

WCE = pd.concat([WCE_class, WCE_stats_temp], axis=1)
WCE.columns = ['Header', '#','Game ID','Home', 'Kicks for', 'Kicks against',
       'Marks for', 'Marks against', 'Handballs for', 'Handballs against',
       'Disposals for', 'Disposals against', 'Hitouts for', 'Hitouts against',
       'Tackles for', 'Tackles against', 'Rebound 50s for',
       'Rebound 50s against', 'Inside 50s for', 'Inside 50s against',
       'Clearances for', 'Clearances against', 'Clangers for',
       'Clangers against', 'Frees for', 'Frees against',
       'Contested possessions for', 'Contested possessions against',
       'Uncontested possessions for', 'Uncontested possessions against',
       'Contested marks for', 'Contested marks against', 'Marks inside 50 for',
       'Marks inside 50 against', '1%s for', '1%s against', 'Bounces for',
       'Bounces against', 'Goal assists for', 'Goal assists against',
       'Margin']

In [24]:
Wes_stats_temp[['Kicks for','Kicks against']] = Wes_stats['KI'].str.split('-',expand=True)
Wes_stats_temp[['Marks for','Marks against']] = Wes_stats['MK'].str.split('-',expand=True)
Wes_stats_temp[['Handballs for','Handballs against']] = Wes_stats['HB'].str.split('-',expand=True)
Wes_stats_temp[['Disposals for','Disposals against']] = Wes_stats['DI'].str.split('-',expand=True)
Wes_stats_temp[['Hitouts for','Hitouts against']] = Wes_stats['HO'].str.split('-',expand=True)
Wes_stats_temp[['Tackles for','Tackles against']] = Wes_stats['TK'].str.split('-',expand=True)
Wes_stats_temp[['Rebound 50s for','Rebound 50s against']] = Wes_stats['RB'].str.split('-',expand=True)
Wes_stats_temp[['Inside 50s for','Inside 50s against']] = Wes_stats['IF'].str.split('-',expand=True)
Wes_stats_temp[['Clearances for','Clearances against']] = Wes_stats['CL'].str.split('-',expand=True)
Wes_stats_temp[['Clangers for','Clangers against']] = Wes_stats['CG'].str.split('-',expand=True)
Wes_stats_temp[['Frees for','Frees against']] = Wes_stats['FF'].str.split('-',expand=True)
Wes_stats_temp[['Contested possessions for','Contested possessions against']] = Wes_stats['CP'].str.split('-',expand=True)
Wes_stats_temp[['Uncontested possessions for','Uncontested possessions against']] = Wes_stats['UP'].str.split('-',expand=True)
Wes_stats_temp[['Contested marks for','Contested marks against']] = Wes_stats['CM'].str.split('-',expand=True)
Wes_stats_temp[['Marks inside 50 for','Marks inside 50 against']] = Wes_stats['MI'].str.split('-',expand=True)
Wes_stats_temp[['1%s for','1%s against']] = Wes_stats['1%'].str.split('-',expand=True)
Wes_stats_temp[['Bounces for','Bounces against']] = Wes_stats['BO'].str.split('-',expand=True)
Wes_stats_temp[['Goal assists for','Goal assists against']] = Wes_stats['GA'].str.split('-',expand=True)
Wes_stats_temp['Margin'] = Wes_stats['Margin']

Wes_stats_temp['Kicks for'] = pd.to_numeric(Wes_stats_temp['Kicks for'])
Wes_stats_temp['Kicks against'] = pd.to_numeric(Wes_stats_temp['Kicks against'])
Wes_stats_temp['Marks for'] = pd.to_numeric(Wes_stats_temp['Marks for'])
Wes_stats_temp['Marks against'] = pd.to_numeric(Wes_stats_temp['Marks against'])
Wes_stats_temp['Handballs for'] = pd.to_numeric(Wes_stats_temp['Handballs for'])
Wes_stats_temp['Handballs against'] = pd.to_numeric(Wes_stats_temp['Handballs against'])
Wes_stats_temp['Disposals for'] = pd.to_numeric(Wes_stats_temp['Disposals for'])
Wes_stats_temp['Disposals against'] = pd.to_numeric(Wes_stats_temp['Disposals against'])
Wes_stats_temp['Hitouts for'] = pd.to_numeric(Wes_stats_temp['Hitouts for'])
Wes_stats_temp['Hitouts against'] = pd.to_numeric(Wes_stats_temp['Hitouts against'])
Wes_stats_temp['Tackles for'] = pd.to_numeric(Wes_stats_temp['Tackles for'])
Wes_stats_temp['Tackles against'] = pd.to_numeric(Wes_stats_temp['Tackles against'])
Wes_stats_temp['Rebound 50s for'] = pd.to_numeric(Wes_stats_temp['Rebound 50s for'])
Wes_stats_temp['Rebound 50s against'] = pd.to_numeric(Wes_stats_temp['Rebound 50s against'])
Wes_stats_temp['Inside 50s for'] = pd.to_numeric(Wes_stats_temp['Inside 50s for'])
Wes_stats_temp['Inside 50s against'] = pd.to_numeric(Wes_stats_temp['Inside 50s against'])
Wes_stats_temp['Clearances for'] = pd.to_numeric(Wes_stats_temp['Clearances for'])
Wes_stats_temp['Clearances against'] = pd.to_numeric(Wes_stats_temp['Clearances against'])
Wes_stats_temp['Clangers for'] = pd.to_numeric(Wes_stats_temp['Clangers for'])
Wes_stats_temp['Clangers against'] = pd.to_numeric(Wes_stats_temp['Clearances against'])
Wes_stats_temp['Frees for'] = pd.to_numeric(Wes_stats_temp['Frees for'])
Wes_stats_temp['Frees against'] = pd.to_numeric(Wes_stats_temp['Frees against'])
Wes_stats_temp['Contested possessions for'] = pd.to_numeric(Wes_stats_temp['Contested possessions for'])
Wes_stats_temp['Contested possessions against'] = pd.to_numeric(Wes_stats_temp['Contested possessions against'])
Wes_stats_temp['Uncontested possessions for'] = pd.to_numeric(Wes_stats_temp['Uncontested possessions for'])
Wes_stats_temp['Uncontested possessions against'] = pd.to_numeric(Wes_stats_temp['Uncontested possessions against'])
Wes_stats_temp['Contested marks for'] = pd.to_numeric(Wes_stats_temp['Contested marks for'])
Wes_stats_temp['Contested marks against'] = pd.to_numeric(Wes_stats_temp['Contested marks against'])
Wes_stats_temp['Marks inside 50 for'] = pd.to_numeric(Wes_stats_temp['Marks inside 50 for'])
Wes_stats_temp['Marks inside 50 against'] = pd.to_numeric(Wes_stats_temp['Marks inside 50 against'])
Wes_stats_temp['1%s for'] = pd.to_numeric(Wes_stats_temp['1%s for'])
Wes_stats_temp['1%s against'] = pd.to_numeric(Wes_stats_temp['1%s against'])
Wes_stats_temp['Bounces for'] = pd.to_numeric(Wes_stats_temp['Bounces for'])
Wes_stats_temp['Bounces against'] = pd.to_numeric(Wes_stats_temp['Bounces against'])
Wes_stats_temp['Goal assists for'] = pd.to_numeric(Wes_stats_temp['Goal assists for'])
Wes_stats_temp['Goal assists against'] = pd.to_numeric(Wes_stats_temp['Goal assists against'])

Wes = pd.concat([Wes_class, Wes_stats_temp], axis=1)
Wes.columns = ['Header', '#','Game ID','Home', 'Kicks for', 'Kicks against',
       'Marks for', 'Marks against', 'Handballs for', 'Handballs against',
       'Disposals for', 'Disposals against', 'Hitouts for', 'Hitouts against',
       'Tackles for', 'Tackles against', 'Rebound 50s for',
       'Rebound 50s against', 'Inside 50s for', 'Inside 50s against',
       'Clearances for', 'Clearances against', 'Clangers for',
       'Clangers against', 'Frees for', 'Frees against',
       'Contested possessions for', 'Contested possessions against',
       'Uncontested possessions for', 'Uncontested possessions against',
       'Contested marks for', 'Contested marks against', 'Marks inside 50 for',
       'Marks inside 50 against', '1%s for', '1%s against', 'Bounces for',
       'Bounces against', 'Goal assists for', 'Goal assists against',
       'Margin']

To allow for ordering by round number, the round number must be renamed so that rounds 1 to 9 are labeled as 01-09. This is performed on all 18 team tables

In [25]:
Ade=Ade.replace("R1", "R01")
Ade=Ade.replace("R2", "R02")
Ade=Ade.replace("R3", "R03")
Ade=Ade.replace("R4", "R04")
Ade=Ade.replace("R5", "R05")
Ade=Ade.replace("R6", "R06")
Ade=Ade.replace("R7", "R07")
Ade=Ade.replace("R8", "R08")
Ade=Ade.replace("R9", "R09")
Bri=Bri.replace("R1", "R01")
Bri=Bri.replace("R2", "R02")
Bri=Bri.replace("R3", "R03")
Bri=Bri.replace("R4", "R04")
Bri=Bri.replace("R5", "R05")
Bri=Bri.replace("R6", "R06")
Bri=Bri.replace("R7", "R07")
Bri=Bri.replace("R8", "R08")
Bri=Bri.replace("R9", "R09")
Car=Car.replace("R1", "R01")
Car=Car.replace("R2", "R02")
Car=Car.replace("R3", "R03")
Car=Car.replace("R4", "R04")
Car=Car.replace("R5", "R05")
Car=Car.replace("R6", "R06")
Car=Car.replace("R7", "R07")
Car=Car.replace("R8", "R08")
Car=Car.replace("R9", "R09")
Col=Col.replace("R1", "R01")
Col=Col.replace("R2", "R02")
Col=Col.replace("R3", "R03")
Col=Col.replace("R4", "R04")
Col=Col.replace("R5", "R05")
Col=Col.replace("R6", "R06")
Col=Col.replace("R7", "R07")
Col=Col.replace("R8", "R08")
Col=Col.replace("R9", "R09")
Ess=Ess.replace("R1", "R01")
Ess=Ess.replace("R2", "R02")
Ess=Ess.replace("R3", "R03")
Ess=Ess.replace("R4", "R04")
Ess=Ess.replace("R5", "R05")
Ess=Ess.replace("R6", "R06")
Ess=Ess.replace("R7", "R07")
Ess=Ess.replace("R8", "R08")
Ess=Ess.replace("R9", "R09")
Fre=Fre.replace("R1", "R01")
Fre=Fre.replace("R2", "R02")
Fre=Fre.replace("R3", "R03")
Fre=Fre.replace("R4", "R04")
Fre=Fre.replace("R5", "R05")
Fre=Fre.replace("R6", "R06")
Fre=Fre.replace("R7", "R07")
Fre=Fre.replace("R8", "R08")
Fre=Fre.replace("R9", "R09")
Gee=Gee.replace("R1", "R01")
Gee=Gee.replace("R2", "R02")
Gee=Gee.replace("R3", "R03")
Gee=Gee.replace("R4", "R04")
Gee=Gee.replace("R5", "R05")
Gee=Gee.replace("R6", "R06")
Gee=Gee.replace("R7", "R07")
Gee=Gee.replace("R8", "R08")
Gee=Gee.replace("R9", "R09")
Gol=Gol.replace("R1", "R01")
Gol=Gol.replace("R2", "R02")
Gol=Gol.replace("R3", "R03")
Gol=Gol.replace("R4", "R04")
Gol=Gol.replace("R5", "R05")
Gol=Gol.replace("R6", "R06")
Gol=Gol.replace("R7", "R07")
Gol=Gol.replace("R8", "R08")
Gol=Gol.replace("R9", "R09")
GWS=GWS.replace("R1", "R01")
GWS=GWS.replace("R2", "R02")
GWS=GWS.replace("R3", "R03")
GWS=GWS.replace("R4", "R04")
GWS=GWS.replace("R5", "R05")
GWS=GWS.replace("R6", "R06")
GWS=GWS.replace("R7", "R07")
GWS=GWS.replace("R8", "R08")
GWS=GWS.replace("R9", "R09")
Haw=Haw.replace("R1", "R01")
Haw=Haw.replace("R2", "R02")
Haw=Haw.replace("R3", "R03")
Haw=Haw.replace("R4", "R04")
Haw=Haw.replace("R5", "R05")
Haw=Haw.replace("R6", "R06")
Haw=Haw.replace("R7", "R07")
Haw=Haw.replace("R8", "R08")
Haw=Haw.replace("R9", "R09")
Mel=Mel.replace("R1", "R01")
Mel=Mel.replace("R2", "R02")
Mel=Mel.replace("R3", "R03")
Mel=Mel.replace("R4", "R04")
Mel=Mel.replace("R5", "R05")
Mel=Mel.replace("R6", "R06")
Mel=Mel.replace("R7", "R07")
Mel=Mel.replace("R8", "R08")
Mel=Mel.replace("R9", "R09")
Nor=Nor.replace("R1", "R01")
Nor=Nor.replace("R2", "R02")
Nor=Nor.replace("R3", "R03")
Nor=Nor.replace("R4", "R04")
Nor=Nor.replace("R5", "R05")
Nor=Nor.replace("R6", "R06")
Nor=Nor.replace("R7", "R07")
Nor=Nor.replace("R8", "R08")
Nor=Nor.replace("R9", "R09")
Por=Por.replace("R1", "R01")
Por=Por.replace("R2", "R02")
Por=Por.replace("R3", "R03")
Por=Por.replace("R4", "R04")
Por=Por.replace("R5", "R05")
Por=Por.replace("R6", "R06")
Por=Por.replace("R7", "R07")
Por=Por.replace("R8", "R08")
Por=Por.replace("R9", "R09")
Ric=Ric.replace("R1", "R01")
Ric=Ric.replace("R2", "R02")
Ric=Ric.replace("R3", "R03")
Ric=Ric.replace("R4", "R04")
Ric=Ric.replace("R5", "R05")
Ric=Ric.replace("R6", "R06")
Ric=Ric.replace("R7", "R07")
Ric=Ric.replace("R8", "R08")
Ric=Ric.replace("R9", "R09")
StK=StK.replace("R1", "R01")
StK=StK.replace("R2", "R02")
StK=StK.replace("R3", "R03")
StK=StK.replace("R4", "R04")
StK=StK.replace("R5", "R05")
StK=StK.replace("R6", "R06")
StK=StK.replace("R7", "R07")
StK=StK.replace("R8", "R08")
StK=StK.replace("R9", "R09")
Syd=Syd.replace("R1", "R01")
Syd=Syd.replace("R2", "R02")
Syd=Syd.replace("R3", "R03")
Syd=Syd.replace("R4", "R04")
Syd=Syd.replace("R5", "R05")
Syd=Syd.replace("R6", "R06")
Syd=Syd.replace("R7", "R07")
Syd=Syd.replace("R8", "R08")
Syd=Syd.replace("R9", "R09")
WCE=WCE.replace("R1", "R01")
WCE=WCE.replace("R2", "R02")
WCE=WCE.replace("R3", "R03")
WCE=WCE.replace("R4", "R04")
WCE=WCE.replace("R5", "R05")
WCE=WCE.replace("R6", "R06")
WCE=WCE.replace("R7", "R07")
WCE=WCE.replace("R8", "R08")
WCE=WCE.replace("R9", "R09")
Wes=Wes.replace("R1", "R01")
Wes=Wes.replace("R2", "R02")
Wes=Wes.replace("R3", "R03")
Wes=Wes.replace("R4", "R04")
Wes=Wes.replace("R5", "R05")
Wes=Wes.replace("R6", "R06")
Wes=Wes.replace("R7", "R07")
Wes=Wes.replace("R8", "R08")
Wes=Wes.replace("R9", "R09")

In [26]:
Ade = Ade.sort_values(by=['#'], ascending=True, ignore_index=True)
Bri = Bri.sort_values(by=['#'], ascending=True, ignore_index=True)
Car = Car.sort_values(by=['#'], ascending=True, ignore_index=True)
Col = Col.sort_values(by=['#'], ascending=True, ignore_index=True)
Ess = Ess.sort_values(by=['#'], ascending=True, ignore_index=True)
Fre = Fre.sort_values(by=['#'], ascending=True, ignore_index=True)
Gee = Gee.sort_values(by=['#'], ascending=True, ignore_index=True)
Gol = Gol.sort_values(by=['#'], ascending=True, ignore_index=True)
GWS = GWS.sort_values(by=['#'], ascending=True, ignore_index=True)
Haw = Haw.sort_values(by=['#'], ascending=True, ignore_index=True)
Mel = Mel.sort_values(by=['#'], ascending=True, ignore_index=True)
Nor = Nor.sort_values(by=['#'], ascending=True, ignore_index=True)
Por = Por.sort_values(by=['#'], ascending=True, ignore_index=True)
Ric = Ric.sort_values(by=['#'], ascending=True, ignore_index=True)
StK = StK.sort_values(by=['#'], ascending=True, ignore_index=True)
Syd = Syd.sort_values(by=['#'], ascending=True, ignore_index=True)
WCE = WCE.sort_values(by=['#'], ascending=True, ignore_index=True)
Wes = Wes.sort_values(by=['#'], ascending=True, ignore_index=True)

In [27]:
Ade

,Header,#,Game ID,Home,Kicks for,Kicks against,Marks for,Marks against,Handballs for,Handballs against,...,Contested marks against,Marks inside 50 for,Marks inside 50 against,1%s for,1%s against,Bounces for,Bounces against,Goal assists for,Goal assists against,Margin
0,Adelaide Team Statistics [Players],R01,2023017,0,217,208,102,88,101,167,...,8,15,16,42,49,3,5,10,9,LL
1,Adelaide Team Statistics [Players],R02,2023024,1,216,209,94,94,142,139,...,16,12,16,47,52,9,11,9,12,LL
2,Adelaide Team Statistics [Players],R03,2023036,0,212,188,88,63,120,109,...,8,13,9,49,51,1,2,13,10,LW
3,Adelaide Team Statistics [Players],R04,2023043,1,222,179,87,61,158,155,...,11,12,6,53,52,3,4,12,5,LW
4,Adelaide Team Statistics [Players],R05,2023051,1,242,183,116,71,156,137,...,14,20,7,52,47,3,3,15,6,BW
5,Adelaide Team Statistics [Players],R06,2023065,0,205,185,79,78,150,167,...,8,14,7,43,42,3,6,8,9,LW
6,Adelaide Team Statistics [Players],R07,2023079,1,228,208,105,72,139,147,...,4,9,7,49,60,3,7,3,2,LL
7,Adelaide Team Statistics [Players],R08,2023083,0,189,203,76,90,144,132,...,17,13,12,44,40,2,2,7,10,LL
8,Adelaide Team Statistics [Players],R09,2023098,1,255,201,126,78,186,148,...,7,12,14,52,48,3,5,12,7,BW
9,Adelaide Team Statistics [Players],R10,2023103,0,194,225,63,77,104,174,...,7,5,8,46,51,3,0,3,5,BL


Two new features will be calculated <br>
$\text{Inside 50 Eff For} = \frac{\text{Rebound 50s against}}{\text{Inside 50s for}}$ <br>
$\text{Rebound 50 Eff For} = \frac{\text{Rebound 50s for}}{\text{Inside 50s against}}$ <br>


A new feature called Elo will also be calculated. This will be used to determine the live ranking of each time similar to the power rankings. At the start of each season each team is assigned the same default Elo ranking. As games are played throughout the year, team's Elo ranking will increase the more then win and vice versa. This amount of increase is dependent on who the win against. If a low ranked Elo team wins against a high ranked Elo team, the winner will see a larger increase in Elo than if the higher ranked Elo team was the winner. This Elo metric can be used to determine the probability of a team winning and essentially the less likely an outcome occurs actually does occur the more a teams Elo will be shifted.

In [28]:
# Define a function which finds the elo for each team in each game and returns a dictionary with the game ID as a key and the
# elos as the key's value, in a list. It also outputs the probabilities and a dictionary of the final elos for each team
def elo_applier(df, k_factor):
    # Initialise a dictionary with default elos for each team
    elo_dict = {team: 1500 for team in df['Header'].unique()}
    elos, elo_probs = {}, {}
    
    # Get a home and away dataframe so that we can get the teams on the same row
    home_df = df.loc[df.Home == 1, ['Header', 'Game ID', 'Margin', 'Home']].rename(columns={'Header': 'home_team'})
    away_df = df.loc[df.Home == 0, ['Header', 'Game ID']].rename(columns={'Header': 'away_team'})
    
    df = (pd.merge(home_df, away_df, on='Game ID')
            .sort_values(by='Game ID')
            .drop_duplicates(subset='Game ID', keep='first')
            .reset_index(drop=True))

    # Loop over the rows in the DataFrame
    for index, row in df.iterrows():
        # Get the Game ID
        game_id = row['Game ID']
        
        # Get the margin
        margin = row['Margin']
        
        # If the game already has the elos for the home and away team in the elos dictionary, go to the next game
        if game_id in elos.keys():
            continue
        
        # Get the team and opposition
        home_team = row['home_team']
        away_team = row['away_team']
        
        # Get the team and opposition elo score
        home_team_elo = elo_dict[home_team]
        away_team_elo = elo_dict[away_team]
        
        # Calculated the probability of winning for the team and opposition
        prob_win_home = 1 / (1 + 10**((away_team_elo - home_team_elo) / 400))
        prob_win_away = 1 - prob_win_home
        
        # Add the elos and probabilities our elos dictionary and elo_probs dictionary based on the Game ID
        elos[game_id] = [home_team_elo, away_team_elo]
        elo_probs[game_id] = [prob_win_home, prob_win_away]
        
        # Calculate the new elos of each team
        if margin == 'BW' or margin == 'LW': # Home team wins; update both teams' elo
            new_home_team_elo = home_team_elo + k_factor*(1 - prob_win_home)
            new_away_team_elo = away_team_elo + k_factor*(0 - prob_win_away)
        elif margin == 'BL' or margin == 'LL': # Away team wins; update both teams' elo
            new_home_team_elo = home_team_elo + k_factor*(0 - prob_win_home)
            new_away_team_elo = away_team_elo + k_factor*(1 - prob_win_away)
        elif margin == 'D': # Drawn game' update both teams' elo
            new_home_team_elo = home_team_elo + k_factor*(0.5 - prob_win_home)
            new_away_team_elo = away_team_elo + k_factor*(0.5 - prob_win_away)
        
        # Update elos in elo dictionary
        elo_dict[home_team] = new_home_team_elo
        elo_dict[away_team] = new_away_team_elo
    
    return elos, elo_probs, elo_dict

In [29]:
# Use the elo applier function to get the elos and elo probabilities for each game - we will map these later
elos, probs, elo_dict = elo_applier(pd.concat([Ade,
                                              Bri,
                                              Car,
                                              Col,
                                              Ess,
                                              Fre,
                                              Gee,
                                              Gol,
                                              GWS,
                                              Haw,
                                              Mel,
                                              Nor,
                                              Por,
                                              Ric,
                                              StK,
                                              Syd,
                                              WCE,
                                              Wes]), 30)

### Elo Ladder <a class="anchor" id="elo"></a>

Interestingly the latest Elo metrics, when ranked largest to smallest, very closely follow the AFL ladder.

In [30]:
for team in sorted(elo_dict, key=elo_dict.get)[::-1]:
    print(team, elo_dict[team])

Collingwood Team Statistics [Players] 1624.7070302435227
Brisbane Lions Team Statistics [Players] 1604.5783949357838
Port Adelaide Team Statistics [Players] 1592.3809519629867
Melbourne Team Statistics [Players] 1580.3721339473898
Carlton Team Statistics [Players] 1576.7285079045912
Sydney Team Statistics [Players] 1548.1129335769704
Greater Western Sydney Team Statistics [Players] 1536.777818788957
St Kilda Team Statistics [Players] 1522.8292562451481
Geelong Team Statistics [Players] 1492.91201377354
Richmond Team Statistics [Players] 1487.3992866155222
Adelaide Team Statistics [Players] 1483.8219468319353
Essendon Team Statistics [Players] 1480.012777276485
Western Bulldogs Team Statistics [Players] 1467.274952203353
Gold Coast Team Statistics [Players] 1467.0377257095029
Fremantle Team Statistics [Players] 1457.6113384805662
Hawthorn Team Statistics [Players] 1437.8700389362386
West Coast Team Statistics [Players] 1340.7148683894318
North Melbourne Team Statistics [Players] 1298.85

In [31]:
Ade_gID = Ade['Game ID'].to_numpy()
Ade_home = Ade['Home'].to_numpy()
len(Ade_gID)
Ade_elos_for = np.zeros(len(Ade_gID))
Ade_elos_against = np.zeros(len(Ade_gID))
for i in range (0, (len(Ade_gID))):
    if Ade_home[i] == 1:
        Ade_elos_for[i] = elos[Ade_gID[i]][0]
        Ade_elos_against[i] = elos[Ade_gID[i]][1]
    elif Ade_home[i] == 0:
        Ade_elos_against[i] = elos[Ade_gID[i]][0]
        Ade_elos_for[i] = elos[Ade_gID[i]][1]
Ade_elos_for = pd.DataFrame(Ade_elos_for)
Ade_elos_against = pd.DataFrame(Ade_elos_against)
        
Bri_gID = Bri['Game ID'].to_numpy()
Bri_home = Bri['Home'].to_numpy()
len(Bri_gID)
Bri_elos_for = np.zeros(len(Bri_gID))
Bri_elos_against = np.zeros(len(Bri_gID))
for i in range (0, (len(Bri_gID))):
    if Bri_home[i] == 1:
        Bri_elos_for[i] = elos[Bri_gID[i]][0]
        Bri_elos_against[i] = elos[Bri_gID[i]][1]
    elif Bri_home[i] == 0:
        Bri_elos_against[i] = elos[Bri_gID[i]][0]
        Bri_elos_for[i] = elos[Bri_gID[i]][1]
Bri_elos_for = pd.DataFrame(Bri_elos_for)
Bri_elos_against = pd.DataFrame(Bri_elos_against)
        
Car_gID = Car['Game ID'].to_numpy()
Car_home = Car['Home'].to_numpy()
len(Car_gID)
Car_elos_for = np.zeros(len(Car_gID))
Car_elos_against = np.zeros(len(Car_gID))
for i in range (0, (len(Car_gID))):
    if Car_home[i] == 1:
        Car_elos_for[i] = elos[Car_gID[i]][0]
        Car_elos_against[i] = elos[Car_gID[i]][1]
    elif Car_home[i] == 0:
        Car_elos_against[i] = elos[Car_gID[i]][0]
        Car_elos_for[i] = elos[Car_gID[i]][1]
Car_elos_for = pd.DataFrame(Car_elos_for)
Car_elos_against = pd.DataFrame(Car_elos_against)
        
Col_gID = Col['Game ID'].to_numpy()
Col_home = Col['Home'].to_numpy()
len(Col_gID)
Col_elos_for = np.zeros(len(Col_gID))
Col_elos_against = np.zeros(len(Col_gID))
for i in range (0, (len(Col_gID))):
    if Col_home[i] == 1:
        Col_elos_for[i] = elos[Col_gID[i]][0]
        Col_elos_against[i] = elos[Col_gID[i]][1]
    elif Col_home[i] == 0:
        Col_elos_against[i] = elos[Col_gID[i]][0]
        Col_elos_for[i] = elos[Col_gID[i]][1]
Col_elos_for = pd.DataFrame(Col_elos_for)
Col_elos_against = pd.DataFrame(Col_elos_against)
        
Ess_gID = Ess['Game ID'].to_numpy()
Ess_home = Ess['Home'].to_numpy()
len(Ess_gID)
Ess_elos_for = np.zeros(len(Ess_gID))
Ess_elos_against = np.zeros(len(Ess_gID))
for i in range (0, (len(Ess_gID))):
    if Ess_home[i] == 1:
        Ess_elos_for[i] = elos[Ess_gID[i]][0]
        Ess_elos_against[i] = elos[Ess_gID[i]][1]
    elif Ess_home[i] == 0:
        Ess_elos_against[i] = elos[Ess_gID[i]][0]
        Ess_elos_for[i] = elos[Ess_gID[i]][1]
Ess_elos_for = pd.DataFrame(Ess_elos_for)
Ess_elos_against = pd.DataFrame(Ess_elos_against)
        
Fre_gID = Fre['Game ID'].to_numpy()
Fre_home = Fre['Home'].to_numpy()
len(Fre_gID)
Fre_elos_for = np.zeros(len(Fre_gID))
Fre_elos_against = np.zeros(len(Fre_gID))
for i in range (0, (len(Fre_gID))):
    if Fre_home[i] == 1:
        Fre_elos_for[i] = elos[Fre_gID[i]][0]
        Fre_elos_against[i] = elos[Fre_gID[i]][1]
    elif Fre_home[i] == 0:
        Fre_elos_against[i] = elos[Fre_gID[i]][0]
        Fre_elos_for[i] = elos[Fre_gID[i]][1]
Fre_elos_for = pd.DataFrame(Fre_elos_for)
Fre_elos_against = pd.DataFrame(Fre_elos_against)
        
Gee_gID = Gee['Game ID'].to_numpy()
Gee_home = Gee['Home'].to_numpy()
len(Gee_gID)
Gee_elos_for = np.zeros(len(Gee_gID))
Gee_elos_against = np.zeros(len(Gee_gID))
for i in range (0, (len(Gee_gID))):
    if Gee_home[i] == 1:
        Gee_elos_for[i] = elos[Gee_gID[i]][0]
        Gee_elos_against[i] = elos[Gee_gID[i]][1]
    elif Gee_home[i] == 0:
        Gee_elos_against[i] = elos[Gee_gID[i]][0]
        Gee_elos_for[i] = elos[Gee_gID[i]][1]
Gee_elos_for = pd.DataFrame(Gee_elos_for)
Gee_elos_against = pd.DataFrame(Gee_elos_against)
        
Gol_gID = Gol['Game ID'].to_numpy()
Gol_home = Gol['Home'].to_numpy()
len(Gol_gID)
Gol_elos_for = np.zeros(len(Gol_gID))
Gol_elos_against = np.zeros(len(Gol_gID))
for i in range (0, (len(Gol_gID))):
    if Gol_home[i] == 1:
        Gol_elos_for[i] = elos[Gol_gID[i]][0]
        Gol_elos_against[i] = elos[Gol_gID[i]][1]
    elif Gol_home[i] == 0:
        Gol_elos_against[i] = elos[Gol_gID[i]][0]
        Gol_elos_for[i] = elos[Gol_gID[i]][1]
Gol_elos_for = pd.DataFrame(Gol_elos_for)
Gol_elos_against = pd.DataFrame(Gol_elos_against)
        
GWS_gID = GWS['Game ID'].to_numpy()
GWS_home = GWS['Home'].to_numpy()
len(GWS_gID)
GWS_elos_for = np.zeros(len(GWS_gID))
GWS_elos_against = np.zeros(len(GWS_gID))
for i in range (0, (len(GWS_gID))):
    if GWS_home[i] == 1:
        GWS_elos_for[i] = elos[GWS_gID[i]][0]
        GWS_elos_against[i] = elos[GWS_gID[i]][1]
    elif GWS_home[i] == 0:
        GWS_elos_against[i] = elos[GWS_gID[i]][0]
        GWS_elos_for[i] = elos[GWS_gID[i]][1]
GWS_elos_for = pd.DataFrame(GWS_elos_for)
GWS_elos_against = pd.DataFrame(GWS_elos_against)
        
Haw_gID = Haw['Game ID'].to_numpy()
Haw_home = Haw['Home'].to_numpy()
len(Haw_gID)
Haw_elos_for = np.zeros(len(Haw_gID))
Haw_elos_against = np.zeros(len(Haw_gID))
for i in range (0, (len(Haw_gID))):
    if Haw_home[i] == 1:
        Haw_elos_for[i] = elos[Haw_gID[i]][0]
        Haw_elos_against[i] = elos[Haw_gID[i]][1]
    elif Haw_home[i] == 0:
        Haw_elos_against[i] = elos[Haw_gID[i]][0]
        Haw_elos_for[i] = elos[Haw_gID[i]][1]
Haw_elos_for = pd.DataFrame(Haw_elos_for)
Haw_elos_against = pd.DataFrame(Haw_elos_against)
        
Mel_gID = Mel['Game ID'].to_numpy()
Mel_home = Mel['Home'].to_numpy()
len(Mel_gID)
Mel_elos_for = np.zeros(len(Mel_gID))
Mel_elos_against = np.zeros(len(Mel_gID))
for i in range (0, (len(Mel_gID))):
    if Mel_home[i] == 1:
        Mel_elos_for[i] = elos[Mel_gID[i]][0]
        Mel_elos_against[i] = elos[Mel_gID[i]][1]
    elif Mel_home[i] == 0:
        Mel_elos_against[i] = elos[Mel_gID[i]][0]
        Mel_elos_for[i] = elos[Mel_gID[i]][1]
Mel_elos_for = pd.DataFrame(Mel_elos_for)
Mel_elos_against = pd.DataFrame(Mel_elos_against)
        
Nor_gID = Nor['Game ID'].to_numpy()
Nor_home = Nor['Home'].to_numpy()
len(Nor_gID)
Nor_elos_for = np.zeros(len(Nor_gID))
Nor_elos_against = np.zeros(len(Nor_gID))
for i in range (0, (len(Nor_gID))):
    if Nor_home[i] == 1:
        Nor_elos_for[i] = elos[Nor_gID[i]][0]
        Nor_elos_against[i] = elos[Nor_gID[i]][1]
    elif Nor_home[i] == 0:
        Nor_elos_against[i] = elos[Nor_gID[i]][0]
        Nor_elos_for[i] = elos[Nor_gID[i]][1]
Nor_elos_for = pd.DataFrame(Nor_elos_for)
Nor_elos_against = pd.DataFrame(Nor_elos_against)
        
Por_gID = Por['Game ID'].to_numpy()
Por_home = Por['Home'].to_numpy()
len(Por_gID)
Por_elos_for = np.zeros(len(Por_gID))
Por_elos_against = np.zeros(len(Por_gID))
for i in range (0, (len(Por_gID))):
    if Por_home[i] == 1:
        Por_elos_for[i] = elos[Por_gID[i]][0]
        Por_elos_against[i] = elos[Por_gID[i]][1]
    elif Por_home[i] == 0:
        Por_elos_against[i] = elos[Por_gID[i]][0]
        Por_elos_for[i] = elos[Por_gID[i]][1]
Por_elos_for = pd.DataFrame(Por_elos_for)
Por_elos_against = pd.DataFrame(Por_elos_against)
    
Ric_gID = Ric['Game ID'].to_numpy()
Ric_home = Ric['Home'].to_numpy()
len(Ric_gID)
Ric_elos_for = np.zeros(len(Ric_gID))
Ric_elos_against = np.zeros(len(Ric_gID))
for i in range (0, (len(Ric_gID))):
    if Ric_home[i] == 1:
        Ric_elos_for[i] = elos[Ric_gID[i]][0]
        Ric_elos_against[i] = elos[Ric_gID[i]][1]
    elif Ric_home[i] == 0:
        Ric_elos_against[i] = elos[Ric_gID[i]][0]
        Ric_elos_for[i] = elos[Ric_gID[i]][1]
Ric_elos_for = pd.DataFrame(Ric_elos_for)
Ric_elos_against = pd.DataFrame(Ric_elos_against)
    
StK_gID = StK['Game ID'].to_numpy()
StK_home = StK['Home'].to_numpy()
len(StK_gID)
StK_elos_for = np.zeros(len(StK_gID))
StK_elos_against = np.zeros(len(StK_gID))
for i in range (0, (len(StK_gID))):
    if StK_home[i] == 1:
        StK_elos_for[i] = elos[StK_gID[i]][0]
        StK_elos_against[i] = elos[StK_gID[i]][1]
    elif StK_home[i] == 0:
        StK_elos_against[i] = elos[StK_gID[i]][0]
        StK_elos_for[i] = elos[StK_gID[i]][1]
StK_elos_for = pd.DataFrame(StK_elos_for)
StK_elos_against = pd.DataFrame(StK_elos_against)
        
Syd_gID = Syd['Game ID'].to_numpy()
Syd_home = Syd['Home'].to_numpy()
len(Syd_gID)
Syd_elos_for = np.zeros(len(Syd_gID))
Syd_elos_against = np.zeros(len(Syd_gID))
for i in range (0, (len(Syd_gID))):
    if Syd_home[i] == 1:
        Syd_elos_for[i] = elos[Syd_gID[i]][0]
        Syd_elos_against[i] = elos[Syd_gID[i]][1]
    elif Syd_home[i] == 0:
        Syd_elos_against[i] = elos[Syd_gID[i]][0]
        Syd_elos_for[i] = elos[Syd_gID[i]][1]
Syd_elos_for = pd.DataFrame(Syd_elos_for)
Syd_elos_against = pd.DataFrame(Syd_elos_against)
        
WCE_gID = WCE['Game ID'].to_numpy()
WCE_home = WCE['Home'].to_numpy()
len(WCE_gID)
WCE_elos_for = np.zeros(len(WCE_gID))
WCE_elos_against = np.zeros(len(WCE_gID))
for i in range (0, (len(WCE_gID))):
    if WCE_home[i] == 1:
        WCE_elos_for[i] = elos[WCE_gID[i]][0]
        WCE_elos_against[i] = elos[WCE_gID[i]][1]
    elif WCE_home[i] == 0:
        WCE_elos_against[i] = elos[WCE_gID[i]][0]
        WCE_elos_for[i] = elos[WCE_gID[i]][1]
WCE_elos_for = pd.DataFrame(WCE_elos_for)
WCE_elos_against = pd.DataFrame(WCE_elos_against)
        
Wes_gID = Wes['Game ID'].to_numpy()
Wes_home = Wes['Home'].to_numpy()
len(Wes_gID)
Wes_elos_for = np.zeros(len(Wes_gID))
Wes_elos_against = np.zeros(len(Wes_gID))
for i in range (0, (len(Wes_gID))):
    if Wes_home[i] == 1:
        Wes_elos_for[i] = elos[Wes_gID[i]][0]
        Wes_elos_against[i] = elos[Wes_gID[i]][1]
    elif Wes_home[i] == 0:
        Wes_elos_against[i] = elos[Wes_gID[i]][0]
        Wes_elos_for[i] = elos[Wes_gID[i]][1]
Wes_elos_for = pd.DataFrame(Wes_elos_for)
Wes_elos_against = pd.DataFrame(Wes_elos_against)

In [32]:
Ade = pd.concat([Ade, Ade['Rebound 50s against']/Ade['Inside 50s for'],Ade['Rebound 50s for']/Ade['Inside 50s against'],
                 Ade_elos_for, Ade_elos_against], axis=1)
Ade.columns = ['Header', '#','Game ID','Home', 'Kicks for', 'Kicks against',
       'Marks for', 'Marks against', 'Handballs for', 'Handballs against',
       'Disposals for', 'Disposals against', 'Hitouts for', 'Hitouts against',
       'Tackles for', 'Tackles against', 'Rebound 50s for',
       'Rebound 50s against', 'Inside 50s for', 'Inside 50s against',
       'Clearances for', 'Clearances against', 'Clangers for',
       'Clangers against','Frees for','Frees against',
       'Contested possessions for', 'Contested possessions against',
       'Uncontested possessions for', 'Uncontested possessions against',
       'Contested marks for', 'Contested marks against', 'Marks inside 50 for',
       'Marks inside 50 against', '1%s for', '1%s against', 'Bounces for',
       'Bounces against', 'Goal assists for', 'Goal assists against',
       'Margin', 'Inside 50 Eff', 'Rebound 50 Eff', 'Elos for','Elos against']
Ade.insert(len(Ade.columns)-1, 'Margin', Ade.pop('Margin'))

Bri = pd.concat([Bri, Bri['Rebound 50s against']/Bri['Inside 50s for'],Bri['Rebound 50s for']/Bri['Inside 50s against'],
                 Bri_elos_for, Bri_elos_against], axis=1)
Bri.columns = ['Header', '#','Game ID','Home', 'Kicks for', 'Kicks against',
       'Marks for', 'Marks against', 'Handballs for', 'Handballs against',
       'Disposals for', 'Disposals against', 'Hitouts for', 'Hitouts against',
       'Tackles for', 'Tackles against', 'Rebound 50s for',
       'Rebound 50s against', 'Inside 50s for', 'Inside 50s against',
       'Clearances for', 'Clearances against', 'Clangers for',
       'Clangers against','Frees for','Frees against',
       'Contested possessions for', 'Contested possessions against',
       'Uncontested possessions for', 'Uncontested possessions against',
       'Contested marks for', 'Contested marks against', 'Marks inside 50 for',
       'Marks inside 50 against', '1%s for', '1%s against', 'Bounces for',
       'Bounces against', 'Goal assists for', 'Goal assists against',
       'Margin', 'Inside 50 Eff', 'Rebound 50 Eff', 'Elos for','Elos against']
Bri.insert(len(Bri.columns)-1, 'Margin', Bri.pop('Margin'))

Car = pd.concat([Car, Car['Rebound 50s against']/Car['Inside 50s for'],Car['Rebound 50s for']/Car['Inside 50s against'],
                 Car_elos_for, Car_elos_against], axis=1)
Car.columns = ['Header', '#','Game ID','Home', 'Kicks for', 'Kicks against',
       'Marks for', 'Marks against', 'Handballs for', 'Handballs against',
       'Disposals for', 'Disposals against', 'Hitouts for', 'Hitouts against',
       'Tackles for', 'Tackles against', 'Rebound 50s for',
       'Rebound 50s against', 'Inside 50s for', 'Inside 50s against',
       'Clearances for', 'Clearances against', 'Clangers for',
       'Clangers against','Frees for','Frees against',
       'Contested possessions for', 'Contested possessions against',
       'Uncontested possessions for', 'Uncontested possessions against',
       'Contested marks for', 'Contested marks against', 'Marks inside 50 for',
       'Marks inside 50 against', '1%s for', '1%s against', 'Bounces for',
       'Bounces against', 'Goal assists for', 'Goal assists against',
       'Margin', 'Inside 50 Eff', 'Rebound 50 Eff', 'Elos for','Elos against']
Car.insert(len(Car.columns)-1, 'Margin', Car.pop('Margin'))

Col = pd.concat([Col, Col['Rebound 50s against']/Col['Inside 50s for'],Col['Rebound 50s for']/Col['Inside 50s against'],
                 Col_elos_for, Col_elos_against], axis=1)
Col.columns = ['Header', '#','Game ID','Home', 'Kicks for', 'Kicks against',
       'Marks for', 'Marks against', 'Handballs for', 'Handballs against',
       'Disposals for', 'Disposals against', 'Hitouts for', 'Hitouts against',
       'Tackles for', 'Tackles against', 'Rebound 50s for',
       'Rebound 50s against', 'Inside 50s for', 'Inside 50s against',
       'Clearances for', 'Clearances against', 'Clangers for',
       'Clangers against','Frees for','Frees against',
       'Contested possessions for', 'Contested possessions against',
       'Uncontested possessions for', 'Uncontested possessions against',
       'Contested marks for', 'Contested marks against', 'Marks inside 50 for',
       'Marks inside 50 against', '1%s for', '1%s against', 'Bounces for',
       'Bounces against', 'Goal assists for', 'Goal assists against',
       'Margin', 'Inside 50 Eff', 'Rebound 50 Eff', 'Elos for','Elos against']
Col.insert(len(Col.columns)-1, 'Margin', Col.pop('Margin'))

Ess = pd.concat([Ess, Ess['Rebound 50s against']/Ess['Inside 50s for'],Ess['Rebound 50s for']/Ess['Inside 50s against'],
                 Ess_elos_for, Ess_elos_against], axis=1)
Ess.columns = ['Header', '#','Game ID','Home', 'Kicks for', 'Kicks against',
       'Marks for', 'Marks against', 'Handballs for', 'Handballs against',
       'Disposals for', 'Disposals against', 'Hitouts for', 'Hitouts against',
       'Tackles for', 'Tackles against', 'Rebound 50s for',
       'Rebound 50s against', 'Inside 50s for', 'Inside 50s against',
       'Clearances for', 'Clearances against', 'Clangers for',
       'Clangers against','Frees for','Frees against',
       'Contested possessions for', 'Contested possessions against',
       'Uncontested possessions for', 'Uncontested possessions against',
       'Contested marks for', 'Contested marks against', 'Marks inside 50 for',
       'Marks inside 50 against', '1%s for', '1%s against', 'Bounces for',
       'Bounces against', 'Goal assists for', 'Goal assists against',
       'Margin', 'Inside 50 Eff', 'Rebound 50 Eff', 'Elos for','Elos against']
Ess.insert(len(Ess.columns)-1, 'Margin', Ess.pop('Margin'))

Fre = pd.concat([Fre, Fre['Rebound 50s against']/Fre['Inside 50s for'],Fre['Rebound 50s for']/Fre['Inside 50s against'],
                 Fre_elos_for, Fre_elos_against], axis=1)
Fre.columns = ['Header', '#','Game ID','Home', 'Kicks for', 'Kicks against',
       'Marks for', 'Marks against', 'Handballs for', 'Handballs against',
       'Disposals for', 'Disposals against', 'Hitouts for', 'Hitouts against',
       'Tackles for', 'Tackles against', 'Rebound 50s for',
       'Rebound 50s against', 'Inside 50s for', 'Inside 50s against',
       'Clearances for', 'Clearances against', 'Clangers for',
       'Clangers against','Frees for','Frees against',
       'Contested possessions for', 'Contested possessions against',
       'Uncontested possessions for', 'Uncontested possessions against',
       'Contested marks for', 'Contested marks against', 'Marks inside 50 for',
       'Marks inside 50 against', '1%s for', '1%s against', 'Bounces for',
       'Bounces against', 'Goal assists for', 'Goal assists against',
       'Margin', 'Inside 50 Eff', 'Rebound 50 Eff', 'Elos for','Elos against']
Fre.insert(len(Fre.columns)-1, 'Margin', Fre.pop('Margin'))

Gee = pd.concat([Gee, Gee['Rebound 50s against']/Gee['Inside 50s for'],Gee['Rebound 50s for']/Gee['Inside 50s against'],
                 Gee_elos_for, Gee_elos_against], axis=1)
Gee.columns = ['Header', '#','Game ID','Home', 'Kicks for', 'Kicks against',
       'Marks for', 'Marks against', 'Handballs for', 'Handballs against',
       'Disposals for', 'Disposals against', 'Hitouts for', 'Hitouts against',
       'Tackles for', 'Tackles against', 'Rebound 50s for',
       'Rebound 50s against', 'Inside 50s for', 'Inside 50s against',
       'Clearances for', 'Clearances against', 'Clangers for',
       'Clangers against','Frees for','Frees against',
       'Contested possessions for', 'Contested possessions against',
       'Uncontested possessions for', 'Uncontested possessions against',
       'Contested marks for', 'Contested marks against', 'Marks inside 50 for',
       'Marks inside 50 against', '1%s for', '1%s against', 'Bounces for',
       'Bounces against', 'Goal assists for', 'Goal assists against',
       'Margin', 'Inside 50 Eff', 'Rebound 50 Eff', 'Elos for','Elos against']
Gee.insert(len(Gee.columns)-1, 'Margin', Gee.pop('Margin'))

Gol = pd.concat([Gol, Gol['Rebound 50s against']/Gol['Inside 50s for'],Gol['Rebound 50s for']/Gol['Inside 50s against'],
                 Gol_elos_for, Gol_elos_against], axis=1)
Gol.columns = ['Header', '#','Game ID','Home', 'Kicks for', 'Kicks against',
       'Marks for', 'Marks against', 'Handballs for', 'Handballs against',
       'Disposals for', 'Disposals against', 'Hitouts for', 'Hitouts against',
       'Tackles for', 'Tackles against', 'Rebound 50s for',
       'Rebound 50s against', 'Inside 50s for', 'Inside 50s against',
       'Clearances for', 'Clearances against', 'Clangers for',
       'Clangers against','Frees for','Frees against',
       'Contested possessions for', 'Contested possessions against',
       'Uncontested possessions for', 'Uncontested possessions against',
       'Contested marks for', 'Contested marks against', 'Marks inside 50 for',
       'Marks inside 50 against', '1%s for', '1%s against', 'Bounces for',
       'Bounces against', 'Goal assists for', 'Goal assists against',
       'Margin', 'Inside 50 Eff', 'Rebound 50 Eff', 'Elos for','Elos against']
Gol.insert(len(Gol.columns)-1, 'Margin', Gol.pop('Margin'))

GWS = pd.concat([GWS, GWS['Rebound 50s against']/GWS['Inside 50s for'],GWS['Rebound 50s for']/GWS['Inside 50s against'],
                 GWS_elos_for, GWS_elos_against], axis=1)
GWS.columns = ['Header', '#','Game ID','Home', 'Kicks for', 'Kicks against',
       'Marks for', 'Marks against', 'Handballs for', 'Handballs against',
       'Disposals for', 'Disposals against', 'Hitouts for', 'Hitouts against',
       'Tackles for', 'Tackles against', 'Rebound 50s for',
       'Rebound 50s against', 'Inside 50s for', 'Inside 50s against',
       'Clearances for', 'Clearances against', 'Clangers for',
       'Clangers against','Frees for','Frees against',
       'Contested possessions for', 'Contested possessions against',
       'Uncontested possessions for', 'Uncontested possessions against',
       'Contested marks for', 'Contested marks against', 'Marks inside 50 for',
       'Marks inside 50 against', '1%s for', '1%s against', 'Bounces for',
       'Bounces against', 'Goal assists for', 'Goal assists against',
       'Margin', 'Inside 50 Eff', 'Rebound 50 Eff', 'Elos for','Elos against']
GWS.insert(len(GWS.columns)-1, 'Margin', GWS.pop('Margin'))

Haw = pd.concat([Haw, Haw['Rebound 50s against']/Haw['Inside 50s for'],Haw['Rebound 50s for']/Haw['Inside 50s against'],
                 Haw_elos_for, Haw_elos_against], axis=1)
Haw.columns = ['Header', '#','Game ID','Home', 'Kicks for', 'Kicks against',
       'Marks for', 'Marks against', 'Handballs for', 'Handballs against',
       'Disposals for', 'Disposals against', 'Hitouts for', 'Hitouts against',
       'Tackles for', 'Tackles against', 'Rebound 50s for',
       'Rebound 50s against', 'Inside 50s for', 'Inside 50s against',
       'Clearances for', 'Clearances against', 'Clangers for',
       'Clangers against','Frees for','Frees against',
       'Contested possessions for', 'Contested possessions against',
       'Uncontested possessions for', 'Uncontested possessions against',
       'Contested marks for', 'Contested marks against', 'Marks inside 50 for',
       'Marks inside 50 against', '1%s for', '1%s against', 'Bounces for',
       'Bounces against', 'Goal assists for', 'Goal assists against',
       'Margin', 'Inside 50 Eff', 'Rebound 50 Eff', 'Elos for','Elos against']
Haw.insert(len(Haw.columns)-1, 'Margin', Haw.pop('Margin'))

Mel = pd.concat([Mel, Mel['Rebound 50s against']/Mel['Inside 50s for'],Mel['Rebound 50s for']/Mel['Inside 50s against'],
                 Mel_elos_for, Mel_elos_against], axis=1)
Mel.columns = ['Header', '#','Game ID','Home', 'Kicks for', 'Kicks against',
       'Marks for', 'Marks against', 'Handballs for', 'Handballs against',
       'Disposals for', 'Disposals against', 'Hitouts for', 'Hitouts against',
       'Tackles for', 'Tackles against', 'Rebound 50s for',
       'Rebound 50s against', 'Inside 50s for', 'Inside 50s against',
       'Clearances for', 'Clearances against', 'Clangers for',
       'Clangers against','Frees for','Frees against',
       'Contested possessions for', 'Contested possessions against',
       'Uncontested possessions for', 'Uncontested possessions against',
       'Contested marks for', 'Contested marks against', 'Marks inside 50 for',
       'Marks inside 50 against', '1%s for', '1%s against', 'Bounces for',
       'Bounces against', 'Goal assists for', 'Goal assists against',
       'Margin', 'Inside 50 Eff', 'Rebound 50 Eff', 'Elos for','Elos against']
Mel.insert(len(Mel.columns)-1, 'Margin', Mel.pop('Margin'))

Nor = pd.concat([Nor, Nor['Rebound 50s against']/Nor['Inside 50s for'],Nor['Rebound 50s for']/Nor['Inside 50s against'],
                 Nor_elos_for, Nor_elos_against], axis=1)
Nor.columns = ['Header', '#','Game ID','Home', 'Kicks for', 'Kicks against',
       'Marks for', 'Marks against', 'Handballs for', 'Handballs against',
       'Disposals for', 'Disposals against', 'Hitouts for', 'Hitouts against',
       'Tackles for', 'Tackles against', 'Rebound 50s for',
       'Rebound 50s against', 'Inside 50s for', 'Inside 50s against',
       'Clearances for', 'Clearances against', 'Clangers for',
       'Clangers against','Frees for','Frees against',
       'Contested possessions for', 'Contested possessions against',
       'Uncontested possessions for', 'Uncontested possessions against',
       'Contested marks for', 'Contested marks against', 'Marks inside 50 for',
       'Marks inside 50 against', '1%s for', '1%s against', 'Bounces for',
       'Bounces against', 'Goal assists for', 'Goal assists against',
       'Margin', 'Inside 50 Eff', 'Rebound 50 Eff', 'Elos for','Elos against']
Nor.insert(len(Nor.columns)-1, 'Margin', Nor.pop('Margin'))

Por = pd.concat([Por, Por['Rebound 50s against']/Por['Inside 50s for'],Por['Rebound 50s for']/Por['Inside 50s against'],
                 Por_elos_for, Por_elos_against], axis=1)
Por.columns = ['Header', '#','Game ID','Home', 'Kicks for', 'Kicks against',
       'Marks for', 'Marks against', 'Handballs for', 'Handballs against',
       'Disposals for', 'Disposals against', 'Hitouts for', 'Hitouts against',
       'Tackles for', 'Tackles against', 'Rebound 50s for',
       'Rebound 50s against', 'Inside 50s for', 'Inside 50s against',
       'Clearances for', 'Clearances against', 'Clangers for',
       'Clangers against','Frees for','Frees against',
       'Contested possessions for', 'Contested possessions against',
       'Uncontested possessions for', 'Uncontested possessions against',
       'Contested marks for', 'Contested marks against', 'Marks inside 50 for',
       'Marks inside 50 against', '1%s for', '1%s against', 'Bounces for',
       'Bounces against', 'Goal assists for', 'Goal assists against',
       'Margin', 'Inside 50 Eff', 'Rebound 50 Eff', 'Elos for','Elos against']
Por.insert(len(Por.columns)-1, 'Margin', Por.pop('Margin'))

Ric = pd.concat([Ric, Ric['Rebound 50s against']/Ric['Inside 50s for'],Ric['Rebound 50s for']/Ric['Inside 50s against'],
                 Ric_elos_for, Ric_elos_against], axis=1)
Ric.columns = ['Header', '#','Game ID','Home', 'Kicks for', 'Kicks against',
       'Marks for', 'Marks against', 'Handballs for', 'Handballs against',
       'Disposals for', 'Disposals against', 'Hitouts for', 'Hitouts against',
       'Tackles for', 'Tackles against', 'Rebound 50s for',
       'Rebound 50s against', 'Inside 50s for', 'Inside 50s against',
       'Clearances for', 'Clearances against', 'Clangers for',
       'Clangers against','Frees for','Frees against',
       'Contested possessions for', 'Contested possessions against',
       'Uncontested possessions for', 'Uncontested possessions against',
       'Contested marks for', 'Contested marks against', 'Marks inside 50 for',
       'Marks inside 50 against', '1%s for', '1%s against', 'Bounces for',
       'Bounces against', 'Goal assists for', 'Goal assists against',
       'Margin', 'Inside 50 Eff', 'Rebound 50 Eff', 'Elos for','Elos against']
Ric.insert(len(Ric.columns)-1, 'Margin', Ric.pop('Margin'))

StK = pd.concat([StK, StK['Rebound 50s against']/StK['Inside 50s for'],StK['Rebound 50s for']/StK['Inside 50s against'],
                 StK_elos_for, StK_elos_against], axis=1)
StK.columns = ['Header', '#','Game ID','Home', 'Kicks for', 'Kicks against',
       'Marks for', 'Marks against', 'Handballs for', 'Handballs against',
       'Disposals for', 'Disposals against', 'Hitouts for', 'Hitouts against',
       'Tackles for', 'Tackles against', 'Rebound 50s for',
       'Rebound 50s against', 'Inside 50s for', 'Inside 50s against',
       'Clearances for', 'Clearances against', 'Clangers for',
       'Clangers against','Frees for','Frees against',
       'Contested possessions for', 'Contested possessions against',
       'Uncontested possessions for', 'Uncontested possessions against',
       'Contested marks for', 'Contested marks against', 'Marks inside 50 for',
       'Marks inside 50 against', '1%s for', '1%s against', 'Bounces for',
       'Bounces against', 'Goal assists for', 'Goal assists against',
       'Margin', 'Inside 50 Eff', 'Rebound 50 Eff', 'Elos for','Elos against']
StK.insert(len(StK.columns)-1, 'Margin', StK.pop('Margin'))

Syd = pd.concat([Syd, Syd['Rebound 50s against']/Syd['Inside 50s for'],Syd['Rebound 50s for']/Syd['Inside 50s against'],
                 Syd_elos_for, Syd_elos_against], axis=1)
Syd.columns = ['Header', '#','Game ID','Home', 'Kicks for', 'Kicks against',
       'Marks for', 'Marks against', 'Handballs for', 'Handballs against',
       'Disposals for', 'Disposals against', 'Hitouts for', 'Hitouts against',
       'Tackles for', 'Tackles against', 'Rebound 50s for',
       'Rebound 50s against', 'Inside 50s for', 'Inside 50s against',
       'Clearances for', 'Clearances against', 'Clangers for',
       'Clangers against','Frees for','Frees against',
       'Contested possessions for', 'Contested possessions against',
       'Uncontested possessions for', 'Uncontested possessions against',
       'Contested marks for', 'Contested marks against', 'Marks inside 50 for',
       'Marks inside 50 against', '1%s for', '1%s against', 'Bounces for',
       'Bounces against', 'Goal assists for', 'Goal assists against',
       'Margin', 'Inside 50 Eff', 'Rebound 50 Eff', 'Elos for','Elos against']
Syd.insert(len(Syd.columns)-1, 'Margin', Syd.pop('Margin'))

WCE = pd.concat([WCE, WCE['Rebound 50s against']/WCE['Inside 50s for'],WCE['Rebound 50s for']/WCE['Inside 50s against'],
                 WCE_elos_for, WCE_elos_against], axis=1)
WCE.columns = ['Header', '#','Game ID','Home', 'Kicks for', 'Kicks against',
       'Marks for', 'Marks against', 'Handballs for', 'Handballs against',
       'Disposals for', 'Disposals against', 'Hitouts for', 'Hitouts against',
       'Tackles for', 'Tackles against', 'Rebound 50s for',
       'Rebound 50s against', 'Inside 50s for', 'Inside 50s against',
       'Clearances for', 'Clearances against', 'Clangers for',
       'Clangers against','Frees for','Frees against',
       'Contested possessions for', 'Contested possessions against',
       'Uncontested possessions for', 'Uncontested possessions against',
       'Contested marks for', 'Contested marks against', 'Marks inside 50 for',
       'Marks inside 50 against', '1%s for', '1%s against', 'Bounces for',
       'Bounces against', 'Goal assists for', 'Goal assists against',
       'Margin', 'Inside 50 Eff', 'Rebound 50 Eff', 'Elos for','Elos against']
WCE.insert(len(WCE.columns)-1, 'Margin', WCE.pop('Margin'))

Wes = pd.concat([Wes, Wes['Rebound 50s against']/Wes['Inside 50s for'],Wes['Rebound 50s for']/Wes['Inside 50s against'],
                 Wes_elos_for, Wes_elos_against], axis=1)
Wes.columns = ['Header', '#','Game ID','Home', 'Kicks for', 'Kicks against',
       'Marks for', 'Marks against', 'Handballs for', 'Handballs against',
       'Disposals for', 'Disposals against', 'Hitouts for', 'Hitouts against',
       'Tackles for', 'Tackles against', 'Rebound 50s for',
       'Rebound 50s against', 'Inside 50s for', 'Inside 50s against',
       'Clearances for', 'Clearances against', 'Clangers for',
       'Clangers against','Frees for','Frees against',
       'Contested possessions for', 'Contested possessions against',
       'Uncontested possessions for', 'Uncontested possessions against',
       'Contested marks for', 'Contested marks against', 'Marks inside 50 for',
       'Marks inside 50 against', '1%s for', '1%s against', 'Bounces for',
       'Bounces against', 'Goal assists for', 'Goal assists against',
       'Margin', 'Inside 50 Eff', 'Rebound 50 Eff', 'Elos for','Elos against']
Wes.insert(len(Wes.columns)-1, 'Margin', Wes.pop('Margin'))

We now have a completed data table for each team with looks like this. 

These team tables must now be combined to create a data table to capture every match played in the season. This data set will be sorted by ascending round number.

In [33]:
Full_Data = pd.concat([Ade, 
                       Bri, 
                       Car, 
                       Col, 
                       Ess, 
                       Fre, 
                       Gee, 
                       Gol,
                       GWS,
                       Haw,
                       Mel,
                       Nor,
                       Por,
                       Ric,
                       StK,
                       Syd,
                       WCE,
                       Wes], ignore_index=True)

In [34]:
New_Full_Data = Full_Data.sort_values(by=['#'], ignore_index=True)
New_Full_Data.to_excel('2023 Data clean v3.1.xlsx', sheet_name='sheet1', index=False)

The for and against values for each stat will now be taken away from each other apart from the following stats:
- Inside 50 Eff
- Rebound 50 Eff

This will allow for statistics to be normally distributed.

In [35]:
Games = pd.concat([New_Full_Data['Kicks for']-New_Full_Data['Kicks against'].to_numpy(),
                  New_Full_Data['Marks for']-New_Full_Data['Marks against'].to_numpy(),
                  New_Full_Data['Handballs for']-New_Full_Data['Handballs against'].to_numpy(),
                  New_Full_Data['Disposals for']-New_Full_Data['Disposals against'].to_numpy(),
                  New_Full_Data['Hitouts for']-New_Full_Data['Hitouts against'].to_numpy(),
                  New_Full_Data['Tackles for']-New_Full_Data['Tackles against'].to_numpy(),
                  New_Full_Data['Rebound 50s for']-New_Full_Data['Rebound 50s against'].to_numpy(),
                  New_Full_Data['Inside 50s for']-New_Full_Data['Inside 50s against'].to_numpy(),
                  New_Full_Data['Clearances for']-New_Full_Data['Clearances against'].to_numpy(),
                  New_Full_Data['Clangers for']-New_Full_Data['Clangers against'].to_numpy(),
                  New_Full_Data['Frees for']-New_Full_Data['Frees against'].to_numpy(),
                  New_Full_Data['Contested possessions for']-New_Full_Data['Contested possessions against'].to_numpy(),
                  New_Full_Data['Uncontested possessions for']-New_Full_Data['Uncontested possessions against'].to_numpy(),
                  New_Full_Data['Contested marks for']-New_Full_Data['Contested marks against'].to_numpy(),
                  New_Full_Data['Marks inside 50 for']-New_Full_Data['Marks inside 50 against'].to_numpy(),
                  New_Full_Data['1%s for']-New_Full_Data['1%s against'].to_numpy(),
                  New_Full_Data['Bounces for']-New_Full_Data['Bounces against'].to_numpy(),
                  New_Full_Data['Goal assists for']-New_Full_Data['Goal assists against'].to_numpy(),
                  New_Full_Data['Inside 50 Eff'], New_Full_Data['Rebound 50 Eff'],
                  New_Full_Data['Elos for'] -New_Full_Data['Elos against'].to_numpy()], axis = 1)
Games.columns = ['Kicks', 'Marks', 'Handballs', 'Disposals','Hitouts', 'Tackles', 'Rebound 50s', 'Inside 50s',
                 'Clearances', 'Clangers','Frees', 'Contested possessions', 'Uncontested possessions',
                 'Contested marks', 'Marks inside 50', '1%s', 'Bounces', 'Goal assists', 'Inside 50 Eff', 
                 'Rebound 50 Eff', 'Elos']
n=len(Games.columns)
Games.to_excel('2023 Game diff v3.1.xlsx', sheet_name='sheet1', index=False)

## Model Training  <a class="anchor" id="train"></a>

The model will now be trained. The training set used will be 80% of all the matches played during the year.

In [36]:
X = Games
Y = New_Full_Data.iloc[:,44]

In [37]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=999)
model = GaussianNB()
model.fit(X_train, Y_train)
acc1 = model.score(X_test, Y_test)
acc = "{:.1%}".format(acc1)

In [38]:
model = GaussianNB()
model.fit(X, Y)

GaussianNB()

## Inputs  <a class="anchor" id="input"></a>

The input into the model will be based on the previous 5 games played by the home and away team using an exponential moving average prioritising most recent games providing a row of stats for both teams in the match apart from the Elo stat. For stats with for and against records, the home team's against and away team's for will be subtracted from the home team's for and away team's against. For the inside 50 efficiency, the average of the home team's inside 50 efficiency and away team's rebound 50 efficiency will be used, and vice versa for the rebound 50 efficiency.

In [39]:
elo_dict

{'Adelaide Team Statistics [Players]': 1483.8219468319353,
 'Brisbane Lions Team Statistics [Players]': 1604.5783949357838,
 'Carlton Team Statistics [Players]': 1576.7285079045912,
 'Collingwood Team Statistics [Players]': 1624.7070302435227,
 'Essendon Team Statistics [Players]': 1480.012777276485,
 'Fremantle Team Statistics [Players]': 1457.6113384805662,
 'Geelong Team Statistics [Players]': 1492.91201377354,
 'Gold Coast Team Statistics [Players]': 1467.0377257095029,
 'Greater Western Sydney Team Statistics [Players]': 1536.777818788957,
 'Hawthorn Team Statistics [Players]': 1437.8700389362386,
 'Melbourne Team Statistics [Players]': 1580.3721339473898,
 'North Melbourne Team Statistics [Players]': 1298.8580241780755,
 'Port Adelaide Team Statistics [Players]': 1592.3809519629867,
 'Richmond Team Statistics [Players]': 1487.3992866155222,
 'St Kilda Team Statistics [Players]': 1522.8292562451481,
 'Sydney Team Statistics [Players]': 1548.1129335769704,
 'West Coast Team Statist

In [80]:
Home = Car.iloc[:,4:44]
H_Elo = elo_dict['Carlton Team Statistics [Players]']

Away = GWS.iloc[:,4:44]
A_Elo = elo_dict['Greater Western Sydney Team Statistics [Players]']

In [81]:
Home = pd.DataFrame(Home).ewm(span=5).mean().iloc[len(Home)-1,:]
Away = pd.DataFrame(Away).ewm(span=5).mean().iloc[len(Away)-1,:]

In [82]:
Stats = np.zeros(n)
for i in range(0, (18)):
    Stats[i] = (Home[i*2]+Away[i*2+1]-Home[i*2+1]-Away[i*2])/2

Stats[18] = Home[18*2]*2-Away[18*2+1]
Stats[19] = Home[18*2+1]*2-Away[18*2]
Stats[20] = H_Elo-A_Elo

## Outputs <a class="anchor" id="output"></a>

The inputs of Stats will be put into the Gaussian Naive Bayes model. The desired output of the model is the following:
- Model accuracy for the week based on model testing
- Probability of the highest chance result
- Average profit over 100 games (\\$5 is bet for probabilities between 50% and 74.9%, \\$10 is bet for probabilities equal to or greater than 75%)
- What odds to play at
- Odds offered on the match

In [83]:
result = model.predict(Stats.reshape(1, -1))
p = model.predict_proba(Stats.reshape(1, -1)).max()
P = "{:.1%}".format(p)
W = W=p*100

In [84]:
print("Model accuracy this week is %s \n" % (acc))
print("Result of %s with a probability of %s \n" % (result[0], P))
print("Look for payout above $%0.2f \n" % (100/(W*acc1)))

Model accuracy this week is 66.2% 

Result of LL with a probability of 57.4% 

Look for payout above $2.63 

